# Experiments

In [85]:
import pickle
import re
import pandas as pd
import numpy as np
from data import *
from bnMLP import *
import os
from torch.utils.data import TensorDataset, DataLoader, random_split

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import torch
from sklearn.svm import SVC
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import roc_auc_score, pairwise_distances
from utils import *

In [18]:
GLOBAL_CUTOFFS = {}
for a, b in pd.read_csv('cutoffs.csv').values:
    GLOBAL_CUTOFFS[a] = b
    # break

COHORTS = ['Autoimmune: Other', 'Cancer: Other', 'HIV', 'Healthy Control',
           'IBD', 'Multiple Myeloma', 'Transplant']


In [6]:
with open('final_models.pkl', 'rb') as f:
    data = pickle.load(f)


In [8]:
results_6_excel, results_8_excel = data['dict6'], data['dict8']

In [40]:
if not os.path.exists('final_results'):
    os.makedirs('final_results')

#### Post to Post by cohort

In [10]:

columns = []
for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
    columns.append(f'Sensitivity at {t} Specificity')
for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
    columns.append(f'Specificity at {t} Sensitivity')
columns = [
    'Low Responders Accuracy',
    'High Responders Accuracy',
    'Harmonic Mean',
    'AUC'
] + columns


split_pattern = r',\s*(?![^()]*\))'

records = []
for k, v in results_8_excel.items():
    parts = re.split(split_pattern, k)
    seed, cohort, task, setting, clf, coeffs, usage = parts
    records.append({
        'seed': int(seed),
        'cohort': cohort,
        'task': task,
        'setting': setting,
        'clf': clf,
        'coeffs': coeffs,
        'usage': usage,
        **{name: val for name, val in zip(columns, v[-1])}
    })

df = pd.DataFrame(records)

group_cols = ['cohort', 'task', 'setting', 'clf', 'coeffs', 'usage']
metric_cols = columns

agg_df = df.groupby(group_cols)[metric_cols].agg(['mean', 'std']).reset_index()
agg_df.columns = ['_'.join(col).strip('_') for col in agg_df.columns.values]

# print(agg_df)


In [11]:
agg_df.head()

,cohort,task,setting,clf,coeffs,usage,Low Responders Accuracy_mean,Low Responders Accuracy_std,High Responders Accuracy_mean,High Responders Accuracy_std,...,Specificity at 0.75 Sensitivity_mean,Specificity at 0.75 Sensitivity_std,Specificity at 0.8 Sensitivity_mean,Specificity at 0.8 Sensitivity_std,Specificity at 0.85 Sensitivity_mean,Specificity at 0.85 Sensitivity_std,Specificity at 0.9 Sensitivity_mean,Specificity at 0.9 Sensitivity_std,Specificity at 0.95 Sensitivity_mean,Specificity at 0.95 Sensitivity_std
0,Autoimmune: Other,8,binary,mlp,"(4, 1)",Ab raw,0.483657,0.283375,0.535814,0.270041,...,0.236413,0.073648,0.188761,0.059383,0.129714,0.058552,0.058277,0.039342,0.040396,0.033726
1,Autoimmune: Other,8,binary,mlp,"(4, 1)",Demographic,0.521686,0.279460,0.520352,0.275685,...,0.265945,0.072609,0.222303,0.066424,0.16089,0.054670,0.072352,0.044463,0.055754,0.041882
2,Autoimmune: Other,8,binary,nc,"(4, 1)",Ab raw,0.397133,0.063008,0.624284,0.057271,...,0.220943,0.061691,0.184239,0.052999,0.130657,0.046604,0.062564,0.033670,0.043646,0.030186
3,Autoimmune: Other,8,binary,nc,"(4, 1)",Demographic,0.426038,0.067220,0.621502,0.053877,...,0.234517,0.050559,0.198028,0.050590,0.14264,0.044893,0.061371,0.032921,0.047356,0.031698
4,Autoimmune: Other,8,binary,svm,"(4, 1)",Ab raw,0.706571,0.055719,0.335949,0.052944,...,0.239227,0.055156,0.202758,0.050026,0.147002,0.046978,0.06779,0.032058,0.05465,0.032557


In [19]:
X, y , emb_list, real_size, encoder = load_6(binary=True, cutoffs=None)
counts = {}
for cohort in COHORTS:
    
    # idx = X[:, 0] == 'Healthy Control'
    idx = X[:, 0] == cohort
    
    all_low_count = (y[idx] == 0).astype(float).sum()
    all_high_count = (y[idx] == 1).astype(float).sum()
    print(all_low_count, all_high_count, cohort)
    counts[f'{cohort},Test_N_Low'] = all_low_count
    counts[f'{cohort},Train_N_Low'] = all_low_count
    counts[f'{cohort},Test_N_High'] = all_high_count
    counts[f'{cohort},Train_N_High'] = all_high_count
counts

72.0 163.0 Autoimmune: Other
9.0 102.0 Cancer: Other
14.0 70.0 HIV
24.0 465.0 Healthy Control
4.0 33.0 IBD
60.0 92.0 Multiple Myeloma
81.0 73.0 Transplant


{'Autoimmune: Other,Test_N_Low': 72.0,
 'Autoimmune: Other,Train_N_Low': 72.0,
 'Autoimmune: Other,Test_N_High': 163.0,
 'Autoimmune: Other,Train_N_High': 163.0,
 'Cancer: Other,Test_N_Low': 9.0,
 'Cancer: Other,Train_N_Low': 9.0,
 'Cancer: Other,Test_N_High': 102.0,
 'Cancer: Other,Train_N_High': 102.0,
 'HIV,Test_N_Low': 14.0,
 'HIV,Train_N_Low': 14.0,
 'HIV,Test_N_High': 70.0,
 'HIV,Train_N_High': 70.0,
 'Healthy Control,Test_N_Low': 24.0,
 'Healthy Control,Train_N_Low': 24.0,
 'Healthy Control,Test_N_High': 465.0,
 'Healthy Control,Train_N_High': 465.0,
 'IBD,Test_N_Low': 4.0,
 'IBD,Train_N_Low': 4.0,
 'IBD,Test_N_High': 33.0,
 'IBD,Train_N_High': 33.0,
 'Multiple Myeloma,Test_N_Low': 60.0,
 'Multiple Myeloma,Train_N_Low': 60.0,
 'Multiple Myeloma,Test_N_High': 92.0,
 'Multiple Myeloma,Train_N_High': 92.0,
 'Transplant,Test_N_Low': 81.0,
 'Transplant,Train_N_Low': 81.0,
 'Transplant,Test_N_High': 73.0,
 'Transplant,Train_N_High': 73.0}

In [20]:
agg_df['Train_Timepoint'] = 'Post-B'
agg_df['Test_Timepoint'] = 'Post-B'
agg_df['Test_Cohort'] = agg_df['cohort']
def update_usage(row, what):
    cohort = row['cohort']
    return counts[f'{cohort},{what}']

for what in ['Train_N_Low', 'Train_N_High', 'Test_N_High', 'Test_N_Low']:
    agg_df[what] = agg_df.apply(lambda z:update_usage(z, what=what), axis=1)
    
agg_df = agg_df.rename(columns={
    'cohort': 'Train_Cohort', 
    'clf': 'Classifier', 
    'AUC_mean': 'CV_AUC_Mean',
    'AUC_std': 'CV_AUC_SD',
    'Sensitivity at 0.8 Specificity_mean': 'CV_Sen_at_80_Spe_Mean',
    'Sensitivity at 0.8 Specificity_std': 'CV_Sen_at_80_Spe_SD',
    'Low Responders Accuracy_mean': 'CV_Acc_Low_Mean',
    'High Responders Accuracy_mean': 'CV_Acc_High_Mean',
    'Low Responders Accuracy_std': 'CV_Acc_Low_SD',
    'High Responders Accuracy_std': 'CV_Acc_High_SD',
})

def update_usage(row):
    usage = row['usage']
    setting = row['setting']
    
    # Case 1: if usage is "Ab raw"
    
    out = 'Ab_raw'
    
    # Case 2: if usage is "Demographic"
    if setting.strip().lower() == 'binary':
        out += '+ Ab_binary'
        
    if usage.strip().lower() == 'demographic':
        out += '+ demographic'
    
    # Case 3: modify further based on 'setting'
    
    if setting.strip().lower() == 'no-cutoff':
        # don't add anything
        pass
    
    return out

# Apply transformation
agg_df['Feature_Set'] = agg_df.apply(update_usage, axis=1)
agg_df[['Train_Timepoint', 'Train_Cohort', 'Train_N_Low', 'Train_N_High', 'Feature_Set', 'Test_Timepoint', 'Test_Cohort', 'Test_N_Low' ,'Test_N_High', 'Classifier',	'CV_AUC_Mean', 'CV_AUC_SD',	'CV_Sen_at_80_Spe_Mean', 'CV_Sen_at_80_Spe_SD', 'CV_Acc_Low_Mean', 'CV_Acc_Low_SD',	'CV_Acc_High_Mean',	'CV_Acc_High_SD']]



,Train_Timepoint,Train_Cohort,Train_N_Low,Train_N_High,Feature_Set,Test_Timepoint,Test_Cohort,Test_N_Low,Test_N_High,Classifier,CV_AUC_Mean,CV_AUC_SD,CV_Sen_at_80_Spe_Mean,CV_Sen_at_80_Spe_SD,CV_Acc_Low_Mean,CV_Acc_Low_SD,CV_Acc_High_Mean,CV_Acc_High_SD
0,Post-B,Autoimmune: Other,72.0,163.0,Ab_raw+ Ab_binary,Post-B,Autoimmune: Other,72.0,163.0,mlp,0.519177,0.037570,0.280638,0.057416,0.483657,0.283375,0.535814,0.270041
1,Post-B,Autoimmune: Other,72.0,163.0,Ab_raw+ Ab_binary+ demographic,Post-B,Autoimmune: Other,72.0,163.0,mlp,0.545551,0.040479,0.310752,0.070853,0.521686,0.279460,0.520352,0.275685
2,Post-B,Autoimmune: Other,72.0,163.0,Ab_raw+ Ab_binary,Post-B,Autoimmune: Other,72.0,163.0,nc,0.528434,0.039630,0.302295,0.055403,0.397133,0.063008,0.624284,0.057271
3,Post-B,Autoimmune: Other,72.0,163.0,Ab_raw+ Ab_binary+ demographic,Post-B,Autoimmune: Other,72.0,163.0,nc,0.541059,0.036547,0.320124,0.052592,0.426038,0.067220,0.621502,0.053877
4,Post-B,Autoimmune: Other,72.0,163.0,Ab_raw+ Ab_binary,Post-B,Autoimmune: Other,72.0,163.0,svm,0.531261,0.035584,0.286267,0.048841,0.706571,0.055719,0.335949,0.052944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Post-B,Transplant,81.0,73.0,Ab_raw+ demographic,Post-B,Transplant,81.0,73.0,mlp,0.552797,0.040570,0.332228,0.065942,0.564265,0.248461,0.509495,0.239451
68,Post-B,Transplant,81.0,73.0,Ab_raw,Post-B,Transplant,81.0,73.0,nc,0.510863,0.043189,0.286088,0.054508,0.520029,0.065614,0.479533,0.062681
69,Post-B,Transplant,81.0,73.0,Ab_raw+ demographic,Post-B,Transplant,81.0,73.0,nc,0.526472,0.046465,0.30361,0.068458,0.554243,0.074772,0.500381,0.059136
70,Post-B,Transplant,81.0,73.0,Ab_raw,Post-B,Transplant,81.0,73.0,svm,0.520031,0.047456,0.292162,0.067263,0.836809,0.058186,0.187333,0.058779


In [23]:
out.to_csv('final_results/model_performance_summary_v1_revised.csv')
out

,Train_Timepoint,Train_Cohort,Train_N_Low,Train_N_High,Feature_Set,Test_Timepoint,Test_Cohort,Test_N_Low,Test_N_High,Classifier,CV_AUC_Mean,CV_AUC_SD,CV_Sen_at_80_Spe_Mean,CV_Sen_at_80_Spe_SD,CV_Acc_Low_Mean,CV_Acc_Low_SD,CV_Acc_High_Mean,CV_Acc_High_SD
0,Post-B,Autoimmune: Other,72.0,163.0,Ab_raw+ Ab_binary,Post-B,Autoimmune: Other,72.0,163.0,mlp,0.519177,0.037570,0.280638,0.057416,0.483657,0.283375,0.535814,0.270041
1,Post-B,Autoimmune: Other,72.0,163.0,Ab_raw+ Ab_binary+ demographic,Post-B,Autoimmune: Other,72.0,163.0,mlp,0.545551,0.040479,0.310752,0.070853,0.521686,0.279460,0.520352,0.275685
2,Post-B,Autoimmune: Other,72.0,163.0,Ab_raw+ Ab_binary,Post-B,Autoimmune: Other,72.0,163.0,nc,0.528434,0.039630,0.302295,0.055403,0.397133,0.063008,0.624284,0.057271
3,Post-B,Autoimmune: Other,72.0,163.0,Ab_raw+ Ab_binary+ demographic,Post-B,Autoimmune: Other,72.0,163.0,nc,0.541059,0.036547,0.320124,0.052592,0.426038,0.067220,0.621502,0.053877
4,Post-B,Autoimmune: Other,72.0,163.0,Ab_raw+ Ab_binary,Post-B,Autoimmune: Other,72.0,163.0,svm,0.531261,0.035584,0.286267,0.048841,0.706571,0.055719,0.335949,0.052944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Post-B,Transplant,81.0,73.0,Ab_raw+ demographic,Post-B,Transplant,81.0,73.0,mlp,0.552797,0.040570,0.332228,0.065942,0.564265,0.248461,0.509495,0.239451
68,Post-B,Transplant,81.0,73.0,Ab_raw,Post-B,Transplant,81.0,73.0,nc,0.510863,0.043189,0.286088,0.054508,0.520029,0.065614,0.479533,0.062681
69,Post-B,Transplant,81.0,73.0,Ab_raw+ demographic,Post-B,Transplant,81.0,73.0,nc,0.526472,0.046465,0.30361,0.068458,0.554243,0.074772,0.500381,0.059136
70,Post-B,Transplant,81.0,73.0,Ab_raw,Post-B,Transplant,81.0,73.0,svm,0.520031,0.047456,0.292162,0.067263,0.836809,0.058186,0.187333,0.058779


#### Post to Post All

In [35]:
columns = []
for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
    columns.append(f'Sensitivity at {t} Specificity')
for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
    columns.append(f'Specificity at {t} Sensitivity')
columns = [
    'Low Responders Accuracy',
    'High Responders Accuracy',
    'Harmonic Mean',
    'AUC'
] + columns
for k, v in results_6_excel.items():
    seed, _, setting, clf, __, usage = re.split(r',\s*(?![^()]*\))', k)


records = []
for k, v in results_6_excel.items():
    parts = re.split(split_pattern, k)
    seed, task, setting, clf, coeffs, usage = parts
    records.append({
        'seed': int(seed),
        'cohort': 'All',
        'task': task,
        'setting': setting,
        'clf': clf,
        'coeffs': coeffs,
        'usage': usage,
        **{name: val for name, val in zip(columns, v[-1])}
    })

df = pd.DataFrame(records)

group_cols = ['task', 'cohort', 'setting', 'clf', 'coeffs', 'usage']
metric_cols = columns

agg_df = df.groupby(group_cols)[metric_cols].agg(['mean', 'std']).reset_index()
agg_df.columns = ['_'.join(col).strip('_') for col in agg_df.columns.values]
agg_df

,task,cohort,setting,clf,coeffs,usage,Low Responders Accuracy_mean,Low Responders Accuracy_std,High Responders Accuracy_mean,High Responders Accuracy_std,...,Specificity at 0.75 Sensitivity_mean,Specificity at 0.75 Sensitivity_std,Specificity at 0.8 Sensitivity_mean,Specificity at 0.8 Sensitivity_std,Specificity at 0.85 Sensitivity_mean,Specificity at 0.85 Sensitivity_std,Specificity at 0.9 Sensitivity_mean,Specificity at 0.9 Sensitivity_std,Specificity at 0.95 Sensitivity_mean,Specificity at 0.95 Sensitivity_std
0,6,All,binary,mlp,"(4, 1)",Ab raw,0.572921,0.172749,0.67822,0.162297,...,0.493206,0.038858,0.431945,0.049203,0.327982,0.051969,0.193796,0.071317,0.09531,0.057351
1,6,All,binary,mlp,"(4, 1)",Demographic,0.644741,0.169995,0.696249,0.138374,...,0.60616,0.036515,0.55136,0.043810,0.459103,0.061108,0.324699,0.080503,0.177426,0.091796
2,6,All,binary,nc,"(4, 1)",Ab raw,0.54907,0.042800,0.66208,0.055566,...,0.333472,0.051213,0.277888,0.050271,0.207805,0.040829,0.134128,0.032574,0.077983,0.023814
3,6,All,binary,nc,"(4, 1)",Demographic,0.589658,0.042097,0.713142,0.049810,...,0.38929,0.073052,0.329967,0.067699,0.250464,0.063405,0.162638,0.044944,0.093714,0.030963
4,6,All,binary,svm,"(4, 1)",Ab raw,0.691062,0.037217,0.555638,0.037663,...,0.438089,0.051454,0.366809,0.051421,0.275348,0.048729,0.177031,0.041524,0.099713,0.031732
5,6,All,binary,svm,"(4, 1)",Demographic,0.744039,0.043451,0.603672,0.035942,...,0.552353,0.070853,0.479273,0.082779,0.376729,0.086989,0.258246,0.081154,0.158006,0.063979
6,6,All,no-cutoff,mlp,"(4, 1)",Ab raw,0.584875,0.156188,0.670334,0.157725,...,0.48819,0.041704,0.431632,0.045136,0.334727,0.056983,0.204605,0.062188,0.107816,0.060860
7,6,All,no-cutoff,mlp,"(4, 1)",Demographic,0.666021,0.156551,0.715709,0.126016,...,0.648492,0.035424,0.601295,0.037576,0.522265,0.047696,0.398905,0.072268,0.239715,0.110474
8,6,All,no-cutoff,nc,"(4, 1)",Ab raw,0.547397,0.045727,0.656191,0.058674,...,0.325801,0.051683,0.27178,0.048664,0.201948,0.038854,0.129076,0.028580,0.077481,0.022917
9,6,All,no-cutoff,nc,"(4, 1)",Demographic,0.614024,0.039258,0.730845,0.045814,...,0.41372,0.089823,0.354238,0.084730,0.269227,0.073797,0.173214,0.054522,0.102504,0.038080


In [62]:
agg_df['Train_Timepoint'] = 'Post-B'
agg_df['Test_Timepoint'] = 'Post-B'
agg_df['Test_Cohort'] = 'All'


X, y , emb_list, real_size, encoder = load_6(binary=True, cutoffs=None)
counts = {}
# for cohort in COHORTS:
    
    # idx = X[:, 0] == 'Healthy Control'
    # idx = X[:, 0] == cohort
all_low_count = (y == 0).astype(float).sum()
all_high_count = (y == 1).astype(float).sum()
counts[f'Test_N_Low'] = all_low_count
counts[f'Train_N_Low'] = all_low_count
counts[f'Test_N_High'] = all_high_count
counts[f'Train_N_High'] = all_high_count


def update_usage(row, what):
    return counts[f'{what}']

# Apply transformation
for what in ['Train_N_Low', 'Train_N_High', 'Test_N_High', 'Test_N_Low']:
    agg_df[what] = agg_df.apply(lambda z:update_usage(z, what=what), axis=1)
    
agg_df = agg_df.rename(columns={
    'cohort': 'Train_Cohort', 
    'clf': 'Classifier', 
    'AUC_mean': 'CV_AUC_Mean',
    'AUC_std': 'CV_AUC_SD',
    'Sensitivity at 0.8 Specificity_mean': 'CV_Sen_at_80_Spe_Mean',
    'Sensitivity at 0.8 Specificity_std': 'CV_Sen_at_80_Spe_SD',
    'Low Responders Accuracy_mean': 'CV_Acc_Low_Mean',
    'High Responders Accuracy_mean': 'CV_Acc_High_Mean',
    'Low Responders Accuracy_std': 'CV_Acc_Low_SD',
    'High Responders Accuracy_std': 'CV_Acc_High_SD',
})

def update_usage(row):
    # Start with base usage name
    usage = row['usage']
    setting = row['setting']
    
    
    # Case 1: if usage is "Ab raw"
    
    out = 'Ab_raw'
    
    
    # Case 2: if usage is "Demographic"
    if setting.strip().lower() == 'binary':
        out += '+ Ab_binary'
        
    if usage.strip().lower() == 'demographic':
        out += '+ demographic'
    
    # Case 3: modify further based on 'setting'
    
    if setting.strip().lower() == 'no-cutoff':
        # don't add anything
        pass
    
    return out

# Apply transformation
agg_df['Feature_Set'] = agg_df.apply(update_usage, axis=1)
agg_df[['Train_Timepoint', 'Train_Cohort', 'Train_N_Low', 'Train_N_High', 'Feature_Set', 'Test_Timepoint', 'Test_Cohort', 'Test_N_Low' ,'Test_N_High', 'Classifier', 'CV_AUC_Mean', 'CV_AUC_SD',	'CV_Sen_at_80_Spe_Mean', 'CV_Sen_at_80_Spe_SD', 'CV_Acc_Low_Mean', 'CV_Acc_Low_SD',	'CV_Acc_High_Mean',	'CV_Acc_High_SD']]



,Train_Timepoint,Train_Cohort,Train_N_Low,Train_N_High,Feature_Set,Test_Timepoint,Test_Cohort,Test_N_Low,Test_N_High,Classifier,CV_AUC_Mean,CV_AUC_SD,CV_Sen_at_80_Spe_Mean,CV_Sen_at_80_Spe_SD,CV_Acc_Low_Mean,CV_Acc_Low_SD,CV_Acc_High_Mean,CV_Acc_High_SD
0,Post-B,All,264.0,998.0,Ab_raw+ Ab_binary,Post-B,All,264.0,998.0,mlp,0.684762,0.013148,0.475634,0.029027,0.572921,0.172749,0.67822,0.162297
1,Post-B,All,264.0,998.0,Ab_raw+ Ab_binary+ demographic,Post-B,All,264.0,998.0,mlp,0.74264,0.018529,0.558895,0.037587,0.644741,0.169995,0.696249,0.138374
2,Post-B,All,264.0,998.0,Ab_raw+ Ab_binary,Post-B,All,264.0,998.0,nc,0.585022,0.039592,0.329084,0.062969,0.54907,0.042800,0.66208,0.055566
3,Post-B,All,264.0,998.0,Ab_raw+ Ab_binary+ demographic,Post-B,All,264.0,998.0,nc,0.622994,0.050419,0.385864,0.080376,0.589658,0.042097,0.713142,0.049810
4,Post-B,All,264.0,998.0,Ab_raw+ Ab_binary,Post-B,All,264.0,998.0,svm,0.666246,0.017892,0.46645,0.030549,0.691062,0.037217,0.555638,0.037663
5,Post-B,All,264.0,998.0,Ab_raw+ Ab_binary+ demographic,Post-B,All,264.0,998.0,svm,0.721375,0.023013,0.548728,0.032280,0.744039,0.043451,0.603672,0.035942
6,Post-B,All,264.0,998.0,Ab_raw,Post-B,All,264.0,998.0,mlp,0.687912,0.020413,0.480037,0.037763,0.584875,0.156188,0.670334,0.157725
7,Post-B,All,264.0,998.0,Ab_raw+ demographic,Post-B,All,264.0,998.0,mlp,0.765951,0.017666,0.594364,0.035159,0.666021,0.156551,0.715709,0.126016
8,Post-B,All,264.0,998.0,Ab_raw,Post-B,All,264.0,998.0,nc,0.580031,0.041890,0.326875,0.067398,0.547397,0.045727,0.656191,0.058674
9,Post-B,All,264.0,998.0,Ab_raw+ demographic,Post-B,All,264.0,998.0,nc,0.640067,0.052502,0.416181,0.083975,0.614024,0.039258,0.730845,0.045814


In [63]:
out = agg_df[['Train_Timepoint', 'Train_Cohort', 'Train_N_Low', 'Train_N_High', 'Feature_Set', 'Test_Timepoint', 'Test_Cohort', 'Test_N_Low' ,'Test_N_High', 'Classifier',	'CV_AUC_Mean', 'CV_AUC_SD',	'CV_Sen_at_80_Spe_Mean', 'CV_Sen_at_80_Spe_SD', 'CV_Acc_Low_Mean', 'CV_Acc_Low_SD',	'CV_Acc_High_Mean',	'CV_Acc_High_SD']]
out.to_csv('final_results/model_performance_summary_v2_revised.csv')

#### Post to Post from All to cohort

In [73]:
metrics = {}
device = 'cuda' if torch.cuda.is_available() else 'cpu'

for k, v in results_6_excel.items():
    seed, task, setting, clf, coeffs, usage = re.split(r',\s*(?![^()]*\))', k)

    seed = int(seed)
    if setting == 'binary':
        cutoffs = GLOBAL_CUTOFFS
    else:
        cutoffs = None

    if 'raw' in usage:
        use_cat = False
    else:
        use_cat = True
    if use_cat:
            start_from = len(emb_list)
    else:
        start_from = 11
        
        
    
    X, y , emb_list, real_size, encoder = load_6(binary=True, cutoffs=cutoffs)
    size = real_size if use_cat else real_size - 1
    
    en = {}
    for k_cohort, v_cohort in (X[:, [0, 5]]):
        en[k_cohort] = v_cohort
     
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    
    
    for_stratify = y
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, for_stratify), start=0):
        #################################################### Dataset
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        
        X_train_tensor = torch.from_numpy(X_train[:, start_from:].astype(np.float32))
        y_train_tensor = torch.from_numpy(y_train).float()
        
        X_test_tensor = torch.from_numpy(X_test[:, start_from:].astype(np.float32))
        y_test_tensor = torch.from_numpy(y_test).float()
        
        
        train_idx, val_idx = train_test_split(
            np.arange(len(X_train_tensor)),
            test_size=0.2,
            stratify=y_train_tensor.numpy() ,
            random_state=seed
        )
        
        # create datasets
        train_dataset = TensorDataset(X_train_tensor[train_idx], y_train_tensor[train_idx])
        val_dataset   = TensorDataset(X_train_tensor[val_idx],  y_train_tensor[val_idx])
        
        # data loaders
        train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, generator=torch.Generator().manual_seed(seed))
        val_loader   = DataLoader(val_dataset,   batch_size=len(val_dataset), shuffle=False)
        test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor),
                                 batch_size=len(X_test_tensor), shuffle=False)
        
        m = BottleneckMLP(emb_list, size, start_from, 8, (128, 64, 2), p=0.2, use_cat=use_cat, binary=True).to(device)
        
        m.load_state_dict(v[0][fold])
        
        
        for cohort in en.keys():
            for x_batch, y_batch in test_loader:
                pass
            
            index = X_test[:, 5] == en[cohort]
            
            
            
            if (y_batch[index] == 0).float().sum() < 1:
                print(cohort, fold, seed, 'not enough negative samples')
                continue
            if clf == 'mlp':
                with torch.no_grad():
                    m.eval()
                    logits, two_d = m(x_batch[index].to(device))
            else:
                with torch.no_grad():
                    x_for_clf = []
                    y_for_clf = []
                    for x_batch_train, y_batch_train in train_loader:
                        with torch.no_grad():
                            m.eval()
                            _, two_d = m(x_batch_train.to(device))
                        x_for_clf.extend(two_d)
                        y_for_clf.extend(y_batch_train)
                x_for_clf = torch.stack(x_for_clf).cpu().numpy()
                y_for_clf = torch.stack(y_for_clf).cpu().numpy()
                if clf == 'svm':
                    one = (y_for_clf == 1).astype(float).sum()
                    zero = (y_for_clf == 0).astype(float).sum()
                    w = (one/zero)
                    delta = 1
        
                    classifier = SVC(kernel="rbf", class_weight={0: w, 1: 1}, probability=True)
                    classifier.fit(x_for_clf, y_for_clf)
                    with torch.no_grad():
                        m.eval()
                        x_in = m(x_batch[index].to(device))[-1].cpu().numpy()
                    logits = classifier.decision_function(x_in)
                elif clf == 'nc':
                    classifier = NearestCentroid(metric="euclidean")
                    classifier.fit(x_for_clf, y_for_clf)
                    with torch.no_grad():
                        m.eval()
                        x_in = m(x_batch[index].to(device))[-1].cpu().numpy()
                    logits = classifier.predict(x_in)

            if 1:
                if clf == 'mlp':
                    y_pred = (logits.reshape(-1) > 0.5).int().cpu()
                elif clf == 'svm':
                    y_pred = torch.from_numpy(logits >= delta).int()
                elif clf == 'nc':
                    
                    y_pred = torch.from_numpy(logits).int()
                a_test = (y_pred[y_batch[index]==0] == y_batch[index][y_batch[index]==0]).float().mean().item() + 1e-20
                b_test = (y_pred[y_batch[index]==1] == y_batch[index][y_batch[index]==1]).float().mean().item() + 1e-20
                c_test = 2 / ((1/a_test) + 1/b_test)
                if clf == 'svm':
                    logits = torch.sigmoid(torch.from_numpy(logits))
                elif clf == 'nc':
                    D = pairwise_distances(x_in, classifier.centroids_, metric=classifier.metric)
                    pos_idx = np.where(classifier.classes_ == 1)[0][0]
                    logits = torch.from_numpy(-D[:, pos_idx])
                    

                try:
                    auc_test = roc_auc_score(y_batch[index].cpu().numpy(), logits.cpu().numpy())
                except:
                    auc_test = None
                sen_test = []
                spec_test = []
                for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
                    sen_, spec_ = at_95(logits.cpu().numpy(), y_batch[index], t=t)
                    sen_test.append(sen_)
                    spec_test.append(spec_)
                    
            
            metrics[f'{seed}, {fold}, {cohort}, {clf}, {setting}, {usage}'] = ([a_test, b_test, c_test, auc_test, *spec_test, *sen_test])                        
    
        # print('-----------------')

IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 1 0 not enough negative samples
Cancer: Other 3 0 no

In [74]:
columns = []
for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
    columns.append(f'Sensitivity at {t} Specificity')
for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
    columns.append(f'Specificity at {t} Sensitivity')
columns = [
    'Low Responders Accuracy',
    'High Responders Accuracy',
    'Harmonic Mean',
    'AUC'
] + columns



records = []
for k, v in metrics.items():
    seed, fold, cohort, clf, setting, usage = k.split(', ')
    records.append({
        'seed': int(seed),
        'fold': int(fold),
        'cohort': cohort,
        'setting': setting,
        'clf': clf,
        'usage': usage,
        **{name: val for name, val in zip(columns, v)}
    })

df = pd.DataFrame(records)

group_cols = ['cohort', 'setting', 'clf', 'usage']
metric_cols = columns

agg_df = df.groupby(group_cols)[metric_cols].agg(['mean', 'std']).reset_index()
agg_df.columns = ['_'.join(col).strip('_') for col in agg_df.columns.values]
agg_df

,cohort,setting,clf,usage,Low Responders Accuracy_mean,Low Responders Accuracy_std,High Responders Accuracy_mean,High Responders Accuracy_std,Harmonic Mean_mean,Harmonic Mean_std,...,Specificity at 0.75 Sensitivity_mean,Specificity at 0.75 Sensitivity_std,Specificity at 0.8 Sensitivity_mean,Specificity at 0.8 Sensitivity_std,Specificity at 0.85 Sensitivity_mean,Specificity at 0.85 Sensitivity_std,Specificity at 0.9 Sensitivity_mean,Specificity at 0.9 Sensitivity_std,Specificity at 0.95 Sensitivity_mean,Specificity at 0.95 Sensitivity_std
0,Autoimmune: Other,binary,mlp,Ab raw,0.411970,0.214082,0.641123,0.190020,0.430876,0.139752,...,0.254194,0.165143,0.201745,0.161398,0.135531,0.142165,0.068666,0.101490,0.037739,0.072980
1,Autoimmune: Other,binary,mlp,Demographic,0.502758,0.229264,0.557126,0.216069,0.442959,0.120777,...,0.261500,0.148433,0.209888,0.141727,0.153826,0.133901,0.078091,0.103941,0.044225,0.079710
2,Autoimmune: Other,binary,nc,Ab raw,0.398444,0.146331,0.612073,0.132745,0.455480,0.108868,...,0.247546,0.141659,0.202514,0.137624,0.150158,0.120026,0.088302,0.097450,0.050303,0.079010
3,Autoimmune: Other,binary,nc,Demographic,0.410567,0.157251,0.604165,0.142220,0.455383,0.111975,...,0.251254,0.142051,0.205882,0.138295,0.153986,0.124884,0.090883,0.100632,0.055155,0.088952
4,Autoimmune: Other,binary,svm,Ab raw,0.532816,0.152287,0.514478,0.118632,0.499002,0.092706,...,0.239225,0.139107,0.202429,0.135696,0.150252,0.121539,0.089552,0.098510,0.049200,0.078740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Transplant,no-cutoff,mlp,Demographic,0.765108,0.206998,0.255960,0.223688,0.298718,0.197879,...,0.212774,0.143177,0.172576,0.131856,0.131975,0.122562,0.082330,0.100566,0.041262,0.077338
80,Transplant,no-cutoff,nc,Ab raw,0.491249,0.142405,0.520774,0.155175,0.477961,0.102841,...,0.220767,0.152171,0.175905,0.142369,0.125856,0.124716,0.069371,0.091697,0.036853,0.072140
81,Transplant,no-cutoff,nc,Demographic,0.733370,0.147936,0.281284,0.160980,0.368691,0.156690,...,0.207373,0.146723,0.165863,0.135454,0.123736,0.121610,0.070506,0.097855,0.033010,0.071034
82,Transplant,no-cutoff,svm,Ab raw,0.673124,0.148648,0.368150,0.159461,0.442649,0.129051,...,0.226436,0.144228,0.184426,0.137748,0.132142,0.126009,0.071771,0.096377,0.037780,0.075940


In [75]:
X, y , emb_list, real_size, encoder = load_6(binary=True, cutoffs=None)
counts = {}
for cohort in COHORTS:
    
    
    all_low_count_train = (y == 0).astype(float).sum()
    all_high_count_train = (y == 1).astype(float).sum()
    idx = X[:, 0] == cohort
    all_low_count = (y[idx] == 0).astype(float).sum()
    all_high_count = (y[idx] == 1).astype(float).sum()
    counts[f'{cohort},Test_N_Low'] = all_low_count
    counts[f'All,Train_N_Low'] = all_low_count_train
    counts[f'{cohort},Test_N_High'] = all_high_count
    counts[f'All,Train_N_High'] = all_high_count_train
counts

{'Autoimmune: Other,Test_N_Low': 72.0,
 'All,Train_N_Low': 264.0,
 'Autoimmune: Other,Test_N_High': 163.0,
 'All,Train_N_High': 998.0,
 'Cancer: Other,Test_N_Low': 9.0,
 'Cancer: Other,Test_N_High': 102.0,
 'HIV,Test_N_Low': 14.0,
 'HIV,Test_N_High': 70.0,
 'Healthy Control,Test_N_Low': 24.0,
 'Healthy Control,Test_N_High': 465.0,
 'IBD,Test_N_Low': 4.0,
 'IBD,Test_N_High': 33.0,
 'Multiple Myeloma,Test_N_Low': 60.0,
 'Multiple Myeloma,Test_N_High': 92.0,
 'Transplant,Test_N_Low': 81.0,
 'Transplant,Test_N_High': 73.0}

In [81]:
agg_df['Train_Timepoint'] = 'Post-B'
agg_df['Test_Timepoint'] = 'Post-B'

agg_df['Test_Cohort'] = agg_df['cohort']
agg_df['Train_Cohort'] = 'All'



def update_usage(row, what):
    cohort = row['cohort']
    try:
        return counts[f'{cohort},{what}']
    except:
        return counts[f'All,{what}']

# Apply transformation
for what in ['Train_N_Low', 'Train_N_High', 'Test_N_High', 'Test_N_Low']:
    agg_df[what] = agg_df.apply(lambda z:update_usage(z, what=what), axis=1)
    
    
agg_df = agg_df.rename(columns={
    # 'cohort': 'Train_Cohort', 
    'clf': 'Classifier', 
    'AUC_mean': 'CV_AUC_Mean',
    'AUC_std': 'CV_AUC_SD',
    'Sensitivity at 0.8 Specificity_mean': 'CV_Sen_at_80_Spe_Mean',
    'Sensitivity at 0.8 Specificity_std': 'CV_Sen_at_80_Spe_SD',
    'Low Responders Accuracy_mean': 'CV_Acc_Low_Mean',
    'High Responders Accuracy_mean': 'CV_Acc_High_Mean',
    'Low Responders Accuracy_std': 'CV_Acc_Low_SD',
    'High Responders Accuracy_std': 'CV_Acc_High_SD',
})

def update_usage(row):
    # Start with base usage name
    usage = row['usage']
    setting = row['setting']
    
    
    # Case 1: if usage is "Ab raw"
    
    out = 'Ab_raw'
    
    
    # Case 2: if usage is "Demographic"
    if setting.strip().lower() == 'binary':
        out += '+ Ab_binary'
        
    if usage.strip().lower() == 'demographic':
        out += '+ demographic'
    
    # Case 3: modify further based on 'setting'
    
    if setting.strip().lower() == 'no-cutoff':
        # don't add anything
        pass
    
    return out

# Apply transformation
agg_df['Feature_Set'] = agg_df.apply(update_usage, axis=1)
agg_df[['Train_Timepoint', 'Train_Cohort', 'Train_N_Low', 'Train_N_High', 'Feature_Set', 'Test_Timepoint', 'Test_Cohort', 'Test_N_Low' ,'Test_N_High', 'Classifier', 'CV_AUC_Mean', 'CV_AUC_SD',	'CV_Sen_at_80_Spe_Mean', 'CV_Sen_at_80_Spe_SD', 'CV_Acc_Low_Mean', 'CV_Acc_Low_SD',	'CV_Acc_High_Mean',	'CV_Acc_High_SD']]



,Train_Timepoint,Train_Cohort,Train_N_Low,Train_N_High,Feature_Set,Test_Timepoint,Test_Cohort,Test_N_Low,Test_N_High,Classifier,CV_AUC_Mean,CV_AUC_SD,CV_Sen_at_80_Spe_Mean,CV_Sen_at_80_Spe_SD,CV_Acc_Low_Mean,CV_Acc_Low_SD,CV_Acc_High_Mean,CV_Acc_High_SD
0,Post-B,All,264.0,998.0,Ab_raw+ Ab_binary,Post-B,Autoimmune: Other,72.0,163.0,mlp,0.538900,0.093524,0.307631,0.131976,0.411970,0.214082,0.641123,0.190020
1,Post-B,All,264.0,998.0,Ab_raw+ Ab_binary+ demographic,Post-B,Autoimmune: Other,72.0,163.0,mlp,0.545352,0.091868,0.323665,0.141418,0.502758,0.229264,0.557126,0.216069
2,Post-B,All,264.0,998.0,Ab_raw+ Ab_binary,Post-B,Autoimmune: Other,72.0,163.0,nc,0.561573,0.101163,0.357184,0.147727,0.398444,0.146331,0.612073,0.132745
3,Post-B,All,264.0,998.0,Ab_raw+ Ab_binary+ demographic,Post-B,Autoimmune: Other,72.0,163.0,nc,0.557160,0.097453,0.348004,0.143153,0.410567,0.157251,0.604165,0.142220
4,Post-B,All,264.0,998.0,Ab_raw+ Ab_binary,Post-B,Autoimmune: Other,72.0,163.0,svm,0.536881,0.095281,0.311282,0.137500,0.532816,0.152287,0.514478,0.118632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Post-B,All,264.0,998.0,Ab_raw+ demographic,Post-B,Transplant,81.0,73.0,mlp,0.517892,0.102440,0.250500,0.193661,0.765108,0.206998,0.255960,0.223688
80,Post-B,All,264.0,998.0,Ab_raw,Post-B,Transplant,81.0,73.0,nc,0.537281,0.105351,0.327033,0.150193,0.491249,0.142405,0.520774,0.155175
81,Post-B,All,264.0,998.0,Ab_raw+ demographic,Post-B,Transplant,81.0,73.0,nc,0.520133,0.105473,0.308289,0.141439,0.733370,0.147936,0.281284,0.160980
82,Post-B,All,264.0,998.0,Ab_raw,Post-B,Transplant,81.0,73.0,svm,0.528116,0.105473,0.308376,0.161778,0.673124,0.148648,0.368150,0.159461


In [82]:
out = agg_df[['Train_Timepoint', 'Train_Cohort', 'Train_N_Low', 'Train_N_High', 'Feature_Set', 'Test_Timepoint', 'Test_Cohort', 'Test_N_Low' ,'Test_N_High', 'Classifier',	'CV_AUC_Mean', 'CV_AUC_SD',	'CV_Sen_at_80_Spe_Mean', 'CV_Sen_at_80_Spe_SD', 'CV_Acc_Low_Mean', 'CV_Acc_Low_SD',	'CV_Acc_High_Mean',	'CV_Acc_High_SD']]
out.to_csv('final_results/model_performance_summary_v3_revised.csv')

In [312]:
best_for_each_cohort = {'Multiple Myeloma': ('mlp', 'no-cutoff'),
 'Autoimmune: Other': ('mlp', 'no-cutoff'),
 'Healthy Control': ('mlp', 'no-cutoff'),
 'Cancer: Other': ('mlp', 'no-cutoff'),
 'HIV': ('mlp', 'no-cutoff'),
 'Transplant': ('mlp', 'no-cutoff'),
 'IBD': ('mlp', 'no-cutoff')
                       }

In [846]:
# metrics = {}
# this is for 8
out_auc = {}

for k , v in results_8_excel.items():
    split_pattern = r',\s*(?![^()]*\))'
    
    
    seed, cohort, task, setting, clf, _, demographic = re.split(split_pattern, k)
    if setting == 'binary':
        continue
    if demographic != 'Demographic':
        continue

    if clf != 'mlp':
        continue

    
    seed = int(seed)
    
    if setting == 'binary':
        cutoffs = GLOBAL_CUTOFFS
    else:
        cutoffs = None
    X, y , emb_list, real_size, encoder = load_6(binary=True, cutoffs=cutoffs, qcut=False)
    idx = X[:, 0] == cohort
    
    X = X[idx]
    y = y[idx]
    
    en = {}
    for k_cohort, v_cohort in (X[:, [0, 5]]):
        en[k_cohort] = v_cohort
     
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    
    
    for_stratify = y
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, for_stratify), start=0):
        #################################################### Dataset
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        from torch.utils.data import TensorDataset, DataLoader, random_split
        
        start_from = len(emb_list)
        size = real_size 
        X_train_tensor = torch.from_numpy(X_train[:, start_from:].astype(np.float32))
        y_train_tensor = torch.from_numpy(y_train).float()
        
        X_test_tensor = torch.from_numpy(X_test[:, start_from:].astype(np.float32))
        y_test_tensor = torch.from_numpy(y_test).float()
        
        from sklearn.model_selection import train_test_split
        # stratified split using sklearn
        train_idx, val_idx = train_test_split(
            np.arange(len(X_train_tensor)),
            test_size=0.2,
            stratify=y_train_tensor.numpy() ,
            random_state=seed
        )
        
        # create datasets
        train_dataset = TensorDataset(X_train_tensor[train_idx], y_train_tensor[train_idx])
        val_dataset   = TensorDataset(X_train_tensor[val_idx],  y_train_tensor[val_idx])
        
        # data loaders
        train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, generator=torch.Generator().manual_seed(seed))
        val_loader   = DataLoader(val_dataset,   batch_size=len(val_dataset), shuffle=False)
        test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor),
                                 batch_size=len(X_test_tensor), shuffle=False)
        m = BottleneckMLP(emb_list, real_size, len(emb_list), 8, (128, 64, 2), p=0.2, use_cat=True, binary=True).to(device)
        m.load_state_dict(v[0][fold])
        
        for cohort in en.keys():
            
            if best_for_each_cohort[cohort] != (clf, setting):
                continue
            else:
                for x_batch, y_batch in test_loader:
                    pass
                index = x_batch[:, 0] == en[cohort]
                if (y_batch[index] == 0).float().sum() < 1:
                    print(cohort, fold, seed, 'not enough negative samples')
                    continue
                if clf == 'mlp':
                    with torch.no_grad():
                        m.eval()
                        logits, two_d = m(x_batch[index].to(device))
                else:
                    with torch.no_grad():
                        x_for_clf = []
                        y_for_clf = []
                        for x_batch_train, y_batch_train in train_loader:
                            with torch.no_grad():
                                m.eval()
                                _, two_d = m(x_batch_train.to(device))
                            x_for_clf.extend(two_d)
                            y_for_clf.extend(y_batch_train)
                    x_for_clf = torch.stack(x_for_clf).cpu().numpy()
                    y_for_clf = torch.stack(y_for_clf).cpu().numpy()
                    if clf == 'svm':
                        one = (y_for_clf == 1).astype(float).sum()
                        zero = (y_for_clf == 0).astype(float).sum()
                        w = (one/zero)
                        delta = 1
            
                        classifier = SVC(kernel="rbf", class_weight={0: w, 1: 1}, probability=True)
                        classifier.fit(x_for_clf, y_for_clf)
                        with torch.no_grad():
                            m.eval()
                            x_in = m(x_batch[index].to(device))[-1].cpu().numpy()
                        logits = classifier.decision_function(x_in)
                    elif clf == 'nc':
                        classifier = NearestCentroid(metric="euclidean")
                        classifier.fit(x_for_clf, y_for_clf)
                        with torch.no_grad():
                            m.eval()
                            x_in = m(x_batch[index].to(device))[-1].cpu().numpy()
                        logits = classifier.predict(x_in)
    
                if 1:
                    if clf == 'mlp':
                        y_pred = (logits.reshape(-1) > 0.5).int().cpu()
                    elif clf == 'svm':
                        y_pred = torch.from_numpy(logits >= delta).int()
                    elif clf == 'nc':
                        
                        y_pred = torch.from_numpy(logits).int()
                    a_test = (y_pred[y_batch[index]==0] == y_batch[index][y_batch[index]==0]).float().mean().item() + 1e-20
                    b_test = (y_pred[y_batch[index]==1] == y_batch[index][y_batch[index]==1]).float().mean().item() + 1e-20
                    c_test = 2 / ((1/a_test) + 1/b_test)
                    if clf == 'svm':
                        logits = torch.sigmoid(torch.from_numpy(logits))
                    elif clf == 'nc':
                        D = pairwise_distances(x_in, classifier.centroids_, metric=classifier.metric)
                        pos_idx = np.where(classifier.classes_ == 1)[0][0]
                        logits = torch.from_numpy(-D[:, pos_idx])
                        # logits = torch.sigmoid(torch.from_numpy(logits))
    
                    try:
                        auc_test = roc_auc_score(y_batch[index].cpu().numpy(), logits.cpu().numpy())
                        # print(y_batch[index])
                        # print(logits)
                        out_auc[f'{cohort},{clf},{setting},{fold},{seed}'] = (y_batch[index].cpu().numpy(), logits.cpu().numpy())
                        # print(cohort)
                        # print(clf)
                        # print(setting)
                        # print(fold)
                        # print(seed)
                        # # raise
                        # fpr, tpr, thr = roc_curve(y_batch[index].cpu().numpy(), logits.cpu().numpy())
                        # roc_auc = auc(fpr, tpr)
                        
                        # RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name="LogReg").plot()
                        # plt.title(f"ROC curve (AUC = {roc_auc:.3f})")
                        # plt.show()
                        # # raise
                    except:
                        auc_test = None
                    # raise
                    sen_test = []
                    spec_test = []
                    for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
                        sen_, spec_ = at_95(logits.cpu().numpy(), y_batch[index], t=t)
                        sen_test.append(sen_)
                        spec_test.append(spec_)
                    
            
            # metrics[f'{seed}, {fold}, {cohort}, {clf}, {setting}'] = ([a_test, b_test, c_test, auc_test, *spec_test, *sen_test])                        
    


In [848]:
to_json = {}
for k, v in out_auc.items():
    to_json[k] = (v[0].reshape(-1).tolist(), v[1].reshape(-1).tolist())

# out_auc
# import json
with open('data_for_auc_mlp-no cutoff-cohort specific-4,1-demographic-post to post.json', 'w') as fp:
    json.dump(to_json,fp)

In [849]:
# metrics = {}
out_auc = {}

for k , v in results_6_excel.items():
    
    split_pattern = r',\s*(?![^()]*\))'
    
    
    seed, task, setting, clf, _, demographic = re.split(split_pattern, k)
    
    
    seed = int(seed)

    if setting == 'binary':
        continue
    if demographic != 'Demographic':
        continue

    if clf != 'mlp':
        continue
    
    if setting == 'binary':
        cutoffs = GLOBAL_CUTOFFS
    else:
        cutoffs = None
    X, y , emb_list, real_size, encoder = load_6(binary=True, cutoffs=cutoffs, qcut=False)
    en = {}
    for k_cohort, v_cohort in (X[:, [0, 5]]):
        en[k_cohort] = v_cohort
     
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    
    
    for_stratify = y
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, for_stratify), start=0):
        #################################################### Dataset
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        from torch.utils.data import TensorDataset, DataLoader, random_split
        
        start_from = len(emb_list)
        size = real_size 
        X_train_tensor = torch.from_numpy(X_train[:, start_from:].astype(np.float32))
        y_train_tensor = torch.from_numpy(y_train).float()
        
        X_test_tensor = torch.from_numpy(X_test[:, start_from:].astype(np.float32))
        y_test_tensor = torch.from_numpy(y_test).float()
        
        from sklearn.model_selection import train_test_split
        # stratified split using sklearn
        train_idx, val_idx = train_test_split(
            np.arange(len(X_train_tensor)),
            test_size=0.2,
            stratify=y_train_tensor.numpy() ,
            random_state=seed
        )
        
        # create datasets
        train_dataset = TensorDataset(X_train_tensor[train_idx], y_train_tensor[train_idx])
        val_dataset   = TensorDataset(X_train_tensor[val_idx],  y_train_tensor[val_idx])
        
        # data loaders
        train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, generator=torch.Generator().manual_seed(seed))
        val_loader   = DataLoader(val_dataset,   batch_size=len(val_dataset), shuffle=False)
        test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor),
                                 batch_size=len(X_test_tensor), shuffle=False)
        m = BottleneckMLP(emb_list, real_size, len(emb_list), 8, (128, 64, 2), p=0.2, use_cat=True, binary=True).to(device)
        m.load_state_dict(v[0][fold])
        
        for cohort in en.keys():
            
            if best_for_each_cohort[cohort] != (clf, setting):
                continue
            else:
                for x_batch, y_batch in test_loader:
                    pass
                index = x_batch[:, 0] == en[cohort]
                if (y_batch[index] == 0).float().sum() < 1:
                    print(cohort, fold, seed, 'not enough negative samples')
                    continue
                if clf == 'mlp':
                    with torch.no_grad():
                        m.eval()
                        logits, two_d = m(x_batch[index].to(device))
                else:
                    with torch.no_grad():
                        x_for_clf = []
                        y_for_clf = []
                        for x_batch_train, y_batch_train in train_loader:
                            with torch.no_grad():
                                m.eval()
                                _, two_d = m(x_batch_train.to(device))
                            x_for_clf.extend(two_d)
                            y_for_clf.extend(y_batch_train)
                    x_for_clf = torch.stack(x_for_clf).cpu().numpy()
                    y_for_clf = torch.stack(y_for_clf).cpu().numpy()
                    if clf == 'svm':
                        one = (y_for_clf == 1).astype(float).sum()
                        zero = (y_for_clf == 0).astype(float).sum()
                        w = (one/zero)
                        delta = 1
            
                        classifier = SVC(kernel="rbf", class_weight={0: w, 1: 1}, probability=True)
                        classifier.fit(x_for_clf, y_for_clf)
                        with torch.no_grad():
                            m.eval()
                            x_in = m(x_batch[index].to(device))[-1].cpu().numpy()
                        logits = classifier.decision_function(x_in)
                    elif clf == 'nc':
                        classifier = NearestCentroid(metric="euclidean")
                        classifier.fit(x_for_clf, y_for_clf)
                        with torch.no_grad():
                            m.eval()
                            x_in = m(x_batch[index].to(device))[-1].cpu().numpy()
                        logits = classifier.predict(x_in)
    
                if 1:
                    if clf == 'mlp':
                        y_pred = (logits.reshape(-1) > 0.5).int().cpu()
                    elif clf == 'svm':
                        y_pred = torch.from_numpy(logits >= delta).int()
                    elif clf == 'nc':
                        
                        y_pred = torch.from_numpy(logits).int()
                    a_test = (y_pred[y_batch[index]==0] == y_batch[index][y_batch[index]==0]).float().mean().item() + 1e-20
                    b_test = (y_pred[y_batch[index]==1] == y_batch[index][y_batch[index]==1]).float().mean().item() + 1e-20
                    c_test = 2 / ((1/a_test) + 1/b_test)
                    if clf == 'svm':
                        logits = torch.sigmoid(torch.from_numpy(logits))
                    elif clf == 'nc':
                        D = pairwise_distances(x_in, classifier.centroids_, metric=classifier.metric)
                        pos_idx = np.where(classifier.classes_ == 1)[0][0]
                        logits = torch.from_numpy(-D[:, pos_idx])
                        # logits = torch.sigmoid(torch.from_numpy(logits))
    
                    try:
                        auc_test = roc_auc_score(y_batch[index].cpu().numpy(), logits.cpu().numpy())
                        # print(y_batch[index])
                        # print(logits)
                        out_auc[f'{cohort},{clf},{setting},{fold},{seed}'] = (y_batch[index].cpu().numpy(), logits.cpu().numpy())
                        # print(cohort)
                        # print(clf)
                        # print(setting)
                        # print(fold)
                        # print(seed)
                        # # raise
                        # fpr, tpr, thr = roc_curve(y_batch[index].cpu().numpy(), logits.cpu().numpy())
                        # roc_auc = auc(fpr, tpr)
                        
                        # RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name="LogReg").plot()
                        # plt.title(f"ROC curve (AUC = {roc_auc:.3f})")
                        # plt.show()
                        # # raise
                    except:
                        auc_test = None
                    # raise
                    sen_test = []
                    spec_test = []
                    for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
                        sen_, spec_ = at_95(logits.cpu().numpy(), y_batch[index], t=t)
                        sen_test.append(sen_)
                        spec_test.append(spec_)
                    
            
            # metrics[f'{seed}, {fold}, {cohort}, {clf}, {setting}'] = ([a_test, b_test, c_test, auc_test, *spec_test, *sen_test])                        
    


IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 0 1 not enough negative samples
Cancer: Other 1 1 not enough negative samples
IBD 1 1 not enough negative samples
HIV 4 1 not enough negative samples
IBD 0 2 not enough negative samples
IBD 1 2 not enough negative samples
Cancer: Other 2 2 not enough negative samples
Cancer: Other 1 3 not enough negative samples
IBD 2 3 not enough negative samples
Cancer: Other 3 3 not enough negative samples
IBD 4 3 not enough negative samples
IBD 3 4 not enough negative samples
Cancer: Other 1 5 not enough negative samples
IBD 2 5 not enough negative samples
Cancer: Other 0 6 not enough negative samples
IBD 2 6 not enough negative samples
IBD 3 6 not enough negative samples
IBD 1 7 not enough negative samples
IBD 4 7 not enough negative samples
IBD 0 8 not enough negative samples
Cancer: Other 1 8 not enough negative samples
IBD 2 8 not enough negative samples
HIV 3 8 not enough n

In [826]:
out_auc['Multiple Myeloma,mlp,no-cutoff,0,0'][0]

array([1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

In [850]:
to_json = {}
for k, v in out_auc.items():
    to_json[k] = (v[0].reshape(-1).tolist(), v[1].reshape(-1).tolist())

In [851]:
out_auc
import json
with open('data_for_auc_mlp-no cutoff-universal-4,1-demographic-post to post.json', 'w') as fp:
    json.dump(to_json,fp)

In [861]:
# metrics = {}
out_auc = {}

for k, v in results_6_excel.items():
    # print(k)
    # raise
    seed, task, setting, clf, coeffs, usage = re.split(r',\s*(?![^()]*\))', k)
    # print(usage)
    seed = int(seed)
    if setting == 'binary':
        continue
    if usage != 'Demographic':
        continue

    if clf != 'mlp':
        continue

    # print(seed, task, setting, clf, coeffs, usage)
    if setting == 'binary':
        cutoffs = GLOBAL_CUTOFFS
    else:
        cutoffs = None

    if 'raw' in usage:
        use_cat = False
    else:
        use_cat = True
    if use_cat:
            start_from = len(emb_list)
    else:
        start_from = 11
        
    
    X6, y6 , pid6, emb_list6, real_size6, encoder6 = load_6(binary=True, cutoffs=cutoffs, qcut=False, include_pid=True)
    
    X2, y2 , pid2, emb_list2, real_size2, encoder2 = load_2(binary=True, cutoffs=cutoffs, qcut=False, include_pid=True)
    
    size = real_size6 if use_cat else real_size6 - 1
    
    en = {}
    for k_cohort, v_cohort in (X6[:, [0, 5]]):
        en[k_cohort] = v_cohort
     
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    
    
    for_stratify = y6
    for fold, (train_idx, test_idx) in enumerate(skf.split(X6, for_stratify), start=0):
        
        index = np.isin(pid2, pid6[test_idx])
        
        X_train = X6[train_idx]
        y_train = y6[train_idx]
        X_test = X2[index]
        y_test = y2[index]
        
        
        
        #################################################### Dataset
        # X_train, X_test = X6[train_idx], X6[test_idx]
        # y_train, y_test = y6[train_idx], y6[test_idx]
        from torch.utils.data import TensorDataset, DataLoader, random_split
        
        # start_from = len(emb_list)
        # size = real_size 
        X_train_tensor = torch.from_numpy(X_train[:, start_from:].astype(np.float32))
        y_train_tensor = torch.from_numpy(y_train).float()
        
        X_test_tensor = torch.from_numpy(X_test[:, start_from:].astype(np.float32))
        y_test_tensor = torch.from_numpy(y_test).float()
        
        from sklearn.model_selection import train_test_split
        # stratified split using sklearn
        train_idx, val_idx = train_test_split(
            np.arange(len(X_train_tensor)),
            test_size=0.2,
            stratify=y_train_tensor.numpy() ,
            random_state=seed
        )
        
        # create datasets
        train_dataset = TensorDataset(X_train_tensor[train_idx], y_train_tensor[train_idx])
        val_dataset   = TensorDataset(X_train_tensor[val_idx],  y_train_tensor[val_idx])
        
        # data loaders
        train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, generator=torch.Generator().manual_seed(seed))
        val_loader   = DataLoader(val_dataset,   batch_size=len(val_dataset), shuffle=False)
        test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor),
                                 batch_size=len(X_test_tensor), shuffle=False)
        # print(size, start_from)
        m = BottleneckMLP(emb_list, size, start_from, 8, (128, 64, 2), p=0.2, use_cat=use_cat, binary=True).to(device)
        # print(m)
        # print(fold, k, len(emb_list))
        # for a, b in v[0][fold].items():
            # print(a, b.shape)
        m.load_state_dict(v[0][fold])
        
        
        for cohort in en.keys():
            for x_batch, y_batch in test_loader:
                pass
            # index = x_batch[:, 0] == en[cohort]
            index = X_test[:, 5] == en[cohort]
            # print(cohort, en[cohort])
            
            # print(y_batch[index])
            # print(x_batch.shape, X_test[:, :].shape)
            
            
            if (y_batch[index] == 0).float().sum() < 1:
                print(cohort, fold, seed, 'not enough negative samples')
                continue
            if clf == 'mlp':
                with torch.no_grad():
                    m.eval()
                    logits, two_d = m(x_batch[index].to(device))
            else:
                with torch.no_grad():
                    x_for_clf = []
                    y_for_clf = []
                    for x_batch_train, y_batch_train in train_loader:
                        with torch.no_grad():
                            m.eval()
                            _, two_d = m(x_batch_train.to(device))
                        x_for_clf.extend(two_d)
                        y_for_clf.extend(y_batch_train)
                x_for_clf = torch.stack(x_for_clf).cpu().numpy()
                y_for_clf = torch.stack(y_for_clf).cpu().numpy()
                if clf == 'svm':
                    one = (y_for_clf == 1).astype(float).sum()
                    zero = (y_for_clf == 0).astype(float).sum()
                    w = (one/zero)
                    delta = 1
        
                    classifier = SVC(kernel="rbf", class_weight={0: w, 1: 1}, probability=True)
                    classifier.fit(x_for_clf, y_for_clf)
                    with torch.no_grad():
                        m.eval()
                        x_in = m(x_batch[index].to(device))[-1].cpu().numpy()
                    logits = classifier.decision_function(x_in)
                elif clf == 'nc':
                    classifier = NearestCentroid(metric="euclidean")
                    classifier.fit(x_for_clf, y_for_clf)
                    with torch.no_grad():
                        m.eval()
                        x_in = m(x_batch[index].to(device))[-1].cpu().numpy()
                    logits = classifier.predict(x_in)

            if 1:
                if clf == 'mlp':
                    y_pred = (logits.reshape(-1) > 0.5).int().cpu()
                elif clf == 'svm':
                    y_pred = torch.from_numpy(logits >= delta).int()
                elif clf == 'nc':
                    
                    y_pred = torch.from_numpy(logits).int()
                a_test = (y_pred[y_batch[index]==0] == y_batch[index][y_batch[index]==0]).float().mean().item() + 1e-20
                b_test = (y_pred[y_batch[index]==1] == y_batch[index][y_batch[index]==1]).float().mean().item() + 1e-20
                c_test = 2 / ((1/a_test) + 1/b_test)
                if clf == 'svm':
                    logits = torch.sigmoid(torch.from_numpy(logits))
                elif clf == 'nc':
                    D = pairwise_distances(x_in, classifier.centroids_, metric=classifier.metric)
                    pos_idx = np.where(classifier.classes_ == 1)[0][0]
                    logits = torch.from_numpy(-D[:, pos_idx])
                    # logits = torch.sigmoid(torch.from_numpy(logits))

                try:
                    auc_test = roc_auc_score(y_batch[index].cpu().numpy(), logits.cpu().numpy())
                    out_auc[f'{cohort},{clf},{setting},{fold},{seed}'] = (y_batch[index].cpu().numpy(), logits.cpu().numpy())
                except:
                    auc_test = None
                sen_test = []
                spec_test = []
                for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
                    sen_, spec_ = at_95(logits.cpu().numpy(), y_batch[index], t=t)
                    sen_test.append(sen_)
                    spec_test.append(spec_)
                    
            
            # metrics[f'{seed}, {fold}, {cohort}, {clf}, {setting}, {usage}'] = ([a_test, b_test, c_test, auc_test, *spec_test, *sen_test])                        
    
        # print('-----------------')

IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 0 1 not enough negative samples
Cancer: Other 1 1 not enough negative samples
IBD 1 1 not enough negative samples
IBD 2 1 not enough negative samples
HIV 4 1 not enough negative samples
IBD 0 2 not enough negative samples
IBD 1 2 not enough negative samples
Cancer: Other 2 2 not enough negative samples
IBD 0 3 not enough negative samples
Cancer: Other 1 3 not enough negative samples
IBD 2 3 not enough negative samples
Cancer: Other 3 3 not enough negative samples
IBD 4 3 not enough negative samples
Cancer: Other 1 4 not enough negative samples
IBD 2 4 not enough negative samples
Cancer: Other 3 4 not enough negative samples
IBD 3 4 not enough negative samples
Cancer: Other 1 5 not enough negative samples
IBD 2 5 not enough negative samples
IBD 4 5 not enough negative samples
Cancer: Other 0 6 not enough negative samples
IBD 0 6 no

/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.w

IBD 4 16 not enough negative samples
IBD 0 17 not enough negative samples
IBD 1 17 not enough negative samples
HIV 3 17 not enough negative samples
IBD 3 17 not enough negative samples
Cancer: Other 0 18 not enough negative samples
IBD 0 18 not enough negative samples
IBD 2 18 not enough negative samples
IBD 3 18 not enough negative samples
HIV 0 19 not enough negative samples
IBD 0 19 not enough negative samples
Cancer: Other 2 19 not enough negative samples
IBD 2 19 not enough negative samples
Cancer: Other 3 19 not enough negative samples
IBD 3 20 not enough negative samples
IBD 4 20 not enough negative samples
Cancer: Other 2 21 not enough negative samples
IBD 2 21 not enough negative samples
Cancer: Other 3 21 not enough negative samples
IBD 3 21 not enough negative samples
Cancer: Other 0 22 not enough negative samples
IBD 0 22 not enough negative samples
Cancer: Other 2 22 not enough negative samples
IBD 3 22 not enough negative samples
IBD 0 23 not enough negative samples
Cance

/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.w

IBD 0 33 not enough negative samples
IBD 1 33 not enough negative samples
Cancer: Other 4 33 not enough negative samples
IBD 4 33 not enough negative samples
IBD 0 34 not enough negative samples
IBD 4 34 not enough negative samples
Cancer: Other 0 35 not enough negative samples
IBD 0 35 not enough negative samples
Cancer: Other 2 35 not enough negative samples
IBD 2 35 not enough negative samples
IBD 3 35 not enough negative samples
Cancer: Other 0 36 not enough negative samples
IBD 1 36 not enough negative samples
Cancer: Other 3 36 not enough negative samples
IBD 4 36 not enough negative samples
Cancer: Other 0 37 not enough negative samples
IBD 0 37 not enough negative samples
IBD 4 37 not enough negative samples
Cancer: Other 0 38 not enough negative samples
Cancer: Other 2 38 not enough negative samples
IBD 2 38 not enough negative samples
IBD 3 38 not enough negative samples
IBD 4 38 not enough negative samples
IBD 0 39 not enough negative samples
IBD 1 39 not enough negative sam

In [862]:
to_json = {}
for k, v in out_auc.items():
    to_json[k] = (v[0].reshape(-1).tolist(), v[1].reshape(-1).tolist())

# out_auc
# import json
with open('data_for_auc_mlp-no cutoff-universal-4,1-demographic-post to pre.json', 'w') as fp:
    json.dump(to_json,fp)

In [864]:
# metrics = {}
out_acu = {}

for k, v in results_8_excel.items():
    # print(k)
    # raise
    # print(k)
    # raise
    
    # print(usage)
    

    split_pattern = r',\s*(?![^()]*\))'
    
    
    seed, cohort, task, setting, clf, _, demographic = re.split(split_pattern, k)
    seed = int(seed)
    
    if setting == 'binary':
        continue
    if demographic != 'Demographic':
        continue

    if clf != 'mlp':
        continue
    if setting == 'binary':
        cutoffs = GLOBAL_CUTOFFS
    else:
        cutoffs = None

    if 'raw' in demographic:
        use_cat = False
    else:
        use_cat = True
    if use_cat:
            start_from = len(emb_list)
    else:
        start_from = 11
        
    
    X6, y6 , pid6, emb_list6, real_size6, encoder6 = load_6(binary=True, cutoffs=cutoffs, qcut=False, include_pid=True)
    
    X2, y2 , pid2, emb_list2, real_size2, encoder2 = load_2(binary=True, cutoffs=cutoffs, qcut=False, include_pid=True)
    
    idx = X6[:, 0] == cohort
    X6 = X6[idx]
    y6 = y6[idx]
    pid6 = pid6[idx]

    idx = X2[:, 0] == cohort
    X2 = X2[idx]
    y2 = y2[idx]
    pid2 = pid2[idx]
    
    # if (y6[idx] == 0).sum() < 5:
    #     continue
    size = real_size6 if use_cat else real_size6 - 1
    
    # en = {}
    # for k_cohort, v_cohort in (X6[idx, [0, 5]]):
    #     en[k_cohort] = v_cohort
     
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    
    
    for_stratify = y6
    for fold, (train_idx, test_idx) in enumerate(skf.split(X6, for_stratify), start=0):
        
        index = np.isin(pid2, pid6[test_idx])
        
        X_train = X6[train_idx]
        y_train = y6[train_idx]
        X_test = X2[index]
        y_test = y2[index]

        if (y_test == 0).sum() < 1:
            print('not enough samples for', cohort, fold, seed)
            continue
        
        
        #################################################### Dataset
        # X_train, X_test = X6[train_idx], X6[test_idx]
        # y_train, y_test = y6[train_idx], y6[test_idx]
        from torch.utils.data import TensorDataset, DataLoader, random_split
        
        # start_from = len(emb_list)
        # size = real_size 
        X_train_tensor = torch.from_numpy(X_train[:, start_from:].astype(np.float32))
        y_train_tensor = torch.from_numpy(y_train).float()
        
        X_test_tensor = torch.from_numpy(X_test[:, start_from:].astype(np.float32))
        y_test_tensor = torch.from_numpy(y_test).float()
        
        from sklearn.model_selection import train_test_split
        # stratified split using sklearn
        train_idx, val_idx = train_test_split(
            np.arange(len(X_train_tensor)),
            test_size=0.2,
            stratify=y_train_tensor.numpy() ,
            random_state=seed
        )
        
        # create datasets
        train_dataset = TensorDataset(X_train_tensor[train_idx], y_train_tensor[train_idx])
        val_dataset   = TensorDataset(X_train_tensor[val_idx],  y_train_tensor[val_idx])
        
        # data loaders
        train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, generator=torch.Generator().manual_seed(seed))
        val_loader   = DataLoader(val_dataset,   batch_size=len(val_dataset), shuffle=False)
        test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor),
                                 batch_size=len(X_test_tensor), shuffle=False)
        # print(size, start_from)
        m = BottleneckMLP(emb_list, size, start_from, 8, (128, 64, 2), p=0.2, use_cat=use_cat, binary=True).to(device)
        # print(m)
        # print(fold, k, len(emb_list))
        # for a, b in v[0][fold].items():
            # print(a, b.shape)
        m.load_state_dict(v[0][fold])
        
        
        # for cohort in en.keys():
        for x_batch, y_batch in test_loader:
            pass
        # index = x_batch[:, 0] == en[cohort]
        # index = X_test[:, 5] == en[cohort]
        # print(cohort, en[cohort])
        
        # print(y_batch[index])
        # print(x_batch.shape, X_test[:, :].shape)
        
        
        # if (y_batch[index] == 0).float().sum() < 1:
        #     print(cohort, fold, seed, 'not enough negative samples')
        #     continue
        if clf == 'mlp':
            with torch.no_grad():
                m.eval()
                logits, two_d = m(x_batch.to(device))
        else:
            with torch.no_grad():
                x_for_clf = []
                y_for_clf = []
                for x_batch_train, y_batch_train in train_loader:
                    with torch.no_grad():
                        m.eval()
                        _, two_d = m(x_batch_train.to(device))
                    x_for_clf.extend(two_d)
                    y_for_clf.extend(y_batch_train)
            x_for_clf = torch.stack(x_for_clf).cpu().numpy()
            y_for_clf = torch.stack(y_for_clf).cpu().numpy()
            if clf == 'svm':
                one = (y_for_clf == 1).astype(float).sum()
                zero = (y_for_clf == 0).astype(float).sum()
                w = (one/zero)
                delta = 1
    
                classifier = SVC(kernel="rbf", class_weight={0: w, 1: 1}, probability=True)
                classifier.fit(x_for_clf, y_for_clf)
                with torch.no_grad():
                    m.eval()
                    x_in = m(x_batch.to(device))[-1].cpu().numpy()
                logits = classifier.decision_function(x_in)
            elif clf == 'nc':
                classifier = NearestCentroid(metric="euclidean")
                classifier.fit(x_for_clf, y_for_clf)
                with torch.no_grad():
                    m.eval()
                    x_in = m(x_batch.to(device))[-1].cpu().numpy()
                logits = classifier.predict(x_in)

        if 1:
            if clf == 'mlp':
                y_pred = (logits.reshape(-1) > 0.5).int().cpu()
            elif clf == 'svm':
                y_pred = torch.from_numpy(logits >= delta).int()
            elif clf == 'nc':
                
                y_pred = torch.from_numpy(logits).int()
            a_test = (y_pred[y_batch==0] == y_batch[y_batch==0]).float().mean().item() + 1e-20
            b_test = (y_pred[y_batch==1] == y_batch[y_batch==1]).float().mean().item() + 1e-20
            c_test = 2 / ((1/a_test) + 1/b_test)
            if clf == 'svm':
                logits = torch.sigmoid(torch.from_numpy(logits))
            elif clf == 'nc':
                D = pairwise_distances(x_in, classifier.centroids_, metric=classifier.metric)
                pos_idx = np.where(classifier.classes_ == 1)[0][0]
                logits = torch.from_numpy(-D[:, pos_idx])
                # logits = torch.sigmoid(torch.from_numpy(logits))

            try:
                auc_test = roc_auc_score(y_batch.cpu().numpy(), logits.cpu().numpy())
                out_auc[f'{cohort},{clf},{setting},{fold},{seed}'] = (y_batch[index].cpu().numpy(), logits.cpu().numpy())
            except:
                auc_test = None
            sen_test = []
            spec_test = []
            for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
                sen_, spec_ = at_95(logits.cpu().numpy(), y_batch, t=t)
                sen_test.append(sen_)
                spec_test.append(spec_)
                
        
        # metrics[f'{seed}, {cohort}, {fold}, {clf}, {setting}, {usage}'] = ([a_test, b_test, c_test, auc_test, *spec_test, *sen_test])                        
        

not enough samples for Cancer: Other 4 0
not enough samples for Cancer: Other 1 1
not enough samples for Cancer: Other 1 2
not enough samples for Cancer: Other 0 3
not enough samples for Cancer: Other 1 5
not enough samples for Cancer: Other 3 6
not enough samples for Cancer: Other 4 8
not enough samples for Cancer: Other 2 9
not enough samples for Cancer: Other 0 10
not enough samples for Cancer: Other 1 10
not enough samples for Cancer: Other 1 11
not enough samples for Cancer: Other 3 13
not enough samples for Cancer: Other 1 14
not enough samples for Cancer: Other 4 16
not enough samples for Cancer: Other 1 18
not enough samples for Cancer: Other 2 19
not enough samples for Cancer: Other 1 21
not enough samples for Cancer: Other 0 22
not enough samples for Cancer: Other 1 25
not enough samples for Cancer: Other 1 28
not enough samples for Cancer: Other 1 29
not enough samples for Cancer: Other 1 30
not enough samples for Cancer: Other 0 31
not enough samples for Cancer: Other 2 32


In [865]:
to_json = {}
for k, v in out_auc.items():
    to_json[k] = (v[0].reshape(-1).tolist(), v[1].reshape(-1).tolist())

# out_auc
# import json
with open('data_for_auc_mlp-no cutoff-cohort specific-4,1-demographic-post to pre.json', 'w') as fp:
    json.dump(to_json,fp)

## Train on 6 test on 2

#### Post to Pre from All to cohort

In [86]:
metrics = {}

for k, v in results_6_excel.items():
    
    seed, task, setting, clf, coeffs, usage = re.split(r',\s*(?![^()]*\))', k)
    seed = int(seed)
    if setting == 'binary':
        cutoffs = GLOBAL_CUTOFFS
    else:
        cutoffs = None

    if 'raw' in usage:
        use_cat = False
    else:
        use_cat = True
    if use_cat:
            start_from = len(emb_list)
    else:
        start_from = 11
        
    
    X6, y6 , pid6, emb_list6, real_size6, encoder6 = load_6(binary=True, cutoffs=cutoffs, include_pid=True)
    
    X2, y2 , pid2, emb_list2, real_size2, encoder2 = load_2(binary=True, cutoffs=cutoffs, include_pid=True)
    
    size = real_size6 if use_cat else real_size6 - 1
    
    en = {}
    for k_cohort, v_cohort in (X6[:, [0, 5]]):
        en[k_cohort] = v_cohort
     
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    
    
    for_stratify = y6
    for fold, (train_idx, test_idx) in enumerate(skf.split(X6, for_stratify), start=0):
        
        index = np.isin(pid2, pid6[test_idx])
        
        X_train = X6[train_idx]
        y_train = y6[train_idx]
        X_test = X2[index]
        y_test = y2[index]
        
        #################################################### Dataset
        
        X_train_tensor = torch.from_numpy(X_train[:, start_from:].astype(np.float32))
        y_train_tensor = torch.from_numpy(y_train).float()
        
        X_test_tensor = torch.from_numpy(X_test[:, start_from:].astype(np.float32))
        y_test_tensor = torch.from_numpy(y_test).float()
        
        
        # stratified split using sklearn
        train_idx, val_idx = train_test_split(
            np.arange(len(X_train_tensor)),
            test_size=0.2,
            stratify=y_train_tensor.numpy() ,
            random_state=seed
        )
        
        # create datasets
        train_dataset = TensorDataset(X_train_tensor[train_idx], y_train_tensor[train_idx])
        val_dataset   = TensorDataset(X_train_tensor[val_idx],  y_train_tensor[val_idx])
        
        # data loaders
        train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, generator=torch.Generator().manual_seed(seed))
        val_loader   = DataLoader(val_dataset,   batch_size=len(val_dataset), shuffle=False)
        test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor),
                                 batch_size=len(X_test_tensor), shuffle=False)
        m = BottleneckMLP(emb_list, size, start_from, 8, (128, 64, 2), p=0.2, use_cat=use_cat, binary=True).to(device)
        m.load_state_dict(v[0][fold])
        
        
        for cohort in en.keys():
            for x_batch, y_batch in test_loader:
                pass
            
            index = X_test[:, 5] == en[cohort]
            
            if (y_batch[index] == 0).float().sum() < 1:
                print(cohort, fold, seed, 'not enough negative samples')
                continue
            if clf == 'mlp':
                with torch.no_grad():
                    m.eval()
                    logits, two_d = m(x_batch[index].to(device))
            else:
                with torch.no_grad():
                    x_for_clf = []
                    y_for_clf = []
                    for x_batch_train, y_batch_train in train_loader:
                        with torch.no_grad():
                            m.eval()
                            _, two_d = m(x_batch_train.to(device))
                        x_for_clf.extend(two_d)
                        y_for_clf.extend(y_batch_train)
                x_for_clf = torch.stack(x_for_clf).cpu().numpy()
                y_for_clf = torch.stack(y_for_clf).cpu().numpy()
                if clf == 'svm':
                    one = (y_for_clf == 1).astype(float).sum()
                    zero = (y_for_clf == 0).astype(float).sum()
                    w = (one/zero)
                    delta = 1
        
                    classifier = SVC(kernel="rbf", class_weight={0: w, 1: 1}, probability=True)
                    classifier.fit(x_for_clf, y_for_clf)
                    with torch.no_grad():
                        m.eval()
                        x_in = m(x_batch[index].to(device))[-1].cpu().numpy()
                    logits = classifier.decision_function(x_in)
                elif clf == 'nc':
                    classifier = NearestCentroid(metric="euclidean")
                    classifier.fit(x_for_clf, y_for_clf)
                    with torch.no_grad():
                        m.eval()
                        x_in = m(x_batch[index].to(device))[-1].cpu().numpy()
                    logits = classifier.predict(x_in)

            if 1:
                if clf == 'mlp':
                    y_pred = (logits.reshape(-1) > 0.5).int().cpu()
                elif clf == 'svm':
                    y_pred = torch.from_numpy(logits >= delta).int()
                elif clf == 'nc':
                    
                    y_pred = torch.from_numpy(logits).int()
                a_test = (y_pred[y_batch[index]==0] == y_batch[index][y_batch[index]==0]).float().mean().item() + 1e-20
                b_test = (y_pred[y_batch[index]==1] == y_batch[index][y_batch[index]==1]).float().mean().item() + 1e-20
                c_test = 2 / ((1/a_test) + 1/b_test)
                if clf == 'svm':
                    logits = torch.sigmoid(torch.from_numpy(logits))
                elif clf == 'nc':
                    D = pairwise_distances(x_in, classifier.centroids_, metric=classifier.metric)
                    pos_idx = np.where(classifier.classes_ == 1)[0][0]
                    logits = torch.from_numpy(-D[:, pos_idx])
                    # logits = torch.sigmoid(torch.from_numpy(logits))

                try:
                    auc_test = roc_auc_score(y_batch[index].cpu().numpy(), logits.cpu().numpy())
                except:
                    auc_test = None
                sen_test = []
                spec_test = []
                for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
                    sen_, spec_ = at_95(logits.cpu().numpy(), y_batch[index], t=t)
                    sen_test.append(sen_)
                    spec_test.append(spec_)
                    
            
            metrics[f'{seed}, {fold}, {cohort}, {clf}, {setting}, {usage}'] = ([a_test, b_test, c_test, auc_test, *spec_test, *sen_test])                        
    

IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative samples
IBD 3 0 not enough negative samples
IBD 4 0 not enough negative samples
IBD 1 0 not enough negative samples
Cancer: Other 3 0 not enough negative sa

/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.w

IBD 4 16 not enough negative samples
IBD 0 16 not enough negative samples
Cancer: Other 2 16 not enough negative samples
IBD 2 16 not enough negative samples


/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.w

IBD 4 16 not enough negative samples
IBD 0 16 not enough negative samples
Cancer: Other 2 16 not enough negative samples
IBD 2 16 not enough negative samples


...

IBD 2 32 not enough negative samples
IBD 3 32 not enough negative samples
Cancer: Other 4 32 not enough negative samples
IBD 4 32 not enough negative samples
IBD 0 32 not enough negative samples


/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.w

IBD 2 32 not enough negative samples
IBD 3 32 not enough negative samples
Cancer: Other 4 32 not enough negative samples
IBD 4 32 not enough negative samples
IBD 0 32 not enough negative samples


/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/smousa13/data/ana/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.w

IBD 2 32 not enough negative samples
IBD 3 32 not enough negative samples
Cancer: Other 4 32 not enough negative samples
IBD 4 32 not enough negative samples
IBD 0 33 not enough negative samples
IBD 1 33 not enough negative samples
Cancer: Other 4 33 not enough negative samples
IBD 4 33 not enough negative samples
IBD 0 33 not enough negative samples
IBD 1 33 not enough negative samples
Cancer: Other 4 33 not enough negative samples
IBD 4 33 not enough negative samples
IBD 0 33 not enough negative samples
IBD 1 33 not enough negative samples
Cancer: Other 4 33 not enough negative samples
IBD 4 33 not enough negative samples
IBD 0 33 not enough negative samples
IBD 1 33 not enough negative samples
Cancer: Other 4 33 not enough negative samples
IBD 4 33 not enough negative samples
IBD 0 33 not enough negative samples
IBD 1 33 not enough negative samples
Cancer: Other 4 33 not enough negative samples
IBD 4 33 not enough negative samples
IBD 0 33 not enough negative samples
IBD 1 33 not en

In [783]:
columns = []
for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
    columns.append(f'Sensitivity at {t} Specificity')
for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
    columns.append(f'Specificity at {t} Sensitivity')
columns = [
    'Low Responders Accuracy',
    'High Responders Accuracy',
    'Harmonic Mean',
    'AUC'
] + columns



records = []
for k, v in metrics.items():
    seed, fold, cohort, clf, setting, usage = k.split(', ')
    records.append({
        'seed': int(seed),
        'fold': int(fold),
        'cohort': cohort,
        'setting': setting,
        'clf': clf,
        'usage': usage,
        **{name: val for name, val in zip(columns, v)}
    })

df = pd.DataFrame(records)

group_cols = ['cohort', 'setting', 'clf', 'usage']
metric_cols = columns

agg_df = df.groupby(group_cols)[metric_cols].agg(['mean', 'std']).reset_index()
agg_df.columns = ['_'.join(col).strip('_') for col in agg_df.columns.values]
agg_df

,cohort,setting,clf,usage,Low Responders Accuracy_mean,Low Responders Accuracy_std,High Responders Accuracy_mean,High Responders Accuracy_std,Harmonic Mean_mean,Harmonic Mean_std,...,Specificity at 0.75 Sensitivity_mean,Specificity at 0.75 Sensitivity_std,Specificity at 0.8 Sensitivity_mean,Specificity at 0.8 Sensitivity_std,Specificity at 0.85 Sensitivity_mean,Specificity at 0.85 Sensitivity_std,Specificity at 0.9 Sensitivity_mean,Specificity at 0.9 Sensitivity_std,Specificity at 0.95 Sensitivity_mean,Specificity at 0.95 Sensitivity_std
0,Autoimmune: Other,binary,mlp,Ab raw,0.332560,0.263971,0.707106,0.215322,0.355744,0.229747,...,0.164627,0.204925,0.121301,0.180765,0.085408,0.155961,0.029827,0.091218,0.003152,0.035180
1,Autoimmune: Other,binary,mlp,Demographic,0.411552,0.281038,0.582601,0.244102,0.365014,0.204857,...,0.160507,0.175133,0.114341,0.155003,0.077651,0.135490,0.029032,0.084865,0.001691,0.019171
2,Autoimmune: Other,binary,nc,Ab raw,0.324891,0.208503,0.678197,0.165884,0.387952,0.197113,...,0.208600,0.208040,0.154906,0.184771,0.117507,0.162262,0.046473,0.102790,0.002454,0.021393
3,Autoimmune: Other,binary,nc,Demographic,0.316463,0.214761,0.658394,0.171643,0.369742,0.199861,...,0.196191,0.196362,0.142819,0.168677,0.109442,0.156061,0.044357,0.103235,0.002403,0.019635
4,Autoimmune: Other,binary,svm,Ab raw,0.465705,0.228333,0.583814,0.154910,0.472822,0.174866,...,0.191339,0.189281,0.150813,0.175317,0.105943,0.154581,0.043714,0.099126,0.002810,0.021536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Transplant,no-cutoff,mlp,Demographic,0.758551,0.230812,0.248275,0.235312,0.276364,0.204195,...,0.162217,0.150194,0.133221,0.136061,0.093980,0.117515,0.052908,0.096070,0.024917,0.070492
80,Transplant,no-cutoff,nc,Ab raw,0.457266,0.182022,0.486646,0.180783,0.426059,0.136331,...,0.188999,0.171921,0.147059,0.150917,0.110886,0.134572,0.063422,0.108781,0.023550,0.068592
81,Transplant,no-cutoff,nc,Demographic,0.725728,0.182564,0.252982,0.177594,0.324263,0.177518,...,0.170369,0.169583,0.130543,0.154496,0.094894,0.133636,0.054325,0.100453,0.023777,0.067905
82,Transplant,no-cutoff,svm,Ab raw,0.668888,0.179599,0.356127,0.178480,0.420379,0.153721,...,0.185020,0.173224,0.139231,0.152063,0.102178,0.135516,0.056370,0.104947,0.022163,0.064406


In [901]:
counts = {}
X6, y6 , pid6, emb_list6, real_size6, encoder6 = load_6(binary=True, cutoffs=cutoffs, include_pid=True)
    
X2, y2 , pid2, emb_list2, real_size2, encoder2 = load_2(binary=True, cutoffs=cutoffs, include_pid=True)


X_train = X6
y_train = y6
X_test = X2
y_test = y2
for cohort in COHORTS:     
    
    idx2 = X_test[:, 0] == cohort

    counts[f'{cohort},Test_N_Low'] = (y_test[idx2] == 0).astype(float).sum()
    counts[f'{cohort},Test_N_High'] = (y_test[idx2] == 1).astype(float).sum()
    counts[f'All,Train_N_Low'] = (y_train == 0).astype(float).sum()
    counts[f'All,Train_N_High'] = (y_train == 1).astype(float).sum()

counts

{'Autoimmune: Other,Test_N_Low': 29.0,
 'Autoimmune: Other,Test_N_High': 68.0,
 'All,Train_N_Low': 264.0,
 'All,Train_N_High': 998.0,
 'Cancer: Other,Test_N_Low': 6.0,
 'Cancer: Other,Test_N_High': 79.0,
 'HIV,Test_N_Low': 14.0,
 'HIV,Test_N_High': 63.0,
 'Healthy Control,Test_N_Low': 21.0,
 'Healthy Control,Test_N_High': 425.0,
 'IBD,Test_N_Low': 3.0,
 'IBD,Test_N_High': 28.0,
 'Multiple Myeloma,Test_N_Low': 42.0,
 'Multiple Myeloma,Test_N_High': 78.0,
 'Transplant,Test_N_Low': 50.0,
 'Transplant,Test_N_High': 56.0}

In [785]:
agg_df['Train_Timepoint'] = 'Post-B'
agg_df['Test_Timepoint'] = 'Pre-B'

agg_df['Test_Cohort'] = agg_df['cohort']
agg_df['Train_Cohort'] = 'All'



def update_usage(row, what):
    cohort = row['cohort']
    return counts[f'{cohort},{what}']

# Apply transformation
for what in ['Train_N_Low', 'Train_N_High', 'Test_N_High', 'Test_N_Low']:
    agg_df[what] = agg_df.apply(lambda z:update_usage(z, what=what), axis=1)
    
    
agg_df = agg_df.rename(columns={
    # 'cohort': 'Train_Cohort', 
    'clf': 'Classifier', 
    'AUC_mean': 'CV_AUC_Mean',
    'AUC_std': 'CV_AUC_SD',
    'Sensitivity at 0.8 Specificity_mean': 'CV_Sen_at_80_Spe_Mean',
    'Sensitivity at 0.8 Specificity_std': 'CV_Sen_at_80_Spe_SD',
    'Low Responders Accuracy_mean': 'CV_Acc_Low_Mean',
    'High Responders Accuracy_mean': 'CV_Acc_High_Mean',
    'Low Responders Accuracy_std': 'CV_Acc_Low_SD',
    'High Responders Accuracy_std': 'CV_Acc_High_SD',
})

def update_usage(row):
    # Start with base usage name
    usage = row['usage']
    setting = row['setting']
    
    
    # Case 1: if usage is "Ab raw"
    
    out = 'Ab_raw'
    
    
    # Case 2: if usage is "Demographic"
    if setting.strip().lower() == 'binary':
        out += '+ Ab_binary'
        
    if usage.strip().lower() == 'demographic':
        out += '+ demographic'
    
    # Case 3: modify further based on 'setting'
    
    if setting.strip().lower() == 'no-cutoff':
        # don't add anything
        pass
    
    return out

# Apply transformation
agg_df['Feature_Set'] = agg_df.apply(update_usage, axis=1)
agg_df[['Train_Timepoint', 'Train_Cohort', 'Train_N_Low', 'Train_N_High', 'Feature_Set', 'Test_Timepoint', 'Test_Cohort', 'Test_N_Low' ,'Test_N_High', 'Classifier', 'CV_AUC_Mean', 'CV_AUC_SD',	'CV_Sen_at_80_Spe_Mean', 'CV_Sen_at_80_Spe_SD', 'CV_Acc_Low_Mean', 'CV_Acc_Low_SD',	'CV_Acc_High_Mean',	'CV_Acc_High_SD']]



,Train_Timepoint,Train_Cohort,Train_N_Low,Train_N_High,Feature_Set,Test_Timepoint,Test_Cohort,Test_N_Low,Test_N_High,Classifier,CV_AUC_Mean,CV_AUC_SD,CV_Sen_at_80_Spe_Mean,CV_Sen_at_80_Spe_SD,CV_Acc_Low_Mean,CV_Acc_Low_SD,CV_Acc_High_Mean,CV_Acc_High_SD
0,Post-B,All,58.0,130.0,Ab_raw+ Ab_binary,Pre-B,Autoimmune: Other,4.0,13.0,mlp,0.521546,0.158262,0.353736,0.225988,0.332560,0.263971,0.707106,0.215322
1,Post-B,All,58.0,130.0,Ab_raw+ Ab_binary+ demographic,Pre-B,Autoimmune: Other,4.0,13.0,mlp,0.494749,0.155905,0.319418,0.215676,0.411552,0.281038,0.582601,0.244102
2,Post-B,All,58.0,130.0,Ab_raw+ Ab_binary,Pre-B,Autoimmune: Other,4.0,13.0,nc,0.563639,0.162460,0.411397,0.238082,0.324891,0.208503,0.678197,0.165884
3,Post-B,All,58.0,130.0,Ab_raw+ Ab_binary+ demographic,Pre-B,Autoimmune: Other,4.0,13.0,nc,0.543587,0.157837,0.386387,0.223567,0.316463,0.214761,0.658394,0.171643
4,Post-B,All,58.0,130.0,Ab_raw+ Ab_binary,Pre-B,Autoimmune: Other,4.0,13.0,svm,0.538651,0.159970,0.370519,0.220795,0.465705,0.228333,0.583814,0.154910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Post-B,All,63.0,60.0,Ab_raw+ demographic,Pre-B,Transplant,15.0,11.0,mlp,0.506230,0.120656,0.260114,0.214195,0.758551,0.230812,0.248275,0.235312
80,Post-B,All,63.0,60.0,Ab_raw,Pre-B,Transplant,15.0,11.0,nc,0.529683,0.134833,0.364987,0.191334,0.457266,0.182022,0.486646,0.180783
81,Post-B,All,63.0,60.0,Ab_raw+ demographic,Pre-B,Transplant,15.0,11.0,nc,0.510128,0.135874,0.325188,0.188201,0.725728,0.182564,0.252982,0.177594
82,Post-B,All,63.0,60.0,Ab_raw,Pre-B,Transplant,15.0,11.0,svm,0.510580,0.137285,0.329528,0.193784,0.668888,0.179599,0.356127,0.178480


In [786]:
out = agg_df[['Train_Timepoint', 'Train_Cohort', 'Train_N_Low', 'Train_N_High', 'Feature_Set', 'Test_Timepoint', 'Test_Cohort', 'Test_N_Low' ,'Test_N_High', 'Classifier',	'CV_AUC_Mean', 'CV_AUC_SD',	'CV_Sen_at_80_Spe_Mean', 'CV_Sen_at_80_Spe_SD', 'CV_Acc_Low_Mean', 'CV_Acc_Low_SD',	'CV_Acc_High_Mean',	'CV_Acc_High_SD']]
out.to_csv('final_results/model_performance_summary_v4.csv')

#### Post to Pre All

In [787]:
metrics = {}

for k, v in results_6_excel.items():
    seed, task, setting, clf, coeffs, usage = re.split(r',\s*(?![^()]*\))', k)
    seed = int(seed)
    if setting == 'binary':
        cutoffs = GLOBAL_CUTOFFS
    else:
        cutoffs = None

    if 'raw' in usage:
        use_cat = False
    else:
        use_cat = True
    if use_cat:
            start_from = len(emb_list)
    else:
        start_from = 11
        
    
    X6, y6 , pid6, emb_list6, real_size6, encoder6 = load_6(binary=True, cutoffs=cutoffs, include_pid=True)
    
    X2, y2 , pid2, emb_list2, real_size2, encoder2 = load_2(binary=True, cutoffs=cutoffs, include_pid=True)
    
    size = real_size6 if use_cat else real_size6 - 1
    
     
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    
    
    for_stratify = y6
    for fold, (train_idx, test_idx) in enumerate(skf.split(X6, for_stratify), start=0):
        
        index = np.isin(pid2, pid6[test_idx])
        
        X_train = X6[train_idx]
        y_train = y6[train_idx]
        X_test = X2[index]
        y_test = y2[index]
        
        
        
        #################################################### Dataset
        
        X_train_tensor = torch.from_numpy(X_train[:, start_from:].astype(np.float32))
        y_train_tensor = torch.from_numpy(y_train).float()
        
        X_test_tensor = torch.from_numpy(X_test[:, start_from:].astype(np.float32))
        y_test_tensor = torch.from_numpy(y_test).float()
        
        from sklearn.model_selection import train_test_split
        # stratified split using sklearn
        train_idx, val_idx = train_test_split(
            np.arange(len(X_train_tensor)),
            test_size=0.2,
            stratify=y_train_tensor.numpy() ,
            random_state=seed
        )
        
        # create datasets
        train_dataset = TensorDataset(X_train_tensor[train_idx], y_train_tensor[train_idx])
        val_dataset   = TensorDataset(X_train_tensor[val_idx],  y_train_tensor[val_idx])
        
        # data loaders
        train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, generator=torch.Generator().manual_seed(seed))
        val_loader   = DataLoader(val_dataset,   batch_size=len(val_dataset), shuffle=False)
        test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor),
                                 batch_size=len(X_test_tensor), shuffle=False)
        
        m = BottleneckMLP(emb_list, size, start_from, 8, (128, 64, 2), p=0.2, use_cat=use_cat, binary=True).to(device)
        
        m.load_state_dict(v[0][fold])
        
        
        
        for x_batch, y_batch in test_loader:
            pass
        
        if clf == 'mlp':
            with torch.no_grad():
                m.eval()
                logits, two_d = m(x_batch.to(device))
        else:
            with torch.no_grad():
                x_for_clf = []
                y_for_clf = []
                for x_batch_train, y_batch_train in train_loader:
                    with torch.no_grad():
                        m.eval()
                        _, two_d = m(x_batch_train.to(device))
                    x_for_clf.extend(two_d)
                    y_for_clf.extend(y_batch_train)
            x_for_clf = torch.stack(x_for_clf).cpu().numpy()
            y_for_clf = torch.stack(y_for_clf).cpu().numpy()
            if clf == 'svm':
                one = (y_for_clf == 1).astype(float).sum()
                zero = (y_for_clf == 0).astype(float).sum()
                w = (one/zero)
                delta = 1
    
                classifier = SVC(kernel="rbf", class_weight={0: w, 1: 1}, probability=True)
                classifier.fit(x_for_clf, y_for_clf)
                with torch.no_grad():
                    m.eval()
                    x_in = m(x_batch.to(device))[-1].cpu().numpy()
                logits = classifier.decision_function(x_in)
            elif clf == 'nc':
                classifier = NearestCentroid(metric="euclidean")
                classifier.fit(x_for_clf, y_for_clf)
                with torch.no_grad():
                    m.eval()
                    x_in = m(x_batch.to(device))[-1].cpu().numpy()
                logits = classifier.predict(x_in)

        if 1:
            if clf == 'mlp':
                y_pred = (logits.reshape(-1) > 0.5).int().cpu()
            elif clf == 'svm':
                y_pred = torch.from_numpy(logits >= delta).int()
            elif clf == 'nc':
                
                y_pred = torch.from_numpy(logits).int()
            a_test = (y_pred[y_batch==0] == y_batch[y_batch==0]).float().mean().item() + 1e-20
            b_test = (y_pred[y_batch==1] == y_batch[y_batch==1]).float().mean().item() + 1e-20
            c_test = 2 / ((1/a_test) + 1/b_test)
            if clf == 'svm':
                logits = torch.sigmoid(torch.from_numpy(logits))
            elif clf == 'nc':
                D = pairwise_distances(x_in, classifier.centroids_, metric=classifier.metric)
                pos_idx = np.where(classifier.classes_ == 1)[0][0]
                logits = torch.from_numpy(-D[:, pos_idx])
                # logits = torch.sigmoid(torch.from_numpy(logits))

            try:
                auc_test = roc_auc_score(y_batch.cpu().numpy(), logits.cpu().numpy())
            except:
                auc_test = None
            sen_test = []
            spec_test = []
            for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
                sen_, spec_ = at_95(logits.cpu().numpy(), y_batch, t=t)
                sen_test.append(sen_)
                spec_test.append(spec_)
                
        
        metrics[f'{seed}, {fold}, {clf}, {setting}, {usage}'] = ([a_test, b_test, c_test, auc_test, *spec_test, *sen_test])                        
    
        

In [788]:
columns = []
for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
    columns.append(f'Sensitivity at {t} Specificity')
for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
    columns.append(f'Specificity at {t} Sensitivity')
columns = [
    'Low Responders Accuracy',
    'High Responders Accuracy',
    'Harmonic Mean',
    'AUC'
] + columns


records = []
for k, v in metrics.items():
    seed, fold, clf, setting, usage = k.split(', ')
    records.append({
        'seed': int(seed),
        'fold': int(fold),
        'cohort': 'All',
        'setting': setting,
        'clf': clf,
        'usage': usage,
        **{name: val for name, val in zip(columns, v)}
    })

df = pd.DataFrame(records)

group_cols = ['cohort', 'setting', 'clf', 'usage']
metric_cols = columns

agg_df = df.groupby(group_cols)[metric_cols].agg(['mean', 'std']).reset_index()
agg_df.columns = ['_'.join(col).strip('_') for col in agg_df.columns.values]
agg_df

,cohort,setting,clf,usage,Low Responders Accuracy_mean,Low Responders Accuracy_std,High Responders Accuracy_mean,High Responders Accuracy_std,Harmonic Mean_mean,Harmonic Mean_std,...,Specificity at 0.75 Sensitivity_mean,Specificity at 0.75 Sensitivity_std,Specificity at 0.8 Sensitivity_mean,Specificity at 0.8 Sensitivity_std,Specificity at 0.85 Sensitivity_mean,Specificity at 0.85 Sensitivity_std,Specificity at 0.9 Sensitivity_mean,Specificity at 0.9 Sensitivity_std,Specificity at 0.95 Sensitivity_mean,Specificity at 0.95 Sensitivity_std
0,All,binary,mlp,Ab raw,0.546679,0.193326,0.689418,0.171713,0.557660,0.089702,...,0.465884,0.116691,0.398561,0.134542,0.297685,0.154316,0.172299,0.138171,0.086329,0.095995
1,All,binary,mlp,Demographic,0.632616,0.184636,0.699507,0.149374,0.623668,0.083498,...,0.574320,0.109623,0.508895,0.123962,0.417694,0.152192,0.288840,0.165482,0.150385,0.136545
2,All,binary,nc,Ab raw,0.518382,0.099174,0.670994,0.102654,0.572452,0.060303,...,0.299790,0.106659,0.246237,0.100230,0.190513,0.091330,0.123596,0.073609,0.069000,0.056936
3,All,binary,nc,Demographic,0.568769,0.094535,0.726695,0.084805,0.629702,0.063332,...,0.368454,0.133882,0.306229,0.125295,0.238079,0.115049,0.154580,0.091888,0.081413,0.065334
4,All,binary,svm,Ab raw,0.663529,0.104491,0.568282,0.095380,0.599462,0.054068,...,0.410931,0.127360,0.336310,0.120319,0.263023,0.114299,0.172264,0.098212,0.094827,0.074866
5,All,binary,svm,Demographic,0.723340,0.092172,0.613580,0.087891,0.654790,0.051889,...,0.515967,0.139353,0.433739,0.148361,0.345280,0.148934,0.236658,0.134909,0.137051,0.105774
6,All,no-cutoff,mlp,Ab raw,0.555810,0.180396,0.678054,0.169472,0.563384,0.080274,...,0.466702,0.109443,0.402152,0.117815,0.304421,0.136663,0.176550,0.123484,0.091774,0.091220
7,All,no-cutoff,mlp,Demographic,0.634172,0.174214,0.718785,0.136188,0.639169,0.077850,...,0.612687,0.094053,0.554801,0.110707,0.473465,0.142577,0.317607,0.180307,0.164450,0.153855
8,All,no-cutoff,nc,Ab raw,0.515509,0.105706,0.660709,0.112453,0.563920,0.063254,...,0.298961,0.104730,0.241912,0.095393,0.183702,0.086119,0.115515,0.066469,0.064643,0.053530
9,All,no-cutoff,nc,Demographic,0.592777,0.106073,0.735204,0.088069,0.645536,0.064131,...,0.387195,0.138742,0.324726,0.132948,0.248718,0.119506,0.166859,0.102750,0.093044,0.076183


In [903]:
counts = {}
X6, y6 , pid6, emb_list6, real_size6, encoder6 = load_6(binary=True, cutoffs=cutoffs, include_pid=True)
    
X2, y2 , pid2, emb_list2, real_size2, encoder2 = load_2(binary=True, cutoffs=cutoffs, include_pid=True)


X_train = X6
y_train = y6
X_test = X2
y_test = y2


counts[f'Test_N_Low'] = (y_test == 0).astype(float).sum()
counts[f'Test_N_High'] = (y_test == 1).astype(float).sum()
counts[f'Train_N_Low'] = (y_train == 0).astype(float).sum()
counts[f'Train_N_High'] = (y_train == 1).astype(float).sum()

counts

{'Test_N_Low': 165.0,
 'Test_N_High': 797.0,
 'Train_N_Low': 264.0,
 'Train_N_High': 998.0}

In [789]:
agg_df['Train_Timepoint'] = 'Post-B'
agg_df['Test_Timepoint'] = 'Pre-B'

agg_df['Test_Cohort'] = agg_df['cohort']
agg_df['Train_Cohort'] = 'All'



def update_usage(row, what):
    # cohort = row['cohort']
    return counts[f'{what}']

# Apply transformation
for what in ['Train_N_Low', 'Train_N_High', 'Test_N_High', 'Test_N_Low']:
    agg_df[what] = agg_df.apply(lambda z:update_usage(z, what=what), axis=1)
    
    
agg_df = agg_df.rename(columns={
    # 'cohort': 'Train_Cohort', 
    'clf': 'Classifier', 
    'AUC_mean': 'CV_AUC_Mean',
    'AUC_std': 'CV_AUC_SD',
    'Sensitivity at 0.8 Specificity_mean': 'CV_Sen_at_80_Spe_Mean',
    'Sensitivity at 0.8 Specificity_std': 'CV_Sen_at_80_Spe_SD',
    'Low Responders Accuracy_mean': 'CV_Acc_Low_Mean',
    'High Responders Accuracy_mean': 'CV_Acc_High_Mean',
    'Low Responders Accuracy_std': 'CV_Acc_Low_SD',
    'High Responders Accuracy_std': 'CV_Acc_High_SD',
})

def update_usage(row):
    # Start with base usage name
    usage = row['usage']
    setting = row['setting']
    
    
    # Case 1: if usage is "Ab raw"
    
    out = 'Ab_raw'
    
    
    # Case 2: if usage is "Demographic"
    if setting.strip().lower() == 'binary':
        out += '+ Ab_binary'
        
    if usage.strip().lower() == 'demographic':
        out += '+ demographic'
    
    # Case 3: modify further based on 'setting'
    
    if setting.strip().lower() == 'no-cutoff':
        # don't add anything
        pass
    
    return out

# Apply transformation
agg_df['Feature_Set'] = agg_df.apply(update_usage, axis=1)
agg_df[['Train_Timepoint', 'Train_Cohort', 'Train_N_Low', 'Train_N_High', 'Feature_Set', 'Test_Timepoint', 'Test_Cohort', 'Test_N_Low' ,'Test_N_High', 'Classifier', 'CV_AUC_Mean', 'CV_AUC_SD',	'CV_Sen_at_80_Spe_Mean', 'CV_Sen_at_80_Spe_SD', 'CV_Acc_Low_Mean', 'CV_Acc_Low_SD',	'CV_Acc_High_Mean',	'CV_Acc_High_SD']]



,Train_Timepoint,Train_Cohort,Train_N_Low,Train_N_High,Feature_Set,Test_Timepoint,Test_Cohort,Test_N_Low,Test_N_High,Classifier,CV_AUC_Mean,CV_AUC_SD,CV_Sen_at_80_Spe_Mean,CV_Sen_at_80_Spe_SD,CV_Acc_Low_Mean,CV_Acc_Low_SD,CV_Acc_High_Mean,CV_Acc_High_SD
0,Post-B,All,211.0,798.0,Ab_raw+ Ab_binary,Pre-B,All,35.0,156.0,mlp,0.672697,0.051384,0.463321,0.089896,0.546679,0.193326,0.689418,0.171713
1,Post-B,All,211.0,798.0,Ab_raw+ Ab_binary+ demographic,Pre-B,All,35.0,156.0,mlp,0.732898,0.049382,0.558058,0.098004,0.632616,0.184636,0.699507,0.149374
2,Post-B,All,211.0,798.0,Ab_raw+ Ab_binary,Pre-B,All,35.0,156.0,nc,0.563766,0.074537,0.307254,0.111550,0.518382,0.099174,0.670994,0.102654
3,Post-B,All,211.0,798.0,Ab_raw+ Ab_binary+ demographic,Pre-B,All,35.0,156.0,nc,0.610702,0.083908,0.370715,0.138304,0.568769,0.094535,0.726695,0.084805
4,Post-B,All,211.0,798.0,Ab_raw+ Ab_binary,Pre-B,All,35.0,156.0,svm,0.654550,0.056922,0.451986,0.095772,0.663529,0.104491,0.568282,0.095380
5,Post-B,All,211.0,798.0,Ab_raw+ Ab_binary+ demographic,Pre-B,All,35.0,156.0,svm,0.713870,0.053128,0.552253,0.095589,0.723340,0.092172,0.613580,0.087891
6,Post-B,All,211.0,798.0,Ab_raw,Pre-B,All,35.0,156.0,mlp,0.672738,0.053988,0.463179,0.097002,0.555810,0.180396,0.678054,0.169472
7,Post-B,All,211.0,798.0,Ab_raw+ demographic,Pre-B,All,35.0,156.0,mlp,0.747724,0.044974,0.580958,0.095335,0.634172,0.174214,0.718785,0.136188
8,Post-B,All,211.0,798.0,Ab_raw,Pre-B,All,35.0,156.0,nc,0.558908,0.076167,0.304340,0.117314,0.515509,0.105706,0.660709,0.112453
9,Post-B,All,211.0,798.0,Ab_raw+ demographic,Pre-B,All,35.0,156.0,nc,0.626030,0.081634,0.396166,0.134052,0.592777,0.106073,0.735204,0.088069


In [790]:
out = agg_df[['Train_Timepoint', 'Train_Cohort', 'Train_N_Low', 'Train_N_High', 'Feature_Set', 'Test_Timepoint', 'Test_Cohort', 'Test_N_Low' ,'Test_N_High', 'Classifier',	'CV_AUC_Mean', 'CV_AUC_SD',	'CV_Sen_at_80_Spe_Mean', 'CV_Sen_at_80_Spe_SD', 'CV_Acc_Low_Mean', 'CV_Acc_Low_SD',	'CV_Acc_High_Mean',	'CV_Acc_High_SD']]
out.to_csv('final_results/model_performance_summary_v5_revised.csv')

#### Post to Pre cohort by cohort

In [907]:
counts = {}
X6, y6 , pid6, emb_list6, real_size6, encoder6 = load_6(binary=True, cutoffs=cutoffs, include_pid=True)
    
X2, y2 , pid2, emb_list2, real_size2, encoder2 = load_2(binary=True, cutoffs=cutoffs, include_pid=True)


X_train = X6
y_train = y6
X_test = X2
y_test = y2

for cohort in COHORTS:     
    
    
    # idx = X[:, 0] == 'Healthy Control'
    idx1 = X_train[:, 0] == cohort
    idx2 = X_test[:, 0] == cohort

    counts[f'{cohort},Test_N_Low'] = (y_test[idx2] == 0).astype(float).sum()
    counts[f'{cohort},Test_N_High'] = (y_test[idx2] == 1).astype(float).sum()
    counts[f'{cohort},Train_N_Low'] = (y_train[idx1] == 0).astype(float).sum()
    counts[f'{cohort},Train_N_High'] = (y_train[idx1] == 1).astype(float).sum()

counts

{'Autoimmune: Other,Test_N_Low': 29.0,
 'Autoimmune: Other,Test_N_High': 68.0,
 'Autoimmune: Other,Train_N_Low': 72.0,
 'Autoimmune: Other,Train_N_High': 163.0,
 'Cancer: Other,Test_N_Low': 6.0,
 'Cancer: Other,Test_N_High': 79.0,
 'Cancer: Other,Train_N_Low': 9.0,
 'Cancer: Other,Train_N_High': 102.0,
 'HIV,Test_N_Low': 14.0,
 'HIV,Test_N_High': 63.0,
 'HIV,Train_N_Low': 14.0,
 'HIV,Train_N_High': 70.0,
 'Healthy Control,Test_N_Low': 21.0,
 'Healthy Control,Test_N_High': 425.0,
 'Healthy Control,Train_N_Low': 24.0,
 'Healthy Control,Train_N_High': 465.0,
 'IBD,Test_N_Low': 3.0,
 'IBD,Test_N_High': 28.0,
 'IBD,Train_N_Low': 4.0,
 'IBD,Train_N_High': 33.0,
 'Multiple Myeloma,Test_N_Low': 42.0,
 'Multiple Myeloma,Test_N_High': 78.0,
 'Multiple Myeloma,Train_N_Low': 60.0,
 'Multiple Myeloma,Train_N_High': 92.0,
 'Transplant,Test_N_Low': 50.0,
 'Transplant,Test_N_High': 56.0,
 'Transplant,Train_N_Low': 81.0,
 'Transplant,Train_N_High': 73.0}

In [796]:
metrics = {}

for k, v in results_8_excel.items():
    
    seed, cohort, task, setting, clf, coeffs, usage = re.split(r',\s*(?![^()]*\))', k)
    
    seed = int(seed)
    if setting == 'binary':
        cutoffs = GLOBAL_CUTOFFS
    else:
        cutoffs = None

    if 'raw' in usage:
        use_cat = False
    else:
        use_cat = True
    if use_cat:
            start_from = len(emb_list)
    else:
        start_from = 11
        
    
    X6, y6 , pid6, emb_list6, real_size6, encoder6 = load_6(binary=True, cutoffs=cutoffs, include_pid=True)
    
    X2, y2 , pid2, emb_list2, real_size2, encoder2 = load_2(binary=True, cutoffs=cutoffs, include_pid=True)
    
    idx = X6[:, 0] == cohort
    X6 = X6[idx]
    y6 = y6[idx]
    pid6 = pid6[idx]

    idx = X2[:, 0] == cohort
    X2 = X2[idx]
    y2 = y2[idx]
    pid2 = pid2[idx]
    
    size = real_size6 if use_cat else real_size6 - 1
         
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    
    
    for_stratify = y6
    for fold, (train_idx, test_idx) in enumerate(skf.split(X6, for_stratify), start=0):
        
        index = np.isin(pid2, pid6[test_idx])
        
        X_train = X6[train_idx]
        y_train = y6[train_idx]
        X_test = X2[index]
        y_test = y2[index]

        if (y_test == 0).sum() < 1:
            print('not enough samples for', cohort, fold, seed)
            continue
        
        
        #################################################### Dataset
        
        X_train_tensor = torch.from_numpy(X_train[:, start_from:].astype(np.float32))
        y_train_tensor = torch.from_numpy(y_train).float()
        
        X_test_tensor = torch.from_numpy(X_test[:, start_from:].astype(np.float32))
        y_test_tensor = torch.from_numpy(y_test).float()
        
        
        # stratified split using sklearn
        train_idx, val_idx = train_test_split(
            np.arange(len(X_train_tensor)),
            test_size=0.2,
            stratify=y_train_tensor.numpy() ,
            random_state=seed
        )
        
        # create datasets
        train_dataset = TensorDataset(X_train_tensor[train_idx], y_train_tensor[train_idx])
        val_dataset   = TensorDataset(X_train_tensor[val_idx],  y_train_tensor[val_idx])
        
        # data loaders
        train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, generator=torch.Generator().manual_seed(seed))
        val_loader   = DataLoader(val_dataset,   batch_size=len(val_dataset), shuffle=False)
        test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor),
                                 batch_size=len(X_test_tensor), shuffle=False)
        
        m = BottleneckMLP(emb_list, size, start_from, 8, (128, 64, 2), p=0.2, use_cat=use_cat, binary=True).to(device)
        
        m.load_state_dict(v[0][fold])
        
        
        # for cohort in en.keys():
        for x_batch, y_batch in test_loader:
            pass
        
        if clf == 'mlp':
            with torch.no_grad():
                m.eval()
                logits, two_d = m(x_batch.to(device))
        else:
            with torch.no_grad():
                x_for_clf = []
                y_for_clf = []
                for x_batch_train, y_batch_train in train_loader:
                    with torch.no_grad():
                        m.eval()
                        _, two_d = m(x_batch_train.to(device))
                    x_for_clf.extend(two_d)
                    y_for_clf.extend(y_batch_train)
            x_for_clf = torch.stack(x_for_clf).cpu().numpy()
            y_for_clf = torch.stack(y_for_clf).cpu().numpy()
            if clf == 'svm':
                one = (y_for_clf == 1).astype(float).sum()
                zero = (y_for_clf == 0).astype(float).sum()
                w = (one/zero)
                delta = 1
    
                classifier = SVC(kernel="rbf", class_weight={0: w, 1: 1}, probability=True)
                classifier.fit(x_for_clf, y_for_clf)
                with torch.no_grad():
                    m.eval()
                    x_in = m(x_batch.to(device))[-1].cpu().numpy()
                logits = classifier.decision_function(x_in)
            elif clf == 'nc':
                classifier = NearestCentroid(metric="euclidean")
                classifier.fit(x_for_clf, y_for_clf)
                with torch.no_grad():
                    m.eval()
                    x_in = m(x_batch.to(device))[-1].cpu().numpy()
                logits = classifier.predict(x_in)

        if 1:
            if clf == 'mlp':
                y_pred = (logits.reshape(-1) > 0.5).int().cpu()
            elif clf == 'svm':
                y_pred = torch.from_numpy(logits >= delta).int()
            elif clf == 'nc':
                
                y_pred = torch.from_numpy(logits).int()
            a_test = (y_pred[y_batch==0] == y_batch[y_batch==0]).float().mean().item() + 1e-20
            b_test = (y_pred[y_batch==1] == y_batch[y_batch==1]).float().mean().item() + 1e-20
            c_test = 2 / ((1/a_test) + 1/b_test)
            if clf == 'svm':
                logits = torch.sigmoid(torch.from_numpy(logits))
            elif clf == 'nc':
                D = pairwise_distances(x_in, classifier.centroids_, metric=classifier.metric)
                pos_idx = np.where(classifier.classes_ == 1)[0][0]
                logits = torch.from_numpy(-D[:, pos_idx])
                # logits = torch.sigmoid(torch.from_numpy(logits))

            try:
                auc_test = roc_auc_score(y_batch.cpu().numpy(), logits.cpu().numpy())
            except:
                auc_test = None
            sen_test = []
            spec_test = []
            for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
                sen_, spec_ = at_95(logits.cpu().numpy(), y_batch, t=t)
                sen_test.append(sen_)
                spec_test.append(spec_)
                
        
        metrics[f'{seed}, {cohort}, {fold}, {clf}, {setting}, {usage}'] = ([a_test, b_test, c_test, auc_test, *spec_test, *sen_test])                        
        

not enough samples for Cancer: Other 4 0
not enough samples for Cancer: Other 4 0
not enough samples for Cancer: Other 4 0
not enough samples for Cancer: Other 4 0
not enough samples for Cancer: Other 4 0
not enough samples for Cancer: Other 4 0
not enough samples for Cancer: Other 4 0
not enough samples for Cancer: Other 4 0
not enough samples for Cancer: Other 4 0
not enough samples for Cancer: Other 4 0
not enough samples for Cancer: Other 4 0
not enough samples for Cancer: Other 4 0
not enough samples for Cancer: Other 1 1
not enough samples for Cancer: Other 1 1
not enough samples for Cancer: Other 1 1
not enough samples for Cancer: Other 1 1
not enough samples for Cancer: Other 1 1
not enough samples for Cancer: Other 1 1
not enough samples for Cancer: Other 1 1
not enough samples for Cancer: Other 1 1
not enough samples for Cancer: Other 1 1
not enough samples for Cancer: Other 1 1
not enough samples for Cancer: Other 1 1
not enough samples for Cancer: Other 1 1
not enough sampl

In [799]:
columns = []
for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
    columns.append(f'Sensitivity at {t} Specificity')
for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
    columns.append(f'Specificity at {t} Sensitivity')
columns = [
    'Low Responders Accuracy',
    'High Responders Accuracy',
    'Harmonic Mean',
    'AUC'
] + columns
for k, v in results_6_excel.items():
    seed, _, setting, clf, __, usage = re.split(r',\s*(?![^()]*\))', k)


records = []
for k, v in metrics.items():
    
    seed,  cohort, fold, clf, setting, usage = k.split(', ')
    
    records.append({
        'seed': int(seed),
        'fold': int(fold),
        'cohort': cohort,
        
        'setting': setting,
        'clf': clf,
        
        'usage': usage,
        **{name: val for name, val in zip(columns, v)}
    })

df = pd.DataFrame(records)

group_cols = ['cohort', 'setting', 'clf', 'usage']
metric_cols = columns

agg_df = df.groupby(group_cols)[metric_cols].agg(['mean', 'std']).reset_index()
agg_df.columns = ['_'.join(col).strip('_') for col in agg_df.columns.values]
agg_df

,cohort,setting,clf,usage,Low Responders Accuracy_mean,Low Responders Accuracy_std,High Responders Accuracy_mean,High Responders Accuracy_std,Harmonic Mean_mean,Harmonic Mean_std,...,Specificity at 0.75 Sensitivity_mean,Specificity at 0.75 Sensitivity_std,Specificity at 0.8 Sensitivity_mean,Specificity at 0.8 Sensitivity_std,Specificity at 0.85 Sensitivity_mean,Specificity at 0.85 Sensitivity_std,Specificity at 0.9 Sensitivity_mean,Specificity at 0.9 Sensitivity_std,Specificity at 0.95 Sensitivity_mean,Specificity at 0.95 Sensitivity_std
0,Autoimmune: Other,binary,mlp,Ab raw,0.490417,0.352998,0.512023,0.296374,0.304688,0.210677,...,0.199056,0.186359,0.153683,0.174722,0.115943,0.159368,0.045194,0.098619,0.000401,0.008953
1,Autoimmune: Other,binary,mlp,Demographic,0.536604,0.350811,0.489099,0.304677,0.315351,0.220781,...,0.211838,0.192582,0.161337,0.177231,0.121193,0.162208,0.050566,0.109054,0.001317,0.016690
2,Autoimmune: Other,binary,nc,Ab raw,0.421147,0.238259,0.586698,0.153931,0.432855,0.192064,...,0.171063,0.182876,0.128583,0.164165,0.098082,0.145560,0.046692,0.104346,0.000200,0.004477
3,Autoimmune: Other,binary,nc,Demographic,0.440633,0.236083,0.585957,0.166261,0.445324,0.175251,...,0.184069,0.180349,0.141173,0.166050,0.105621,0.150476,0.045678,0.099951,0.000655,0.011703
4,Autoimmune: Other,binary,svm,Ab raw,0.715190,0.207241,0.323346,0.151572,0.406954,0.150733,...,0.205219,0.184838,0.154670,0.175532,0.112998,0.154266,0.048125,0.098179,0.000755,0.013858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Transplant,no-cutoff,mlp,Demographic,0.495041,0.291569,0.536702,0.280428,0.367923,0.185840,...,0.214903,0.175159,0.167972,0.163703,0.109242,0.142502,0.058584,0.111620,0.023340,0.067919
68,Transplant,no-cutoff,nc,Ab raw,0.534237,0.170507,0.477528,0.168274,0.468434,0.123173,...,0.182874,0.160073,0.145437,0.144917,0.102378,0.124783,0.052470,0.095564,0.024376,0.066651
69,Transplant,no-cutoff,nc,Demographic,0.474986,0.179102,0.540976,0.166879,0.467028,0.124548,...,0.177809,0.155639,0.141918,0.141325,0.102538,0.125637,0.063104,0.108609,0.028437,0.075262
70,Transplant,no-cutoff,svm,Ab raw,0.835873,0.141411,0.172387,0.120410,0.262430,0.155338,...,0.187280,0.154676,0.145490,0.144336,0.103895,0.126659,0.054836,0.098986,0.020968,0.059021


In [800]:
agg_df['Train_Timepoint'] = 'Post-B'
agg_df['Test_Timepoint'] = 'Pre-B'

agg_df['Test_Cohort'] = agg_df['cohort']
agg_df['Train_Cohort'] = agg_df['cohort']


counts = {}
X6, y6 , pid6, emb_list6, real_size6, encoder6 = load_6(binary=True, cutoffs=cutoffs, include_pid=True)
    
X2, y2 , pid2, emb_list2, real_size2, encoder2 = load_2(binary=True, cutoffs=cutoffs, include_pid=True)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    
for_stratify = y6
for fold, (train_idx, test_idx) in enumerate(skf.split(X6, for_stratify), start=0):
    
    index = np.isin(pid2, pid6[test_idx])
    
    X_train = X6[train_idx]
    y_train = y6[train_idx]
    X_test = X2[index]
    y_test = y2[index]
    break   
for cohort in COHORTS:     
    
    
    # idx = X[:, 0] == 'Healthy Control'
    idx1 = X_train[:, 0] == cohort
    idx2 = X_test[:, 0] == cohort
    counts[f'{cohort},Test_N_Low'] = (y_test[idx2] == 0).astype(float).sum()
    counts[f'{cohort},Test_N_High'] = (y_test[idx2] == 1).astype(float).sum()
    counts[f'{cohort},Train_N_Low'] = (y_train[idx1] == 0).astype(float).sum()
    counts[f'{cohort},Train_N_High'] = (y_train[idx1] == 1).astype(float).sum()
    
# counts

def update_usage(row, what):
    cohort = row['cohort']
    return counts[f'{cohort},{what}']

# Apply transformation
for what in ['Train_N_Low', 'Train_N_High', 'Test_N_High', 'Test_N_Low']:
    agg_df[what] = agg_df.apply(lambda z:update_usage(z, what=what), axis=1)
    
    
agg_df = agg_df.rename(columns={
    # 'cohort': 'Train_Cohort', 
    'clf': 'Classifier', 
    'AUC_mean': 'CV_AUC_Mean',
    'AUC_std': 'CV_AUC_SD',
    'Sensitivity at 0.8 Specificity_mean': 'CV_Sen_at_80_Spe_Mean',
    'Sensitivity at 0.8 Specificity_std': 'CV_Sen_at_80_Spe_SD',
    'Low Responders Accuracy_mean': 'CV_Acc_Low_Mean',
    'High Responders Accuracy_mean': 'CV_Acc_High_Mean',
    'Low Responders Accuracy_std': 'CV_Acc_Low_SD',
    'High Responders Accuracy_std': 'CV_Acc_High_SD',
})

def update_usage(row):
    # Start with base usage name
    usage = row['usage']
    setting = row['setting']
    
    
    # Case 1: if usage is "Ab raw"
    
    out = 'Ab_raw'
    
    
    # Case 2: if usage is "Demographic"
    if setting.strip().lower() == 'binary':
        out += '+ Ab_binary'
        
    if usage.strip().lower() == 'demographic':
        out += '+ demographic'
    
    # Case 3: modify further based on 'setting'
    
    if setting.strip().lower() == 'no-cutoff':
        # don't add anything
        pass
    
    return out

# Apply transformation
agg_df['Feature_Set'] = agg_df.apply(update_usage, axis=1)
agg_df[['Train_Timepoint', 'Train_Cohort', 'Train_N_Low', 'Train_N_High', 'Feature_Set', 'Test_Timepoint', 'Test_Cohort', 'Test_N_Low' ,'Test_N_High', 'Classifier', 'CV_AUC_Mean', 'CV_AUC_SD',	'CV_Sen_at_80_Spe_Mean', 'CV_Sen_at_80_Spe_SD', 'CV_Acc_Low_Mean', 'CV_Acc_Low_SD',	'CV_Acc_High_Mean',	'CV_Acc_High_SD']]



,Train_Timepoint,Train_Cohort,Train_N_Low,Train_N_High,Feature_Set,Test_Timepoint,Test_Cohort,Test_N_Low,Test_N_High,Classifier,CV_AUC_Mean,CV_AUC_SD,CV_Sen_at_80_Spe_Mean,CV_Sen_at_80_Spe_SD,CV_Acc_Low_Mean,CV_Acc_Low_SD,CV_Acc_High_Mean,CV_Acc_High_SD
0,Post-B,Autoimmune: Other,58.0,130.0,Ab_raw+ Ab_binary,Pre-B,Autoimmune: Other,4.0,13.0,mlp,0.502103,0.142830,0.283873,0.218998,0.490417,0.352998,0.512023,0.296374
1,Post-B,Autoimmune: Other,58.0,130.0,Ab_raw+ Ab_binary+ demographic,Pre-B,Autoimmune: Other,4.0,13.0,mlp,0.515038,0.147047,0.302103,0.232099,0.536604,0.350811,0.489099,0.304677
2,Post-B,Autoimmune: Other,58.0,130.0,Ab_raw+ Ab_binary,Pre-B,Autoimmune: Other,4.0,13.0,nc,0.504874,0.149896,0.328394,0.211834,0.421147,0.238259,0.586698,0.153931
3,Post-B,Autoimmune: Other,58.0,130.0,Ab_raw+ Ab_binary+ demographic,Pre-B,Autoimmune: Other,4.0,13.0,nc,0.513308,0.154704,0.346185,0.214905,0.440633,0.236083,0.585957,0.166261
4,Post-B,Autoimmune: Other,58.0,130.0,Ab_raw+ Ab_binary,Pre-B,Autoimmune: Other,4.0,13.0,svm,0.513446,0.140752,0.321438,0.204253,0.715190,0.207241,0.323346,0.151572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Post-B,Transplant,63.0,60.0,Ab_raw+ demographic,Pre-B,Transplant,15.0,11.0,mlp,0.522608,0.122725,0.320928,0.186668,0.495041,0.291569,0.536702,0.280428
68,Post-B,Transplant,63.0,60.0,Ab_raw,Pre-B,Transplant,15.0,11.0,nc,0.525922,0.122243,0.341092,0.177569,0.534237,0.170507,0.477528,0.168274
69,Post-B,Transplant,63.0,60.0,Ab_raw+ demographic,Pre-B,Transplant,15.0,11.0,nc,0.513054,0.128198,0.325303,0.168747,0.474986,0.179102,0.540976,0.166879
70,Post-B,Transplant,63.0,60.0,Ab_raw,Pre-B,Transplant,15.0,11.0,svm,0.522985,0.122769,0.343226,0.163288,0.835873,0.141411,0.172387,0.120410


In [801]:
out = agg_df[['Train_Timepoint', 'Train_Cohort', 'Train_N_Low', 'Train_N_High', 'Feature_Set', 'Test_Timepoint', 'Test_Cohort', 'Test_N_Low' ,'Test_N_High', 'Classifier',	'CV_AUC_Mean', 'CV_AUC_SD',	'CV_Sen_at_80_Spe_Mean', 'CV_Sen_at_80_Spe_SD', 'CV_Acc_Low_Mean', 'CV_Acc_Low_SD',	'CV_Acc_High_Mean',	'CV_Acc_High_SD']]
out.to_csv('final_results/model_performance_summary_v6_revised.csv')

#### All

In [909]:
pd.concat(
    [pd.read_csv('final_results/model_performance_summary_v1_revised.csv'),
     pd.read_csv('final_results/model_performance_summary_v2_revised.csv'),
     pd.read_csv('final_results/model_performance_summary_v3_revised.csv'),
     pd.read_csv('final_results/model_performance_summary_v4_revised.csv'),
     pd.read_csv('final_results/model_performance_summary_v5_revised.csv'),
     pd.read_csv('final_results/model_performance_summary_v6_revised.csv'),
    ],
     ignore_index=True
).to_csv('final_results/all_final_revised.csv', index=False)

In [759]:
def ml_post_to_pre(SEED, cohort, clf, dim, use_cat, cutoffs, splits=10, binary=True, use_hmean=True, coeffs=(1, 1), p=0.2, n_epochs=300, lr=1e-2):    
    skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=SEED)
    best_models = []
    metrics = []
    # device = 'cpu'

    
    X_2, y_2, pid_2, _, __, ___  = load_2(binary=True, cutoffs=cutoffs, qcut=False, include_pid=True)
    X_6, y_6, pid_6, emb_list, real_size, encoder  = load_6(binary=True, cutoffs=cutoffs, qcut=False, include_pid=True)

    # idx = X[:, 0] == cohort
    # if (y[idx] == 0).sum() < 5:
    #     continue
    for fold, (_, test_idx) in enumerate(skf.split(X_2, y_2), start=1):
    
        index = ~np.isin(pid_6, pid_2[test_idx])
        X_train = X_6[index]
        y_train = y_6[index]
        X_test = X_2[test_idx]
        y_test = y_2[test_idx]

        
        

        if use_cat:
            start_from = len(emb_list)
        else:
            start_from = 11
        
        
        size = real_size if use_cat else real_size - 1
            
        
        X_train_tensor = torch.from_numpy(X_train[:, start_from:].astype(np.float32))
        y_train_tensor = torch.from_numpy(y_train).float()
        
        X_test_tensor = torch.from_numpy(X_test[:, start_from:].astype(np.float32))
        y_test_tensor = torch.from_numpy(y_test).float()
        
        from sklearn.model_selection import train_test_split
        train_idx, val_idx = train_test_split(
            np.arange(len(X_train_tensor)),
            test_size=0.2,
            stratify=y_train_tensor.numpy() if binary else (y_train_tensor.numpy()>3.5).astype(int),  # preserve label distribution
            random_state=SEED
        )
        
        # create datasets
        train_dataset = TensorDataset(X_train_tensor[train_idx], y_train_tensor[train_idx])
        val_dataset   = TensorDataset(X_train_tensor[val_idx],  y_train_tensor[val_idx])
        
        # data loaders
        train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, generator=torch.Generator().manual_seed(SEED))
        val_loader   = DataLoader(val_dataset,   batch_size=len(val_dataset), shuffle=False)
        test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor),
                                 batch_size=len(X_test_tensor), shuffle=False)

        
        
        # set_seed(SEED)


        # p = 0.2 if binary else 0.2
        # p = 
        # lr = 1e-2 if binary else 1e-2
        
        m = BottleneckMLP(emb_list, size, start_from, 8, (128, 64, dim), p=p, use_cat=use_cat, binary=binary).to(device)

        # print(m)
        opt = torch.optim.AdamW(m.parameters(), lr=lr, weight_decay=0)
        lf = nn.BCELoss() if binary else nn.MSELoss()
        
        best = -1 if binary else 1e50
        best_a = -1
        best_b = -1
        best_state_dict = None
        ######################################### Train for some epochs ###########################################
        for epoch in range(n_epochs):
            epoch_loss = 0
            for x_batch, y_batch in train_loader:
                m.train()
                opt.zero_grad()
                y_hat, two_d = m(x_batch.to(device))
                loss = lf(y_hat.reshape(-1), y_batch.to(device).reshape(-1))
                loss.backward()
                opt.step()
                epoch_loss += loss.item()
            if epoch % 50 == 0:
                print('Fold:', fold)
                print('Epoch:', epoch+1)
                print('Train Loss:', epoch_loss / len(train_loader))
            with torch.no_grad():
                ######################################### validation after each epoch ###########################################
                m.eval()
                for x_batch, y_batch in val_loader:
                    
                    if clf == 'mlp':
                        logits, two_d = m(x_batch.to(device))
                    else:
                        with torch.no_grad():
                            m.eval()
                            x_for_clf = []
                            y_for_clf = []
                            for x_batch_train, y_batch_train in train_loader:
                                logits, two_d = m(x_batch_train.to(device))
                                x_for_clf.extend(two_d)
                                y_for_clf.extend(y_batch_train)
                        x_for_clf = torch.stack(x_for_clf).cpu().numpy()
                        y_for_clf = torch.stack(y_for_clf).cpu().numpy()
                        if clf == 'svm':
                            one = (y_for_clf == 1).astype(float).sum()
                            zero = (y_for_clf == 0).astype(float).sum()
                            w = (one/zero)
                            delta = 1
                
                            classifier = SVC(kernel="rbf", class_weight={0: w, 1: 1}, probability=True)
                            classifier.fit(x_for_clf, y_for_clf)
                            
                            x_in = m(x_batch.to(device))[-1].cpu().numpy()
                            logits = classifier.decision_function(x_in)
                        elif clf == 'nc':
                            classifier = NearestCentroid(metric="euclidean")
                            classifier.fit(x_for_clf, y_for_clf)
                            x_in = m(x_batch.to(device))[-1].cpu().numpy()
                            logits = classifier.predict(x_in)
                            
                            
                        
                        
                    if binary:
                        if clf == 'mlp':
                            y_pred = (logits.reshape(-1) > 0.5).int().cpu()
                        elif clf == 'svm':
                            y_pred = torch.from_numpy(logits >= delta).int()
                        elif clf == 'nc':
                            y_pred = torch.from_numpy(logits).int()
                        a_val = (y_pred[y_batch==0] == y_batch[y_batch==0]).float().mean().item() + 1e-20
                        b_val = (y_pred[y_batch==1] == y_batch[y_batch==1]).float().mean().item() + 1e-20
                        ####################################### This is weighting zeros higher
                        
                        c_val = sum(coeffs) / ((coeffs[0]/a_val) + coeffs[1]/b_val)
                        # c_val = (coeffs[0] * a_val + coeffs[1] * b_val) / (coeffs[0] + coeffs[1])
                        
                        try:
                            if clf == 'svm':
                                logits = torch.sigmoid(torch.from_numpy(logits))
                            elif clf == 'nc':
                                D = pairwise_distances(x_in, classifier.centroids_, metric=classifier.metric)
                                pos_idx = np.where(classifier.classes_ == 1)[0][0]
                                logits = torch.from_numpy(-D[:, pos_idx])
                                # logits = torch.sigmoid(torch.from_numpy(logits))
                            auc_val = roc_auc_score(y_batch.cpu().numpy(), logits.reshape(-1).cpu().numpy())
                            sen_val = []
                            spec_val = []
                            for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
                                sen_, spec_ = at_95(logits.cpu().numpy(), y_batch, t=t)
                                sen_val.append(sen_)
                                spec_val.append(spec_)
                        except:
                            pass
                    else:
                        if clf != 'mlp':
                            raise NotImplementedError
                        y_labeled = (y_batch > 3.5).int()
                        zero = y_labeled == 0
                        a_val = lf(logits[zero], y_batch[zero]) ** 0.5 
                        b_val = lf(logits[~zero], y_batch[~zero]) ** 0.5
                        c_val = 2 / ((1/a_val) + 1/b_val)

                    if use_hmean:
                       condition = (binary and c_val > best) or ((not binary) and c_val < best) 
                    else:
                        condition = (binary and spec_ > best) or ((not binary) and c_val < best)
                    if condition:
                        ######################################### Record test scores at best validation epoch ###########################################
                        
                        if binary:
                            if use_hmean:
                                best = c_val
                                best_a = a_val
                                best_b = b_val
                            else:
                                best = spec_val
                        else:
                            best = c_val
                        import copy
                        best_state_dict = copy.deepcopy(m.state_dict())
                        m.eval()
                        for x_batch, y_batch in test_loader:
                            if clf == 'mlp':
                                
                                logits, two_d = m(x_batch.to(device))
                            elif clf == 'svm':
                                x_in = m(x_batch.to(device))[-1].cpu().numpy()
                                logits = classifier.decision_function(x_in)
                            elif clf == 'nc':
                                x_in = m(x_batch.to(device))[-1].cpu().numpy()
                                logits = classifier.predict(x_in)
                                
                            if binary:
                                if clf == 'mlp':
                                    y_pred = (logits.reshape(-1) > 0.5).int().cpu()
                                elif clf == 'svm':
                                    y_pred = torch.from_numpy(logits >= delta).int()
                                elif clf == 'nc':
                                    
                                    y_pred = torch.from_numpy(logits).int()
                                a_test = (y_pred[y_batch==0] == y_batch[y_batch==0]).float().mean().item() + 1e-20
                                b_test = (y_pred[y_batch==1] == y_batch[y_batch==1]).float().mean().item() + 1e-20
                                c_test = 2 / ((1/a_test) + 1/b_test)
                                if clf == 'svm':
                                    logits = torch.sigmoid(torch.from_numpy(logits))
                                elif clf == 'nc':
                                    D = pairwise_distances(x_in, classifier.centroids_, metric=classifier.metric)
                                    pos_idx = np.where(classifier.classes_ == 1)[0][0]
                                    logits = torch.from_numpy(-D[:, pos_idx])
                                    # logits = torch.sigmoid(torch.from_numpy(logits))
                                
                                auc_test = roc_auc_score(y_batch.cpu().numpy(), logits.cpu().numpy())
                                sen_test = []
                                spec_test = []
                                for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
                                    sen_, spec_ = at_95(logits.cpu().numpy(), y_batch, t=t)
                                    sen_test.append(sen_)
                                    spec_test.append(spec_)
                                
                            else:
                                if clf != 'mlp':
                                    raise NotImplementedError
                                y_labeled = (y_batch > 3.5).int()
                                zero = y_labeled == 0
                                a_test = lf(logits[zero], y_batch[zero]) ** 0.5 
                                b_test = lf(logits[~zero], y_batch[~zero]) ** 0.5
                                c_test = 2 / ((1/a_test) + 1/b_test)
                                auc_test = 0
            if epoch % 50 == 0:
                if not binary:
                    print(f'Fold {fold}, Test Metric: ({a_test:.4f}, {b_test:.4f}, {c_test:.4f}, {auc_test:.4f}), Val Best mean: {best}')
                else:
                    print(f'Val Best mean: ({best:.4f}, {best_a:.4f}, {best_b:.4f})')
                    print(f'Test Metric at best Validation Epoch: ({a_test:.4f}, {b_test:.4f}, {c_test:.4f}, {auc_test:.4f}, {sen_test[0]:.4f}, {spec_test[0]:.4f})')
                    
                print(f'{splits}-fold Average: {torch.tensor(metrics).mean(0)}')
                print('----------------------'*2)
        best_models.append(best_state_dict)
        
        if binary:
            metrics.append([a_test, b_test, c_test, auc_test, *spec_test, *sen_test])
            # metrics.extend(spec_test)
            # metrics.extend(sen_test)
        else:
            metrics.append([a_test, b_test, c_test, auc_test])
        print('============================================='*2)

    return best_models, skf, torch.tensor(metrics).mean(0)


In [765]:
# len(results_6_excel.keys())
len(results_8_excel.keys())

7200

In [760]:
post_to_pre = {}
for seed in range(10):
    bm, _, metrics = ml_post_to_pre(
        seed, 'mlp', 2, True, None, 5, binary=True, use_hmean=True, coeffs=(4, 1), p=0.2, n_epochs=300, lr=1e-2)
    post_to_pre[seed] = (bm, _, metrics)

Fold: 1
Epoch: 1
Train Loss: 0.6519887149333954
Val Best mean: (0.0000, 0.0000, 1.0000)
Test Metric at best Validation Epoch: (0.0000, 1.0000, 0.0000, 0.5156, 0.0000, 0.0606)
5-fold Average: nan
--------------------------------------------
Fold: 1
Epoch: 51
Train Loss: 0.05214878963306546
Val Best mean: (0.5690, 0.5217, 0.8929)
Test Metric at best Validation Epoch: (0.5455, 0.8250, 0.6567, 0.7737, 0.6937, 0.6364)
5-fold Average: nan
--------------------------------------------
Fold: 1
Epoch: 101
Train Loss: 0.05329474760219455
Val Best mean: (0.5864, 0.5435, 0.8571)
Test Metric at best Validation Epoch: (0.5758, 0.8375, 0.6824, 0.7927, 0.7500, 0.8182)
5-fold Average: nan
--------------------------------------------


KeyboardInterrupt: 

In [600]:
metrics = {}
device = 'cpu'

for k , v in post_to_pre.items():
    seed = k
    seed = int(seed)
    setting = 'no-cutoff'
    clf = 'mlp'
    
    X, y , emb_list, real_size, encoder = load_2(binary=True, cutoffs=None, qcut=False)
    _, __ , emb_list, real_size, encoder = load_6(binary=True, cutoffs=None, qcut=False)
    en = {}
    for k_cohort, v_cohort in (X[:, [0, 5]]):
        en[k_cohort] = v_cohort
     
    skf = v[1]
    
    
    for_stratify = y
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, for_stratify), start=0):
        #################################################### Dataset
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        from torch.utils.data import TensorDataset, DataLoader, random_split
        
        use_cat = True

        if use_cat:
            start_from = len(emb_list)
        else:
            start_from = 11
        
        
        size = real_size if use_cat else real_size - 1
        
        X_train_tensor = torch.from_numpy(X_train[:, start_from:].astype(np.float32))
        y_train_tensor = torch.from_numpy(y_train).float()
        
        X_test_tensor = torch.from_numpy(X_test[:, start_from:].astype(np.float32))
        y_test_tensor = torch.from_numpy(y_test).float()
        
        from sklearn.model_selection import train_test_split
        # stratified split using sklearn
        train_idx, val_idx = train_test_split(
            np.arange(len(X_train_tensor)),
            test_size=0.2,
            stratify=y_train_tensor.numpy() ,
            random_state=seed
        )
        
        # create datasets
        train_dataset = TensorDataset(X_train_tensor[train_idx], y_train_tensor[train_idx])
        val_dataset   = TensorDataset(X_train_tensor[val_idx],  y_train_tensor[val_idx])
        
        # data loaders
        train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, generator=torch.Generator().manual_seed(seed))
        val_loader   = DataLoader(val_dataset,   batch_size=len(val_dataset), shuffle=False)
        test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor),
                                 batch_size=len(X_test_tensor), shuffle=False)
        m = BottleneckMLP(emb_list, size, start_from, 8, (128, 64, 2), p=0.2, use_cat=use_cat, binary=True).to(device)
        m.load_state_dict(v[0][fold])
        
        for cohort in en.keys():
            for x_batch, y_batch in test_loader:
                pass
            index = x_batch[:, 0] == en[cohort]
            
            if (y_batch[index] == 0).float().sum() < 1:
                print(cohort, fold, seed, 'not enough negative samples')
                continue
            if clf == 'mlp':
                with torch.no_grad():
                    m.eval()
                    logits, two_d = m(x_batch[index].to(device))
            else:
                with torch.no_grad():
                    x_for_clf = []
                    y_for_clf = []
                    for x_batch_train, y_batch_train in train_loader:
                        with torch.no_grad():
                            m.eval()
                            _, two_d = m(x_batch_train.to(device))
                        x_for_clf.extend(two_d)
                        y_for_clf.extend(y_batch_train)
                x_for_clf = torch.stack(x_for_clf).cpu().numpy()
                y_for_clf = torch.stack(y_for_clf).cpu().numpy()
                if clf == 'svm':
                    one = (y_for_clf == 1).astype(float).sum()
                    zero = (y_for_clf == 0).astype(float).sum()
                    w = (one/zero)
                    delta = 1
        
                    classifier = SVC(kernel="rbf", class_weight={0: w, 1: 1}, probability=True)
                    classifier.fit(x_for_clf, y_for_clf)
                    with torch.no_grad():
                        m.eval()
                        x_in = m(x_batch[index].to(device))[-1].cpu().numpy()
                    logits = classifier.decision_function(x_in)
                elif clf == 'nc':
                    classifier = NearestCentroid(metric="euclidean")
                    classifier.fit(x_for_clf, y_for_clf)
                    with torch.no_grad():
                        m.eval()
                        x_in = m(x_batch[index].to(device))[-1].cpu().numpy()
                    logits = classifier.predict(x_in)

            if 1:
                if clf == 'mlp':
                    y_pred = (logits.reshape(-1) > 0.5).int().cpu()
                elif clf == 'svm':
                    y_pred = torch.from_numpy(logits >= delta).int()
                elif clf == 'nc':
                    
                    y_pred = torch.from_numpy(logits).int()
                a_test = (y_pred[y_batch[index]==0] == y_batch[index][y_batch[index]==0]).float().mean().item() + 1e-20
                b_test = (y_pred[y_batch[index]==1] == y_batch[index][y_batch[index]==1]).float().mean().item() + 1e-20
                c_test = 2 / ((1/a_test) + 1/b_test)
                if clf == 'svm':
                    logits = torch.sigmoid(torch.from_numpy(logits))
                elif clf == 'nc':
                    D = pairwise_distances(x_in, classifier.centroids_, metric=classifier.metric)
                    pos_idx = np.where(classifier.classes_ == 1)[0][0]
                    logits = torch.from_numpy(-D[:, pos_idx])
                    # logits = torch.sigmoid(torch.from_numpy(logits))

                try:
                    auc_test = roc_auc_score(y_batch[index].cpu().numpy(), logits.cpu().numpy())
                except:
                    auc_test = None
                sen_test = []
                spec_test = []
                for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
                    sen_, spec_ = at_95(logits.cpu().numpy(), y_batch[index], t=t)
                    sen_test.append(sen_)
                    spec_test.append(spec_)
                    
            
            metrics[f'{seed}, {fold}, {cohort}, {clf}, {setting}'] = ([a_test, b_test, c_test, auc_test, *spec_test, *sen_test])                        
    
res_2_specific = {}
for k, v in metrics.items():
    
    seed, fold, cohort, clf, setting = k.split(', ')
    
    try:
        res_2_specific[f'{seed},{cohort},{clf},{setting}'].append(v)
    except:
        res_2_specific[f'{seed},{cohort},{clf},{setting}'] = [v]



final_2_specific = {}
for k, v in res_2_specific.items():
    
    seed, cohort, clf, setting = k.split(',')
    
    try:
        final_2_specific[f'{seed},{cohort},{clf},{setting}'].append(torch.tensor(v).mean(0))
    except:
        final_2_specific[f'{seed},{cohort},{clf},{setting}'] = [torch.tensor(v).mean(0)]


final_2_cohort_spec = {}
for k, v in final_2_specific.items():
    seed, cohort, clf, setting = k.split(',')
    
    try:
        final_2_cohort_spec[f'{cohort},{clf},{setting}'].append(torch.tensor(v[0]))
    except:
        final_2_cohort_spec[f'{cohort},{clf},{setting}'] = [torch.tensor(v[0])]

values = []
for k, v in final_2_cohort_spec.items():
    
    cohort, clf, setting = k.split(',')
    
    values.append([cohort, setting, clf, *torch.stack(v).mean(0).numpy().tolist()])
        
columns = []
for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
    columns.append(f'Sensitivity at {t} Specificity')

for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
    columns.append(f'Specificity at {t} Sensitivity')
pd.DataFrame(values, 
             columns=[
                 'Cohort', 'Cutoff', 'Classifier', 'Low Responders Accuracy', 'High Responders Accuracy', 'Harmonic Mean',
                 'AUC'
             ]+columns).to_csv('on pre from post unseen pid_oct 16.csv')
    
    

HIV 0 0 not enough negative samples
IBD 0 0 not enough negative samples
Cancer: Other 1 0 not enough negative samples
IBD 1 0 not enough negative samples
IBD 3 0 not enough negative samples
Cancer: Other 0 1 not enough negative samples
IBD 1 1 not enough negative samples
Cancer: Other 3 1 not enough negative samples
IBD 3 1 not enough negative samples
Cancer: Other 2 2 not enough negative samples
IBD 2 2 not enough negative samples
IBD 3 2 not enough negative samples
Cancer: Other 1 3 not enough negative samples
IBD 1 3 not enough negative samples
IBD 2 3 not enough negative samples
IBD 4 3 not enough negative samples
IBD 0 4 not enough negative samples
Cancer: Other 4 4 not enough negative samples
IBD 4 4 not enough negative samples
IBD 0 5 not enough negative samples
Cancer: Other 3 5 not enough negative samples
IBD 3 5 not enough negative samples
Healthy Control 4 5 not enough negative samples
IBD 4 5 not enough negative samples
Cancer: Other 1 6 not enough negative samples
IBD 1 6 

/tmp/ipykernel_3176679/2421434863.py:170: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  final_2_cohort_spec[f'{cohort},{clf},{setting}'] = [torch.tensor(v[0])]
/tmp/ipykernel_3176679/2421434863.py:168: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  final_2_cohort_spec[f'{cohort},{clf},{setting}'].append(torch.tensor(v[0]))


In [466]:
from_6_to_2 = {}
import tqdm

for SEED in tqdm.tqdm(range(100)):
    cutoffs = None
    X_6, y_6 , emb_list, real_size, encoder  = load_6(binary=True, cutoffs=cutoffs, qcut=False)
    best_models, metrics = train_on_whole(X_6, y_6, SEED, binary=True, p=0.2, n_epochs=101, lr=1e-3, use_hmean=True, splits=5)
    
    X_2, y_2 , emb_list, real_size, encoder  = load_2(binary=True, cutoffs=cutoffs, qcut=False)
    en = {}
    for k_cohort, v_cohort in (X_2[:, [0, 5]]):
        en[k_cohort] = v_cohort
    
    X_test_tensor_2 = torch.from_numpy(X_2[:, 5:].astype(np.float32))
    y_test_tensor_2 = torch.from_numpy(y_2).float()
    test_loader_2 = DataLoader(
        TensorDataset(X_test_tensor_2, y_test_tensor_2),
        batch_size=len(X_test_tensor_2), shuffle=False
    )
    for cohort in en.keys():
        for x_batch, y_batch in test_loader_2:
            pass
        
        index = x_batch[:, 0] == en[cohort]
        if (y_batch[index] == 0).float().sum() < 1:
            print(cohort, 'not enough negative samples')
            continue
        with torch.no_grad():
            pred = 0
            for model in best_models:
                m = BottleneckMLP(None, 98, 0, 8, (128, 64, 2), p=0.2, use_cat=False, binary=True).to(device)
                m.load_state_dict(model)
                
                m.eval()
                logits, two_d = m(x_batch[index, 6:].to(device))
                logits = torch.logit(logits) 
                pred += logits/len(best_models)
    
        logits = torch.sigmoid(pred)
        y_pred = (logits.reshape(-1) > 0.5).int().cpu()
        a_test = (y_pred[y_batch[index]==0] == y_batch[index][y_batch[index]==0]).float().mean().item() + 1e-20
        b_test = (y_pred[y_batch[index]==1] == y_batch[index][y_batch[index]==1]).float().mean().item() + 1e-20
        c_test = 2 / ((1/a_test) + 1/b_test)
        auc_test = roc_auc_score(y_batch[index].cpu().numpy(), logits.cpu().numpy())
        sen_test = []
        spec_test = []
        for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
            sen_, spec_ = at_95(logits.cpu().numpy(), y_batch[index], t=t)
            sen_test.append(sen_)
            spec_test.append(spec_)
    
        from_6_to_2[f'{SEED}, {cohort}'] = [a_test, b_test, c_test, auc_test, *spec_test, *sen_test]


  0%|                                                   | 0/100 [00:00<?, ?it/s]

Fold: 1
Epoch: 1
Train Loss: 0.8959692716598511
Fold: 1
Epoch: 51
Train Loss: 0.44814371317625046
Fold: 1
Epoch: 101
Train Loss: 0.17823296040296555
Fold: 2
Epoch: 1
Train Loss: 0.9043856859207153
Fold: 2
Epoch: 51
Train Loss: 0.45703519880771637
Fold: 2
Epoch: 101
Train Loss: 0.18659166246652603
Fold: 3
Epoch: 1
Train Loss: 0.9108399152755737
Fold: 3
Epoch: 51
Train Loss: 0.4509768411517143
Fold: 3
Epoch: 101
Train Loss: 0.18228697776794434
Fold: 4
Epoch: 1
Train Loss: 0.9014526009559631
Fold: 4
Epoch: 51
Train Loss: 0.45278850942850113
Fold: 4
Epoch: 101
Train Loss: 0.19305483996868134
Fold: 5
Epoch: 1
Train Loss: 0.9045437872409821
Fold: 5
Epoch: 51
Train Loss: 0.4368788078427315
Fold: 5
Epoch: 101
Train Loss: 0.17604288458824158


  1%|▍                                          | 1/100 [00:34<56:48, 34.43s/it]

Fold: 1
Epoch: 1
Train Loss: 0.9638942331075668
Fold: 1
Epoch: 51
Train Loss: 0.4584580957889557
Fold: 1
Epoch: 101
Train Loss: 0.18830692395567894
Fold: 2
Epoch: 1
Train Loss: 0.9563350826501846
Fold: 2
Epoch: 51
Train Loss: 0.48010604083538055
Fold: 2
Epoch: 101
Train Loss: 0.1951989307999611
Fold: 3
Epoch: 1
Train Loss: 0.9542172700166702
Fold: 3
Epoch: 51
Train Loss: 0.47426439821720123
Fold: 3
Epoch: 101
Train Loss: 0.19265303388237953
Fold: 4
Epoch: 1
Train Loss: 0.9554267972707748
Fold: 4
Epoch: 51
Train Loss: 0.47423188388347626
Fold: 4
Epoch: 101
Train Loss: 0.19610107690095901
Fold: 5
Epoch: 1
Train Loss: 0.9351204335689545
Fold: 5
Epoch: 51
Train Loss: 0.4696728512644768
Fold: 5
Epoch: 101
Train Loss: 0.18891289830207825


  2%|▊                                          | 2/100 [01:06<53:47, 32.93s/it]

...

 98%|█████████████████████████████████████████▏| 98/100 [49:01<00:57, 28.64s/it]

Fold: 1
Epoch: 1
Train Loss: 0.7039597630500793
Fold: 1
Epoch: 51
Train Loss: 0.40163780748844147
Fold: 1
Epoch: 101
Train Loss: 0.17489475011825562
Fold: 2
Epoch: 1
Train Loss: 0.7075667083263397
Fold: 2
Epoch: 51
Train Loss: 0.4011826664209366
Fold: 2
Epoch: 101
Train Loss: 0.18400012329220772
Fold: 3
Epoch: 1
Train Loss: 0.7024184316396713
Fold: 3
Epoch: 51
Train Loss: 0.4046439006924629
Fold: 3
Epoch: 101
Train Loss: 0.178195770829916
Fold: 4
Epoch: 1
Train Loss: 0.7058202922344208
Fold: 4
Epoch: 51
Train Loss: 0.4000747427344322
Fold: 4
Epoch: 101
Train Loss: 0.18228304013609886
Fold: 5
Epoch: 1
Train Loss: 0.7028413563966751
Fold: 5
Epoch: 51
Train Loss: 0.40669404715299606
Fold: 5
Epoch: 101
Train Loss: 0.18756673857569695


 99%|█████████████████████████████████████████▌| 99/100 [49:32<00:29, 29.24s/it]

Fold: 1
Epoch: 1
Train Loss: 0.7530753761529922
Fold: 1
Epoch: 51
Train Loss: 0.39303451031446457
Fold: 1
Epoch: 101
Train Loss: 0.16576632857322693
Fold: 2
Epoch: 1
Train Loss: 0.7585173100233078
Fold: 2
Epoch: 51
Train Loss: 0.37912190705537796
Fold: 2
Epoch: 101
Train Loss: 0.16328388452529907
Fold: 3
Epoch: 1
Train Loss: 0.7651347368955612
Fold: 3
Epoch: 51
Train Loss: 0.38653507083654404
Fold: 3
Epoch: 101
Train Loss: 0.16558435931801796
Fold: 4
Epoch: 1
Train Loss: 0.7568597197532654
Fold: 4
Epoch: 51
Train Loss: 0.3837090730667114
Fold: 4
Epoch: 101
Train Loss: 0.16257188469171524
Fold: 5
Epoch: 1
Train Loss: 0.7610738426446915
Fold: 5
Epoch: 51
Train Loss: 0.3800748139619827
Fold: 5
Epoch: 101
Train Loss: 0.17045273631811142


100%|█████████████████████████████████████████| 100/100 [50:01<00:00, 30.02s/it]


In [467]:
final = {}
values = []
for k, v in from_6_to_2.items():
    seed, cohort = k.split(', ')
    
    try:
        final[cohort].append(torch.tensor(v))
    except:
        final[cohort] = [torch.tensor(v)]

for k, v in final.items():
    values.append([k, *torch.stack(v).mean(0).numpy().tolist()])
        
    columns = []
    for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
        columns.append(f'Sensitivity at {t} Specificity')
    
    for t in [0.75, 0.8, 0.85, 0.9, 0.95]:
        columns.append(f'Specificity at {t} Sensitivity')
pd.DataFrame(values, 
             columns=[
                 'Cohort', 'Low Responders Accuracy', 'High Responders Accuracy', 'Harmonic Mean',
                 'AUC'
             ]+columns).to_csv('on_pre_from_universal_no_categorical_features_post_model_oct_15.csv')
    

In [447]:
with torch.no_grad():
    pred = 0
    for model in best_models:
        m = BottleneckMLP(None, 98, 0, 8, (128, 64, 2), p=0.2, use_cat=False, binary=True).to(device)
        m.load_state_dict(model)
        
        m.eval()
        logits, two_d = m(x_batch[index, 6:].to(device))
        logits = torch.logit(logits) 
        pred += logits/len(best_models)

{'0, IBD': [1.0,
  0.9642857313156128,
  0.9818181906455803,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 '1, IBD': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 '2, IBD': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 '3, IBD': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 '4, IBD': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 '5, IBD': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 '6, IBD': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 '7, IBD': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 '8, IBD': [1.0,
  0.8571428656578064,
  0.923076928014586,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1

In [451]:
metrics

[[0.5849056839942932,
  0.7250000238418579,
  0.6474613131377297,
  0.6788679245283019],
 [0.7169811129570007,
  0.7200000286102295,
  0.7184873996036123,
  0.7679245283018868],
 [0.6792452931404114,
  0.7035176157951355,
  0.6911684224131646,
  0.7364179387503555],
 [0.698113203048706, 0.7286432385444641, 0.713051576724689, 0.729686166682469],
 [0.6346153616905212, 0.7599999904632568, 0.6916712455341599, 0.71875]]

In [ ]:
def _merge_simple(df, group):

    which = group.split('_')[0]
    pre = df[df[group] == f'Pre {which}']
    post = df[df[group] == f'Post {which}']
    
    result = pd.merge(pre, post, on="PID", how="inner")
    
    cols = ['Sample_ID_x', 'Sample_ID_y', 'PID', 'Disease_Cohort_x', 'Disease_Cohort_y', 'V0_Group_x', 'V0_Group_y',
           'B0_Group_x', 'B0_Group_y', 'SARS-CoV-2 wRBD_x', 'SARS-CoV-2 wRBD_y']
    cols += sorted(result.drop(cols, axis=1).columns)
    new = result[cols]
    data = new[new.columns[9:]]
    
    return data, new


def _merge_disease(df, group, disease):
    which = group.split('_')[0]
    
    pre = df[df[group] == f'Pre {which}']
    pre = pre[pre['Disease_Cohort'] == disease]
    
    post = df[df[group] == f'Post {which}']
    post = post[post['Disease_Cohort'] == disease]
    
    result = pd.merge(pre, post, on="PID", how="inner")
    
    cols = ['Sample_ID_x', 'Sample_ID_y', 'PID', 'Disease_Cohort_x', 'Disease_Cohort_y', 'V0_Group_x', 'V0_Group_y',
           'B0_Group_x', 'B0_Group_y', 'SARS-CoV-2 wRBD_x', 'SARS-CoV-2 wRBD_y']
    cols += sorted(result.drop(cols, axis=1).columns)
    new = result[cols]
    
    data = new[new.columns[9:]] 

    return data, new


def _xy(data, new, t, exclude_hiv, return_types):
    if exclude_hiv:
            x_cols = [i for i in data.columns[2:] if ('_x' in i and 'HIV' not in i)]
    else:
        x_cols = [i for i in data.columns[2:] if '_x' in i]
    
    X = data[x_cols].values
    y = (data[data.columns[1]] > t).astype(int).values

    if return_types:
        dis_types = new['Disease_Cohort_x']
        return X, y, dis_types
    else:
        return X, y


def _post_xy(df, group, t, exclude_hiv, return_types):
    which = group.split('_')[0]
    
    post = df[df[group] == f'Post {which}']
    data = post[post.columns[5:]]
    if exclude_hiv:
        x_cols = [i for i in data.columns[1:] if 'HIV' not in i]
    else:
        x_cols = data.columns[1:]
    X = data[x_cols].values
    y = (data[data.columns[0]] > t).astype(int).values

    if return_types:
        dis_types = post['Disease_Cohort']
    if return_types:
        return X, y, dis_types
    else:
        return X, y

def _post_xy_disease(df, group, disease, t, exclude_hiv, return_types):
    which = group.split('_')[0]
    
    post = df[df[group] == f'Post {which}']
    post = post[post['Disease_Cohort'] == disease]
    data = post[post.columns[5:]]    
    if exclude_hiv:
        x_cols = [i for i in data.columns[1:] if 'HIV' not in i]
    else:
        x_cols = data.columns[1:]
    X = data[x_cols].values
    y = (data[data.columns[0]] > t).astype(int).values

    if return_types:
        dis_types = post['Disease_Cohort']
    if return_types:
        return X, y, dis_types
    else:
        return X, y

In [ ]:
def load_1(t=0.5, path='Leidos_data_v3.csv', exclude_hiv=False, return_types=False):
    df = pd.read_csv(path)
    
    data, new = _merge_simple(df, group='V0_Group')
    return _xy(data, new, t, exclude_hiv, return_types)
    



In [ ]:
def load_2(t=3.5, path='Leidos_data_v3.csv', exclude_hiv=False, return_types=False):
    df = pd.read_csv(path)
    
    data, new = _merge_simple(df, group='B0_Group')
    return _xy(data, new, t, exclude_hiv, return_types)
    

In [ ]:
def load_3(t=0.5, path='Leidos_data_v3.csv', exclude_hiv=False, other_diseases=False):
    df = pd.read_csv(path)  
    if other_diseases:
        disease_types = pd.unique(df['Disease_Cohort'])
        ret = {k: None for k in disease_types}
        for disease in disease_types:
            data, _ = _merge_disease(df, group='V0_Group', disease=disease)
            X, y = _xy(data, _, t, exclude_hiv, return_types=False)
            
            ret[disease] = (X, y)
        return ret
    
    data, _ = _merge_disease(df, group='V0_Group', disease='Healthy Control')
    X, y = _xy(data, _, t, exclude_hiv, return_types=False)
    
    return X, y

In [ ]:
def load_4(t=3.5, path='Leidos_data_v3.csv', exclude_hiv=False, other_diseases=False):
    df = pd.read_csv(path)
    if other_diseases:
        disease_types = pd.unique(df['Disease_Cohort'])
        for disease in disease_types:
            data, _ = _merge_disease(df, group='B0_Group', disease=disease)
            X, y = _xy(data, _, t, exclude_hiv, return_types=False)
            
            ret[disease] = (X, y)
        return ret
    
    data, _ = _merge_disease(df, group='B0_Group', disease='Healthy Control')
    X, y = _xy(data, _, t, exclude_hiv, return_types=False)
    return X, y


In [ ]:
def load_5(t=0.5, path='Leidos_data_v3.csv', exclude_hiv=False, return_types=False):
    df = pd.read_csv(path)
    return _post_xy(df, group='V0_Group', t=t, exclude_hiv=exclude_hiv, return_types=return_types)


In [ ]:
def load_6(t=3.5, path='Leidos_data_v3.csv', exclude_hiv=False, return_types=False):
    df = pd.read_csv(path)
    return _post_xy(df, group='B0_Group', t=t, exclude_hiv=exclude_hiv, return_types=return_types)


In [ ]:
def load_7(t=0.5, path='Leidos_data_v3.csv', exclude_hiv=False, other_diseases=False):
    df = pd.read_csv('Leidos_data_v3.csv')
    
    if other_diseases:
        disease_types = pd.unique(df['Disease_Cohort'])
        ret = {k: None for k in disease_types}
        for disease in disease_types:
            X, y = _post_xy_disease(df, group='V0_Group', disease=disease, t=t, exclude_hiv=exclude_hiv, return_types=False)
            ret[disease] = (X, y)
        return ret
        
    X, y = _post_xy_disease(df, group='V0_Group', disease='Healthy Control', t=t, exclude_hiv=exclude_hiv, return_types=False)

    return X, y


In [ ]:
def load_8(t=3.5, path='Leidos_data_v3.csv', exclude_hiv=False, other_diseases=False):
    df = pd.read_csv('Leidos_data_v3.csv')
    
    if other_diseases:
        disease_types = pd.unique(df['Disease_Cohort'])
        ret = {k: None for k in disease_types}
        for disease in disease_types:
            X, y = _post_xy_disease(df, group='B0_Group', disease=disease, t=t, exclude_hiv=exclude_hiv, return_types=False)
            ret[disease] = (X, y)
        return ret
        
    X, y = _post_xy_disease(df, group='B0_Group', disease='Healthy Control', t=t, exclude_hiv=exclude_hiv, return_types=False)

    return X, y


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes=[128, 64], dropout_rate=0.3):
        super(MLP, self).__init__()

        layers = []
        in_dim = input_size

        for h in hidden_sizes:
            layers.append(nn.Linear(in_dim, h))
            layers.append(nn.BatchNorm1d(h, track_running_stats=False))     
            layers.append(nn.ReLU())             
            layers.append(nn.Dropout(dropout_rate))  
            in_dim = h

        layers.append(nn.Linear(in_dim, 1))     
        self.model = nn.Sequential(*layers)
        self.hidden = nn.Sequential(*layers[:-3])

    def forward(self, x, return_hidden=False):
        if return_hidden:
            return self.hidden(x)
        return torch.sigmoid(self.model(x))
        # return self.model(x)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BinaryFocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(BinaryFocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # prob = torch.sigmoid(inputs)
        prob = 1 * inputs
        prob = torch.clamp(prob, 1e-6, 1 - 1e-6)  # for numerical stability

        loss_pos = -self.alpha * (1 - prob) ** self.gamma * targets * torch.log(prob)
        loss_neg = -(1 - self.alpha) * prob ** self.gamma * (1 - targets) * torch.log(1 - prob)
        loss = loss_pos + loss_neg

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss


In [ ]:
from sklearn.metrics import (
    roc_auc_score, accuracy_score,
    precision_score, recall_score, f1_score
)
import numpy as np


def _metrics_binary(y_true, y_pred_prob, threshold=0.5):
    """
    Compute overall accuracy plus separate accuracies
    for the positive (label-1) and negative (label-0) classes,
    along with AUC, precision, recall, and F1.
    """
    y_true = y_true.reshape(-1)
    y_pred_prob = y_pred_prob.reshape(-1)
    y_pred = (y_pred_prob > threshold).astype(int)

    # ---- accuracies --------------------------------------------------
    acc_overall = accuracy_score(y_true, y_pred)

    pos_mask = y_true == 1
    neg_mask = y_true == 0
    # protect against a split that has no positives / negatives
    acc_pos = np.nan if pos_mask.sum() == 0 else (y_pred[pos_mask] == 1).mean()
    acc_neg = np.nan if neg_mask.sum() == 0 else (y_pred[neg_mask] == 0).mean()

    # ---- other metrics ----------------------------------------------
    auc = roc_auc_score(y_true, y_pred_prob)                # raises only if y_true single-class
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec  = recall_score   (y_true, y_pred, zero_division=0)
    f1   = f1_score       (y_true, y_pred, zero_division=0)

    return dict(
        accuracy_overall=acc_overall,
        accuracy_pos=acc_pos,
        accuracy_neg=acc_neg,
        auc=auc,
        precision=prec,
        recall=rec,
        f1=f1,
    )

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

def average_metrics_list(metrics_list):
    keys = metrics_list[0].keys()
    avg_dict = {}
    for key in keys:
        avg_dict[key] = np.mean([m[key] for m in metrics_list])
    return avg_dict

def train_one_fold(X_tensor, y_tensor, train_idx, val_idx, config):
    X_train, X_val = X_tensor[train_idx].to(device), X_tensor[val_idx].to(device)
    y_train, y_val = (
        y_tensor[train_idx].to(device).unsqueeze(1),
        y_tensor[val_idx].to(device).unsqueeze(1),
    )

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    val_preds = []
    all_metrics = {'train_inner': [], 'tune': [], 'val': []}

    val_pred_models = []

    for inner_train_idx, tune_idx in skf.split(X_train.cpu().numpy(), y_train.cpu().numpy()):
        # prepare split
        X_train_inner = X_train[inner_train_idx]
        y_train_inner = y_train[inner_train_idx]
        X_tune        = X_train[tune_idx]
        y_tune        = y_train[tune_idx]

        set_seed(42)
        model = MLP(
            input_size=X_tune.shape[-1],
            hidden_sizes=config['hidden_sizes'],
            dropout_rate=config['dropout_rate'],
        ).to(device)

        optimizer = optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=9e-2)
        ########################################################################
        criterion = BinaryFocalLoss(
            alpha=(1 - y_train_inner.mean()),
            # alpha=0.05,
            gamma=2
        )
        # criterion = nn.BCELoss()
        #########################################################################

        best_m_tune = -1
        best_model_sd = None

        for epoch in range(config['epochs']):
            model.train()
            optimizer.zero_grad()

            out = model(X_train_inner.squeeze(1))
            out_prob = torch.sigmoid(out)
            loss = criterion(out_prob.reshape(-1), y_train_inner.reshape(-1))
            loss.backward()
            optimizer.step()

            model.eval()
            with torch.no_grad():
                prob_tune  = model(X_tune.squeeze(1)).cpu().numpy()
                pred_tune  = (prob_tune >= 0.5).astype(int)
                metric_tune = accuracy_score(y_tune.cpu().numpy().reshape(-1), pred_tune.reshape(-1))

            if metric_tune > best_m_tune:
                best_m_tune = metric_tune
                best_model_sd = {k: v.clone().detach().cpu() for k, v in model.state_dict().items()}

        # evaluate best model
        model.load_state_dict(best_model_sd)
        model.eval()
        with torch.no_grad():
            prob_val   = model(X_val.squeeze(1))
            prob_train = model(X_train_inner.squeeze(1)).cpu().numpy()
            prob_tune  = model(X_tune.squeeze(1)).cpu().numpy()

        val_preds.append(prob_val)
        all_metrics['train_inner'].append(_metrics_binary(y_train_inner.cpu().numpy(), prob_train))
        all_metrics['tune'].append(_metrics_binary(y_tune.cpu().numpy(), prob_tune))
        all_metrics['val'].append(_metrics_binary(y_val.cpu().numpy(), prob_val.cpu().numpy()))

    # average predictions over all models
    vp = torch.mean(
                torch.stack(val_preds).squeeze(-1), dim=0
            )
    avg_val_pred = vp.cpu().numpy()
    all_metrics['val'] = [_metrics_binary(y_val.cpu().numpy(), avg_val_pred)]

    # average metrics
    avg_metrics = {
        'train_inner': average_metrics_list(all_metrics['train_inner']),
        'tune':        average_metrics_list(all_metrics['tune']),
        'val':         average_metrics_list(all_metrics['val']),
    }

    return None, avg_metrics


In [ ]:
from itertools import product
import numpy as np
import torch
from sklearn.model_selection import StratifiedKFold


def run_search(X, y):
    # -------- shape handling (keeps original behaviour) -----------------------
    if X.ndim == 2:                      # [N, F]  →  [N, 1, F]
        X = X[:, np.newaxis, :]
    elif X.shape[1] != 1:               # [N, C≠1, L]  →  [N, 1, L]
        X = np.transpose(X, (0, 2, 1))

    X_tensor = torch.from_numpy(X).float()
    y_tensor = torch.tensor(y).float()

    # -------- search grid -----------------------------------------------------
    hidden = ([32], [16])
    lrs    = (1e-1, 1e-2)
    drops  = (0.0, 0.1, 0.3)
    
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    # -------- containers for outer-fold results -------------------------------
    fold_best_models  = []
    fold_best_metrics = []     # stores metrics dict from train_one_fold
    fold_best_config  = []

    # -------------------------------------------------------------------------
    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_tensor, y_tensor), 1):
        candidates = []

        for h, lr_, d in product(hidden, lrs, drops):
            config = dict(hidden_sizes=h, dropout_rate=d, lr=lr_, epochs=500)

            model_sd, metrics = train_one_fold(
                X_tensor, y_tensor, train_idx, val_idx, config
            )
            # metrics['tune']['auc'] decides winner
            candidates.append(dict(
                tune_auc=metrics['tune']['auc'],
                tune_acc=metrics['tune']['accuracy_neg'],
                model_sd=model_sd,
                config=config,
                metrics=metrics,
            ))

        # pick the one with highest tune-set AUC/ACC
        best = max(candidates, key=lambda c: c['tune_auc'])

        # -------- save results for this fold ----------------------------------
        fold_best_models.append(best['model_sd'])
        fold_best_metrics.append(best['metrics'])
        fold_best_config.append(best['config'])

        # -------- console summary ---------------------------------------------
        print(f"\nFold {fold_idx:2d} | best tune AUC = {best['tune_auc']:.4f}")
        print("  hyper-params:", best['config'])
        for split in ['train_inner', 'tune', 'val']:
            m = best['metrics'][split]
            print(
                f"  {split:<11}: "
                f"acc={m['accuracy_overall']:.4f} "
                f"(pos={m['accuracy_pos']:.4f}, neg={m['accuracy_neg']:.4f}) | "
                f"auc={m['auc']:.4f}  prec={m['precision']:.4f}  "
                f"rec={m['recall']:.4f}  f1={m['f1']:.4f}"
            )

    # -------------------------------------------------------------------------
    return fold_best_models, fold_best_metrics, fold_best_config


def show_res(metrics):
    import numpy as np

    splits = {
        'val':         'Validation',
        'tune':        'Tuning',
        'train_inner': 'Fitting',
    }

    for split_key, split_name in splits.items():
        # --- mean across the ten folds ---------------------------------------
        mean_overall = np.mean([m[split_key]['accuracy_overall'] for m in metrics])
        mean_low     = np.mean([m[split_key]['accuracy_neg']     for m in metrics])
        mean_high    = np.mean([m[split_key]['accuracy_pos']     for m in metrics])
        mean_auc     = np.mean([m[split_key]['auc']              for m in metrics])
        mean_prec    = np.mean([m[split_key]['precision']        for m in metrics])
        mean_rec     = np.mean([m[split_key]['recall']           for m in metrics])
        mean_f1      = np.mean([m[split_key]['f1']               for m in metrics])

        # --- nicely formatted printout ---------------------------------------
        print(f"\n===== 10-fold CV – mean {split_name} metrics =====")
        print(f"Overall accuracy        : {mean_overall:.4f}")
        print(f"  • Low-resp accuracy   : {mean_low:.4f}")
        print(f"  • High-resp accuracy  : {mean_high:.4f}")
        print(f"AUC                     : {mean_auc:.4f}")
        print(f"Precision               : {mean_prec:.4f}")
        print(f"Recall                  : {mean_rec:.4f}")
        print(f"F1-score                : {mean_f1:.4f}")



In [ ]:
X, y = load_1(exclude_hiv=True)
print(y.mean() * 100, '%')
t1_models, t1_metrics, t1_configs = run_search(X, y)
show_res(t1_metrics)

In [ ]:
X, y = load_2(exclude_hiv=True)
print(y.mean() * 100, '%')
t2_models, t2_metrics, t2_configs = run_search(X, y)
show_res(t2_metrics)


In [ ]:
X, y = load_3(exclude_hiv=True)
print(y.mean() * 100, '%')
t3_models, t3_metrics, t3_configs = run_search(X, y)
show_res(t3_metrics)


In [ ]:
X, y = load_4(exclude_hiv=True)
print(y.mean() * 100, '%')
t4_models, t4_metrics, t4_configs = run_search(X, y)
show_res(t4_metrics)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X, y = load_5(0.5, exclude_hiv=True)
print(y.mean() * 100, '%')
t5_models, t5_metrics, t5_configs = run_search(X, y)
show_res(t5_metrics)


In [ ]:
X, y = load_6(exclude_hiv=True)
print(y.mean() * 100, '%')
t6_models, t6_metrics, t6_configs = run_search(X, y)
show_res(t6_metrics)

In [ ]:
device = 'cuda'
X, y = load_7(exclude_hiv=True)
print(y.mean() * 100, '%')
t7_models, t7_metrics, t7_configs = run_search(X, y)
show_res(t7_metrics)

In [ ]:
X, y = load_8(exclude_hiv=True)
print(y.mean() * 100, '%')
t8_models, t8_metrics, t8_configs = run_search(X, y)
show_res(t8_metrics)

In [ ]:
X, y = load_xy(0.5)
y_hat = []
h = []

X_tensor = torch.from_numpy(X).float()
y_tensor = torch.tensor(y).float()
for_jay = new.copy()
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for (train_idx, val_idx), sd, config in zip(kf.split(X_tensor, y_tensor), t1_05_fold_best_models, t1_05_fold_best_config):
    model = MLP(121, hidden_sizes=config['hidden_sizes'], dropout_rate=config['dropout_rate']).to(device)
    model.load_state_dict(sd)
    model.eval()
    with torch.no_grad():
        # h.append(model(X_tensor.to(device), True).cpu().numpy())
        # y_hat.append(model(X_tensor.to(device)).cpu().numpy())
        # print(y_hat)
        for_jay.loc[val_idx.tolist(), 'y_hat'] = model(X_tensor[val_idx].to(device)).cpu().numpy()
        for v in val_idx:
            for_jay.loc[v, 'latent'] = str(model(X_tensor[v].to(device), True).cpu().numpy())
        # latents = model(X_tensor[val_idx].to(device), True).cpu().detach().numpy()  # shape: (286, 32)

# ✅ Convert to a list of 1D arrays
        # latent_vectors = list(latents)
        # for_jay.loc[val_idx.tolist(), 'latent'] = latent_vectors
for_jay[ ['Sample_ID_x', 'Sample_ID_y', 'PID', 'Disease_Cohort_x', 'Disease_Cohort_y', 'V0_Group_x', 'V0_Group_y',
       'B0_Group_x', 'B0_Group_y', 'SARS-CoV-2 wRBD_x', 'SARS-CoV-2 wRBD_y'] + x_cols + ['y_hat', 'latent']].to_csv('t1 with 0.5.csv')

In [ ]:
# for_jay['latent'].apply(lambda s: np.fromstring(s.strip("[]"), sep=' ')).values
for_jay['latent'].values

In [ ]:
h = []
X, y = load_xy(0.5)
X_tensor = torch.from_numpy(X).float()
for sd, config in zip(t1_05_fold_best_models, t1_05_fold_best_config):
    model = MLP(121, hidden_sizes=config['hidden_sizes'], dropout_rate=config['dropout_rate']).to(device)
    model.load_state_dict(sd)
    model.eval()
    with torch.no_grad():
        h.append(model(X_tensor.to(device), True))



In [ ]:
h = nn.ReLU()(torch.hstack([hh.squeeze(1) for hh in h])).cpu().numpy()


In [ ]:
from sklearn.manifold import TSNE

from sklearn.decomposition import PCA

pca = PCA(n_components=50, random_state=42)
X_reduced = pca.fit_transform(h)  # shape becomes (n_samples, 50)


tsne = TSNE(n_components=2, perplexity=30, learning_rate='auto', init='pca', random_state=42)
X_tsne = tsne.fit_transform(h)  # or X_896 if skipping PCA

import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.scatter(X_tsne[:,0], X_tsne[:,1], c=y)
plt.title("t-SNE on Hidden Representations")
plt.colorbar(label="Class")
plt.savefig('leidos_tsne.png')
plt.show()


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Sample data (replace with your actual data)
# X: (n_samples, n_features), y: binary labels {0, 1}
# X, y = load_4()
def plot_supervised(X, y, task, ax, exclude):
    
    X = X.astype(np.float32)
    y = y.astype(np.float32)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Define the model
    class BottleneckMLP(nn.Module):
        def __init__(self, input_size):
            super(BottleneckMLP, self).__init__()
            dims = [32, 16]
            p = 0.2
            self.encoder = nn.Sequential(
                nn.BatchNorm1d(input_size),
                nn.Linear(input_size, dims[0]),
                nn.BatchNorm1d(dims[0]),
                nn.ReLU(),
                nn.Dropout(p),
                nn.Linear(dims[0], dims[1]),
                nn.BatchNorm1d(dims[1]),
                nn.ReLU(),
                nn.Dropout(p),
                nn.Linear(dims[1], 2),
                nn.Dropout(p),
            )
            self.classifier = nn.Sequential(
                nn.ReLU(),
                nn.Linear(2, 1)
            )

        def forward(self, x, return_features=False):
            z = self.encoder(x)
            if return_features:
                return z
            return torch.sigmoid(self.classifier(z))
    
    # Convert data to tensors
    X_tensor = torch.tensor(X).to(device)
    y_tensor = torch.tensor(y).unsqueeze(1).to(device)
    
    # Initialize and train the model
    torch.manual_seed(0)
    model = BottleneckMLP(input_size=X.shape[1]).to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        output = model(X_tensor)
        loss = criterion(output, y_tensor)
        loss.backward()
        optimizer.step()

    torch.save(model.state_dict(), f'M{task}.pth')
    model.eval()
    with torch.no_grad():
        features_2d = model(X_tensor, return_features=True).cpu().numpy()
    
    
    
    df = pd.DataFrame(features_2d, columns=['Component_1', 'Component_2'])
    for i in range(X.shape[-1]):
        df['dim' + ' ' + str(i + 1)] = X[:, i]
    df['Label'] = y
    # print(df.columns)
    if exclude:
        message = '_No'
    else:
        message = ''
    file_path = f"task_{task}_2d{message}-HIV.csv"
    df.to_csv(file_path, index=False)
    # Plot the 2D bottleneck features
    
    
    # plt.figure(figsize=(4, 4))
    for label in np.unique(y):
        idx = y == label
        if label == 1:
            alpha = 0.3
        else:
            alpha = 1
        ax.scatter(features_2d[idx, 0], features_2d[idx, 1], label=f"Class {int(label)}", alpha=alpha)
    
    ax.set_xlabel("Bottleneck Component 1")
    ax.set_ylabel("Bottleneck Component 2")
    if exclude:
        message = 'out HIV'
    else:
        message = ' HIV'
    ax.set_title(f"2D Bottleneck MLP Task {task} with{message}")
    ax.legend()
    ax.grid(True)
    # ax.tight_layout()
    # plt.show()

# fig, axes = plt.subplots(2, 4, figsize=(20, 10))
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = axes.flatten()
for f, t, ax in zip(
    (load_3, load_4, load_7, load_8), (3, 4, 7, 8), axes):
    exclude = True
    X, y = f(exclude_hiv=exclude)
    plot_supervised(X, y, t, ax, exclude=exclude)
    
plt.savefig('2DMLP.png')

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KernelDensity
from sklearn.svm import OneClassSVM


from sklearn.metrics import accuracy_score, roc_auc_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture

from scipy.spatial.distance import cdist

def plot_cv_supervised_split(X, y, dtypes, task, n_epochs=100, num_folds=10, classifier_type="svm", do_clustering=True):
    X = X.astype(np.float32)
    y = y.astype(np.float32)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    class BottleneckMLP(nn.Module):
        def __init__(self, input_size):
            super(BottleneckMLP, self).__init__()
            dims = [32, 16]
            p = 0.2
            self.encoder = nn.Sequential(
                nn.BatchNorm1d(input_size),
                nn.Linear(input_size, dims[0]),
                nn.BatchNorm1d(dims[0]),
                nn.ReLU(),
                nn.Dropout(p),
                nn.Linear(dims[0], dims[1]),
                nn.BatchNorm1d(dims[1]),
                nn.ReLU(),
                nn.Dropout(p),
                nn.Linear(dims[1], 2),
                nn.Dropout(p),
            )
            self.classifier = nn.Sequential(
                nn.ReLU(),
                nn.Linear(2, 1)
            )

        def forward(self, x, return_features=False):
            z = self.encoder(x)
            if return_features:
                return z
            return torch.sigmoid(self.classifier(z))

    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
    
    fig, axes = plt.subplots(int(num_folds/5), 5, figsize=(25/(10/num_folds), 10))
    axes = axes.flatten()

    accuracies = []
    aucs = []
    per_class_recalls = []
    fold_per_disease_type = []

    ################################################################## Feature Extraction

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = BottleneckMLP(input_size=X.shape[1]).to(device)
        X_train_tensor = torch.tensor(X_train).to(device)
        y_train_tensor = torch.tensor(y_train).unsqueeze(1).to(device)

        torch.manual_seed(0)
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
        criterion = nn.BCELoss()

        model.train()
        for epoch in range(n_epochs):
            optimizer.zero_grad()
            output = model(X_train_tensor)
            loss = criterion(output, y_train_tensor)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            Z_train = model(torch.tensor(X_train).to(device), return_features=True).cpu().numpy()
            Z_test = model(torch.tensor(X_test).to(device), return_features=True).cpu().numpy()

        Z_all = np.vstack([Z_train])
        y_all = np.hstack([y_train])

        ################################################################## Classifier
        if classifier_type == "svm":
            one = (y_all == 1).astype(float).sum()
            zero = (y_all == 0).astype(float).sum()
            w = (one/zero * 3.5)
            # print(w)
            clf = SVC(kernel="rbf", class_weight={0: w, 1: 1}, probability=True)
            clf.fit(Z_all, y_all)
            
            delta = w/50
            # print(delta)
            # delta = 1.
            decision_scores = clf.decision_function(Z_test)
            y_pred = (decision_scores >= delta).astype(np.float32)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]

        elif classifier_type == "knn":
            clf = KNeighborsClassifier(n_neighbors=5)
            clf.fit(Z_all, y_all)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]
            y_pred = (y_pred_proba >= 0.5).astype(np.float32)

        elif classifier_type == "cluster":
            # Manually compute class means (centers)
            center_0 = Z_train[y_train == 0].mean(axis=0)
            center_1 = Z_train[y_train == 1].mean(axis=0)
            centers = np.stack([center_0, center_1])

            # Predict each test point based on closest center
            dists = cdist(Z_test, centers)
            closest = np.argmin(dists, axis=1)
            y_pred = closest.astype(np.float32)  # class 0 or 1

            # Crude probability: inverse normalized distance
            y_pred_proba = 1 - np.min(dists, axis=1) / (np.max(dists) + 1e-8)

        elif classifier_type == "lda":
            clf = LinearDiscriminantAnalysis()
            clf.fit(Z_all, y_all)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]
            y_pred = (y_pred_proba >= 0.5).astype(np.float32)

        elif classifier_type == "gnb":
            clf = GaussianNB()
            clf.fit(Z_all, y_all)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]
            y_pred = (y_pred_proba >= 0.5).astype(np.float32)

        elif classifier_type == "gmm":
            # Fit one GMM per class
            gmm_0 = GaussianMixture(n_components=1).fit(Z_train[y_train == 0])
            gmm_1 = GaussianMixture(n_components=1).fit(Z_train[y_train == 1])
            
            # Compute log-likelihood for each class
            log_p0 = gmm_0.score_samples(Z_test)
            log_p1 = gmm_1.score_samples(Z_test)
            
            y_pred = (log_p1 > log_p0).astype(np.float32)
            # Normalize scores for pseudo-probabilities
            y_pred_proba = 1 / (1 + np.exp(log_p0 - log_p1))

        elif classifier_type == "kde":
            kde0 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(Z_train[y_train == 0])
            kde1 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(Z_train[y_train == 1])
        
            log_p0 = kde0.score_samples(Z_test)
            log_p1 = kde1.score_samples(Z_test)
        
            y_pred = (log_p1 > log_p0).astype(np.float32)
            y_pred_proba = 1 / (1 + np.exp(log_p0 - log_p1))

        elif classifier_type == "kde_anomaly":
            kde = KernelDensity(kernel='gaussian', bandwidth=0.2)
            kde.fit(Z_train[y_train == 1])  # train only on class 1
        
            # Get density scores for class 1 training data to set threshold
            log_density_pos_train = kde.score_samples(Z_train[y_train == 1])
            threshold = np.percentile(log_density_pos_train, 40)  # 60% inclusion
        
            # Score test data
            log_density_test = kde.score_samples(Z_test)
            y_pred = (log_density_test >= threshold).astype(np.float32)
        
            # Normalize for crude probability (optional)
            y_pred_proba = (log_density_test - log_density_test.min()) / (log_density_test.max() - log_density_test.min() + 1e-8)

        elif classifier_type == "svm_anomaly":
            clf = OneClassSVM(kernel='poly', gamma='scale', nu=0.1)
            clf.fit(Z_train[y_train == 1])  # train only on class 1
        
            y_pred_raw = clf.predict(Z_test)  # returns +1 (inlier), -1 (outlier)
            y_pred = (y_pred_raw == 1).astype(np.float32)
        
            # You can optionally assign a fake "confidence" score (not probability)
            y_pred_proba = (clf.decision_function(Z_test) - clf.decision_function(Z_test).min()) / (
            clf.decision_function(Z_test).max() - clf.decision_function(Z_test).min() + 1e-8)

        
        else:
            raise ValueError(f"Unsupported classifier_type: {classifier_type}")

        ################################################################## Metrics
        acc = accuracy_score(y_test, y_pred)
        per_disease_type = {}
        
        for dt in np.unique(dtypes.values[test_idx]):
            idx = dtypes.values[test_idx] == dt
            
            per_disease_type[dt] = accuracy_score(y_test[idx], y_pred[idx])

        fold_per_disease_type.append(per_disease_type)
        try:
            auc = roc_auc_score(y_test, y_pred_proba)
        except:
            auc = np.nan
        recalls = recall_score(y_test, y_pred, average=None, labels=[0, 1])

        accuracies.append(acc)
        aucs.append(auc)
        per_class_recalls.append(recalls)

        ################################################################## Plotting
        ax = axes[fold]

        def plot_points(Zf, labels, name, marker, alpha):
            for label in [0, 1]:
                idx = labels == label
                ax.scatter(Zf[idx, 0], Zf[idx, 1], label=f"{name} Class {int(label)}", marker=marker, alpha=alpha)

        plot_points(Z_train, y_train, "Train", "o", 0.1)
        plot_points(Z_test, y_test, "Test", "*", 1)

        if do_clustering and classifier_type == "cluster":
            ax.scatter(centers[:, 0], centers[:, 1], c='black', marker='X', s=150, label='Class Centers')
        
        x_min, x_max = Z_all[:, 0].min() - 1, Z_all[:, 0].max() + 1
        y_min, y_max = Z_all[:, 1].min() - 1, Z_all[:, 1].max() + 1
        xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                             np.linspace(y_min, y_max, 300))
        grid = np.c_[xx.ravel(), yy.ravel()]

        if classifier_type == "cluster":
            # Plot class centers
            ax.scatter(centers[:, 0], centers[:, 1], c='black', marker='X', s=150, label='Class Centers')
        
            # Decision boundary on meshgrid
            dists = cdist(grid, centers)  # shape: (N_grid_points, 2)
            cluster_pred = np.argmin(dists, axis=1).astype(float)
            Z_grid = cluster_pred.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "svm":
            
            Z_grid = clf.decision_function(grid).reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[delta], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "lda":
            Z_grid = clf.predict_proba(grid)[:, 1].reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "gnb":
            Z_grid = clf.predict_proba(grid)[:, 1].reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "gmm":
            log_p0 = gmm_0.score_samples(grid)
            log_p1 = gmm_1.score_samples(grid)
            Z_grid = 1 / (1 + np.exp(log_p0 - log_p1))  # pseudo-prob
            Z_grid = Z_grid.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')


        if classifier_type == "kde":
            log_p0_grid = kde0.score_samples(grid)
            log_p1_grid = kde1.score_samples(grid)
            Z_grid = 1 / (1 + np.exp(log_p0_grid - log_p1_grid))
            Z_grid = Z_grid.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "kde_anomaly":
            log_density_grid = kde.score_samples(grid)
            Z_grid = (log_density_grid >= threshold).astype(float)
            Z_grid = Z_grid.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "svm_anomaly":
            Z_grid = clf.decision_function(grid).reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0], colors='black', linewidths=1.5, linestyles='--')

        ax.set_title(f"Fold {fold + 1}")
        ax.set_xlabel("Comp 1")
        ax.set_ylabel("Comp 2")
        ax.grid(True)

    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right')
    fig.suptitle(f"Task {task}: 10-Fold Bottleneck + {classifier_type.upper()}", fontsize=18)
    plt.tight_layout(rect=[0, 0, 0.98, 0.95])
    # plt.show()

    mean_accuracy = np.mean(accuracies)
    mean_auc = np.nanmean(aucs)
    mean_per_class_recall = np.mean(np.vstack(per_class_recalls), axis=0)

    from collections import defaultdict
    values_sum = defaultdict(float)
    counts = defaultdict(int)
    
    for f in fold_per_disease_type:
        for key, value in f.items():
            values_sum[key] += value
            counts[key] += 1
    
    # Compute average
    averages = {key: values_sum[key] / counts[key] for key in values_sum}
    
    # Show result
    
    print(f"Task {task} - 10-Fold Results ({classifier_type.upper()}):")
    print(f'  Average Accuracy per disease type:')
    for key, avg in averages.items():
        print(f"    {key}: {avg:.4f}")
    print(f"  Average Accuracy: {mean_accuracy:.4f}")
    print(f"  Average AUC:      {mean_auc:.4f}")
    print(f"  Recall Class 0:   {mean_per_class_recall[0]:.4f}")
    print(f"  Recall Class 1:   {mean_per_class_recall[1]:.4f}")
    return mean_auc, mean_per_class_recall[0], mean_per_class_recall[1]

# for i, f in enumerate((load_1, load_2, load_3, load_4, load_5, load_6, load_7, load_8)):
for i, f in zip([1, 2, 5, 6], (load_1, load_2, load_5, load_6)):
    X, y, dtypes = f(exclude_hiv=True, return_types=True)
    plot_cv_supervised_split(
        X, 
        y, 
        dtypes,
        n_epochs=100, 
        num_folds=10, task=i, 
        classifier_type="svm")



In [ ]:
class BottleneckMLP(nn.Module):
        def __init__(self, input_size):
            super(BottleneckMLP, self).__init__()
            dims = [32, 16]
            p = 0.2
            self.encoder = nn.Sequential(
                nn.BatchNorm1d(input_size),
                nn.Linear(input_size, dims[0]),
                nn.BatchNorm1d(dims[0]),
                nn.ReLU(),
                nn.Dropout(p),
                nn.Linear(dims[0], dims[1]),
                nn.BatchNorm1d(dims[1]),
                nn.ReLU(),
                nn.Dropout(p),
                nn.Linear(dims[1], 2),
                nn.Dropout(p),
            )
            self.classifier = nn.Sequential(
                nn.ReLU(),
                nn.Linear(2, 1)
            )

        def forward(self, x, return_features=False):
            z = self.encoder(x)
            if return_features:
                return z
            return torch.sigmoid(self.classifier(z))

fig, axes = plt.subplots(7, 4, figsize=(14, 16))  # More vertical space helps
axes = axes.T.reshape(-1)  # Make sure it's row-major: 4 tasks × 7 rows

task_funcs = [(3, load_3), (4, load_4), (7, load_7), (8, load_8)]

ax_idx = 0
for task, f in task_funcs:
    ret = f(exclude_hiv=exclude, other_diseases=True)
    
    disease_order = ['Healthy Control'] + [d for d in ret if d != 'Healthy Control']
    for disease in disease_order:
        ax = axes[ax_idx]
        ax_idx += 1
        X, y = ret[disease]
        torch.manual_seed(0)
        model = BottleneckMLP(input_size=X.shape[1]).to(device)
        model.load_state_dict(torch.load(f'M{task}.pth', weights_only=True))
        model.eval()
        with torch.no_grad():
            z = model(torch.from_numpy(X).to(device).float(), return_features=True).cpu().numpy()
            for label in np.unique(y):
                idx = y == label
                if label == 1:
                    alpha = 0.2
                else:
                    alpha = 1
                ax.scatter(z[idx, 0], z[idx, 1], label=f"Class {int(label)}", alpha=alpha)
                # Create a meshgrid over the bottleneck feature space
                x_min, x_max = z[:, 0].min() - 1, z[:, 0].max() + 1
                y_min, y_max = z[:, 1].min() - 1, z[:, 1].max() + 1
                xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                                     np.linspace(y_min, y_max, 200))
                
                # Stack and convert to tensor
                grid = np.c_[xx.ravel(), yy.ravel()]
                grid_tensor = torch.tensor(grid, dtype=torch.float32).to(device)
                
                # Get bottleneck → output mapping
                with torch.no_grad():
                    logits = torch.sigmoid(model.classifier(grid_tensor))  # You'll need to define this
                    probs = logits.cpu().numpy().reshape(xx.shape)
                
                # Plot decision boundary at 0.5 probability
                ax.contour(xx, yy, probs, levels=[0.5], linewidths=1, linestyles='dashed')

        ax.set_title(f"Task {task} - {disease}")
        ax.legend()
        ax.grid(True)

plt.tight_layout()
plt.savefig('other diseases.png')

        

In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import accuracy_score, roc_auc_score
import pandas as pd
from torch_geometric.data import Batch

# === Define model ===
class BottleneckMLP(nn.Module):
    def __init__(self, input_size):
        super(BottleneckMLP, self).__init__()
        dims = [32, 16]
        p = 0.2
        self.encoder = nn.Sequential(
            nn.BatchNorm1d(input_size),
            nn.Linear(input_size, dims[0]),
            nn.BatchNorm1d(dims[0]),
            nn.ReLU(),
            nn.Dropout(p),
            nn.Linear(dims[0], dims[1]),
            nn.BatchNorm1d(dims[1]),
            nn.ReLU(),
            nn.Dropout(p),
            nn.Linear(dims[1], 2),
            nn.Dropout(p),
        )
        self.classifier = nn.Sequential(
            nn.ReLU(),
            nn.Linear(2, 1)
        )

    def forward(self, x, return_features=False):
        z = self.encoder(x)
        if return_features:
            return z
        return torch.sigmoid(self.classifier(z))

# === Setup ===
task_funcs = [(3, load_3), (4, load_4), (7, load_7), (8, load_8)]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
exclude = True

# Create folders
os.makedirs("plots/folds", exist_ok=True)
os.makedirs("plots/grids", exist_ok=True)

results = []

for task, load_func in task_funcs:
    ret = load_func(exclude_hiv=exclude, other_diseases=True)
    disease_order = ['Healthy Control'] + [d for d in ret if d != 'Healthy Control']

    for disease in disease_order:
        X, y = ret[disease]
        y = np.array(y)

        if len(y) < 10 or len(np.unique(y)) < 2:
            print(f"Skipping Task {task}, Disease {disease}: insufficient data or class diversity")
            continue

        torch.manual_seed(0)
        model = BottleneckMLP(input_size=X.shape[1]).to(device)
        model.load_state_dict(torch.load(f'M{task}.pth', map_location=device, weights_only=True), strict=False)
        model.eval()

        with torch.no_grad():
            X_tensor = torch.from_numpy(X).float().to(device)
            z = model(X_tensor, return_features=True).cpu().numpy()

        classifiers = {
            'SVM': SVC(kernel='linear', probability=True),
            'Centroid': NearestCentroid()
        }

        for clf_name, clf in classifiers.items():
            skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
            fig, axs = plt.subplots(2, 5, figsize=(20, 8))
            axs = axs.flatten()

            fold_accs, fold_aucs, fold_pos_accs, fold_neg_accs = [], [], [], []
            unique_classes = np.unique(y)
            per_class_accs = {int(cls): [] for cls in unique_classes}

            for fold_idx, (train_idx, test_idx) in enumerate(skf.split(z, y)):
                clf.fit(z[train_idx], y[train_idx])
                y_pred = clf.predict(z[test_idx])
                acc = accuracy_score(y[test_idx], y_pred)
                fold_accs.append(acc)

                try:
                    y_score = clf.predict_proba(z[test_idx])[:, 1]
                    auc = roc_auc_score(y[test_idx], y_score)
                except:
                    auc = np.nan
                fold_aucs.append(auc)

                mask_pos = y[test_idx] == 1
                mask_neg = y[test_idx] == 0
                fold_pos_accs.append((y_pred[mask_pos] == 1).mean() if mask_pos.any() else np.nan)
                fold_neg_accs.append((y_pred[mask_neg] == 0).mean() if mask_neg.any() else np.nan)

                # === Per-Class Accuracy ===
                for cls in unique_classes:
                    cls_mask = y[test_idx] == cls
                    if cls_mask.any():
                        acc_cls = (y_pred[cls_mask] == y[test_idx][cls_mask]).mean()
                        per_class_accs[int(cls)].append(acc_cls)
                    else:
                        per_class_accs[int(cls)].append(np.nan)

                # === Plotting ===
                x_min, x_max = z[:, 0].min() - 1, z[:, 0].max() + 1
                y_min, y_max = z[:, 1].min() - 1, z[:, 1].max() + 1
                xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                                     np.linspace(y_min, y_max, 200))
                grid = np.c_[xx.ravel(), yy.ravel()]

                ax = axs[fold_idx]
                for label in np.unique(y):
                    tr = train_idx[y[train_idx] == label]
                    te = test_idx[y[test_idx] == label]
                    ax.scatter(z[tr, 0], z[tr, 1], label=f"Train {int(label)}", alpha=0.2, edgecolor='k', linewidth=0.3)
                    ax.scatter(z[te, 0], z[te, 1], label=f"Test {int(label)}", alpha=0.8)

                try:
                    zz = clf.predict(grid).reshape(xx.shape)
                    ax.contour(xx, yy, zz, levels=[0.5], colors='black', linestyles='dashed')
                except:
                    pass

                ax.set_title(f"{clf_name} Fold {fold_idx+1} (acc={acc:.2f})")
                ax.set_xticks([])
                ax.set_yticks([])
                ax.grid(True)
                ax.legend(fontsize='small', loc='upper right')

                # === Per-Fold Plot ===
                fig_fold, ax_fold = plt.subplots(figsize=(6, 5))
                for label in np.unique(y):
                    tr = train_idx[y[train_idx] == label]
                    te = test_idx[y[test_idx] == label]
                    ax_fold.scatter(z[tr, 0], z[tr, 1], label=f"Train {int(label)}", alpha=0.2, edgecolor='k', linewidth=0.3)
                    ax_fold.scatter(z[te, 0], z[te, 1], label=f"Test {int(label)}", alpha=0.8)
                try:
                    zz = clf.predict(grid).reshape(xx.shape)
                    ax_fold.contour(xx, yy, zz, levels=[0.5], colors='black', linestyles='dashed')
                except:
                    pass

                ax_fold.set_title(f"{clf_name} Task {task} {disease} Fold {fold_idx+1}\nAcc={acc:.2f}, AUC={auc:.2f}")
                ax_fold.legend()
                ax_fold.grid(True)
                fig_fold.tight_layout()
                fold_path = f"plots/folds/{clf_name}_Task{task}_{disease.replace(' ', '_')}_Fold{fold_idx+1}.png"
                fig_fold.savefig(fold_path)
                plt.close(fig_fold)

            # === Combined Plot Save ===
            fig.suptitle(f"{clf_name} - Task {task} - {disease}", fontsize=16)
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            grid_path = f"plots/grids/{clf_name}_folds_Task{task}_{disease.replace(' ', '_')}.png"
            plt.savefig(grid_path)
            plt.close(fig)

            # === Save metrics ===
            mean_per_class_acc = {f"class_{cls}_acc": np.nanmean(per_class_accs[cls]) for cls in per_class_accs}
            results.append({
                'task': task,
                'disease': disease,
                'classifier': clf_name,
                'mean_accuracy': np.mean(fold_accs),
                'mean_auc': np.nanmean(fold_aucs),
                'mean_pos_acc': np.nanmean(fold_pos_accs),
                'mean_neg_acc': np.nanmean(fold_neg_accs),
                **mean_per_class_acc
            })

# === Save results to CSV ===
results_df = pd.DataFrame(results)
results_df.to_csv("classification_results.csv", index=False)
print("✅ Saved metrics to classification_results.csv")


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KernelDensity
from sklearn.svm import OneClassSVM


from sklearn.metrics import accuracy_score, roc_auc_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture

from scipy.spatial.distance import cdist

def plot_cv_transfered_split(X, y, task, disease, n_epochs=100, num_folds=10, classifier_type="svm", do_clustering=True):
    X = X.astype(np.float32)
    y = y.astype(np.float32)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    class BottleneckMLP(nn.Module):
        def __init__(self, input_size):
            super(BottleneckMLP, self).__init__()
            dims = [32, 16]
            p = 0.2
            self.encoder = nn.Sequential(
                nn.BatchNorm1d(input_size),
                nn.Linear(input_size, dims[0]),
                nn.BatchNorm1d(dims[0]),
                nn.ReLU(),
                nn.Dropout(p),
                nn.Linear(dims[0], dims[1]),
                nn.BatchNorm1d(dims[1]),
                nn.ReLU(),
                nn.Dropout(p),
                nn.Linear(dims[1], 2),
                nn.Dropout(p),
            )
            self.classifier = nn.Sequential(
                nn.ReLU(),
                nn.Linear(2, 1)
            )

        def forward(self, x, return_features=False):
            z = self.encoder(x)
            if return_features:
                return z
            return torch.sigmoid(self.classifier(z))

    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
    
    fig, axes = plt.subplots(int(num_folds/5), 5, figsize=(25/(10/num_folds), 10))
    axes = axes.flatten()

    accuracies = []
    aucs = []
    per_class_recalls = []

    ################################################################## Feature Extraction

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = BottleneckMLP(input_size=X.shape[1]).to(device)
        X_train_tensor = torch.tensor(X_train).to(device)
        y_train_tensor = torch.tensor(y_train).unsqueeze(1).to(device)

        model.load_state_dict(torch.load(f'M{task}.pth', weights_only=True))

        model.eval()
        with torch.no_grad():
            Z_train = model(torch.tensor(X_train).to(device), return_features=True).cpu().numpy()
            Z_test = model(torch.tensor(X_test).to(device), return_features=True).cpu().numpy()

        Z_all = np.vstack([Z_train])
        y_all = np.hstack([y_train])

        ################################################################## Classifier
        if classifier_type == "svm":
            one = (y_all == 1).astype(float).sum()
            zero = (y_all == 0).astype(float).sum()
            w = (one/zero * 3.5)
            # print(w)
            clf = SVC(kernel="rbf", class_weight={0: w, 1: 1}, probability=True)
            clf.fit(Z_all, y_all)
            
            delta = w/50
            # print(delta)
            # delta = 1.
            decision_scores = clf.decision_function(Z_test)
            y_pred = (decision_scores >= delta).astype(np.float32)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]

        elif classifier_type == "knn":
            clf = KNeighborsClassifier(n_neighbors=5)
            clf.fit(Z_all, y_all)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]
            y_pred = (y_pred_proba >= 0.5).astype(np.float32)

        elif classifier_type == "cluster":
            # Manually compute class means (centers)
            center_0 = Z_train[y_train == 0].mean(axis=0)
            center_1 = Z_train[y_train == 1].mean(axis=0)
            centers = np.stack([center_0, center_1])

            # Predict each test point based on closest center
            dists = cdist(Z_test, centers)
            closest = np.argmin(dists, axis=1)
            y_pred = closest.astype(np.float32)  # class 0 or 1

            # Crude probability: inverse normalized distance
            y_pred_proba = 1 - np.min(dists, axis=1) / (np.max(dists) + 1e-8)

        elif classifier_type == "lda":
            clf = LinearDiscriminantAnalysis()
            clf.fit(Z_all, y_all)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]
            y_pred = (y_pred_proba >= 0.5).astype(np.float32)

        elif classifier_type == "gnb":
            clf = GaussianNB()
            clf.fit(Z_all, y_all)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]
            y_pred = (y_pred_proba >= 0.5).astype(np.float32)

        elif classifier_type == "gmm":
            # Fit one GMM per class
            gmm_0 = GaussianMixture(n_components=1).fit(Z_train[y_train == 0])
            gmm_1 = GaussianMixture(n_components=1).fit(Z_train[y_train == 1])
            
            # Compute log-likelihood for each class
            log_p0 = gmm_0.score_samples(Z_test)
            log_p1 = gmm_1.score_samples(Z_test)
            
            y_pred = (log_p1 > log_p0).astype(np.float32)
            # Normalize scores for pseudo-probabilities
            y_pred_proba = 1 / (1 + np.exp(log_p0 - log_p1))

        elif classifier_type == "kde":
            kde0 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(Z_train[y_train == 0])
            kde1 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(Z_train[y_train == 1])
        
            log_p0 = kde0.score_samples(Z_test)
            log_p1 = kde1.score_samples(Z_test)
        
            y_pred = (log_p1 > log_p0).astype(np.float32)
            y_pred_proba = 1 / (1 + np.exp(log_p0 - log_p1))

        elif classifier_type == "kde_anomaly":
            kde = KernelDensity(kernel='gaussian', bandwidth=0.2)
            kde.fit(Z_train[y_train == 1])  # train only on class 1
        
            # Get density scores for class 1 training data to set threshold
            log_density_pos_train = kde.score_samples(Z_train[y_train == 1])
            threshold = np.percentile(log_density_pos_train, 10)  # 90% inclusion
        
            # Score test data
            log_density_test = kde.score_samples(Z_test)
            y_pred = (log_density_test >= threshold).astype(np.float32)
        
            # Normalize for crude probability (optional)
            y_pred_proba = (log_density_test - log_density_test.min()) / (log_density_test.max() - log_density_test.min() + 1e-8)

        elif classifier_type == "svm_anomaly":
            clf = OneClassSVM(kernel='poly', gamma='scale', nu=0.1)
            clf.fit(Z_train[y_train == 1])  # train only on class 1
        
            y_pred_raw = clf.predict(Z_test)  # returns +1 (inlier), -1 (outlier)
            y_pred = (y_pred_raw == 1).astype(np.float32)
        
            # You can optionally assign a fake "confidence" score (not probability)
            y_pred_proba = (clf.decision_function(Z_test) - clf.decision_function(Z_test).min()) / (
            clf.decision_function(Z_test).max() - clf.decision_function(Z_test).min() + 1e-8)

        
        else:
            raise ValueError(f"Unsupported classifier_type: {classifier_type}")

        ################################################################## Metrics
        acc = accuracy_score(y_test, y_pred)
        try:
            auc = roc_auc_score(y_test, y_pred_proba)
        except:
            auc = np.nan
        recalls = recall_score(y_test, y_pred, average=None, labels=[0, 1])

        accuracies.append(acc)
        aucs.append(auc)
        per_class_recalls.append(recalls)

        ################################################################## Plotting
        ax = axes[fold]

        def plot_points(Zf, labels, name, marker, alpha):
            for label in [0, 1]:
                idx = labels == label
                ax.scatter(Zf[idx, 0], Zf[idx, 1], label=f"{name} Class {int(label)}", marker=marker, alpha=alpha)

        plot_points(Z_train, y_train, "Train", "o", 0.1)
        plot_points(Z_test, y_test, "Test", "*", 1)

        if do_clustering and classifier_type == "cluster":
            ax.scatter(centers[:, 0], centers[:, 1], c='black', marker='X', s=150, label='Class Centers')
        
        x_min, x_max = Z_all[:, 0].min() - 1, Z_all[:, 0].max() + 1
        y_min, y_max = Z_all[:, 1].min() - 1, Z_all[:, 1].max() + 1
        xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                             np.linspace(y_min, y_max, 300))
        grid = np.c_[xx.ravel(), yy.ravel()]

        if classifier_type == "cluster":
            # Plot class centers
            ax.scatter(centers[:, 0], centers[:, 1], c='black', marker='X', s=150, label='Class Centers')
        
            # Decision boundary on meshgrid
            dists = cdist(grid, centers)  # shape: (N_grid_points, 2)
            cluster_pred = np.argmin(dists, axis=1).astype(float)
            Z_grid = cluster_pred.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "svm":
            
            Z_grid = clf.decision_function(grid).reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[delta], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "lda":
            Z_grid = clf.predict_proba(grid)[:, 1].reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "gnb":
            Z_grid = clf.predict_proba(grid)[:, 1].reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "gmm":
            log_p0 = gmm_0.score_samples(grid)
            log_p1 = gmm_1.score_samples(grid)
            Z_grid = 1 / (1 + np.exp(log_p0 - log_p1))  # pseudo-prob
            Z_grid = Z_grid.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')


        if classifier_type == "kde":
            log_p0_grid = kde0.score_samples(grid)
            log_p1_grid = kde1.score_samples(grid)
            Z_grid = 1 / (1 + np.exp(log_p0_grid - log_p1_grid))
            Z_grid = Z_grid.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "kde_anomaly":
            log_density_grid = kde.score_samples(grid)
            Z_grid = (log_density_grid >= threshold).astype(float)
            Z_grid = Z_grid.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "svm_anomaly":
            Z_grid = clf.decision_function(grid).reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0], colors='black', linewidths=1.5, linestyles='--')

        ax.set_title(f"Fold {fold + 1}")
        ax.set_xlabel("Comp 1")
        ax.set_ylabel("Comp 2")
        ax.grid(True)

    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right')
    fig.suptitle(f"Task {task}-{disease}: 10-Fold Bottleneck + {classifier_type.upper()}", fontsize=18)
    plt.tight_layout(rect=[0, 0, 0.98, 0.95])
    # plt.show()

    mean_accuracy = np.mean(accuracies)
    mean_auc = np.nanmean(aucs)
    mean_per_class_recall = np.mean(np.vstack(per_class_recalls), axis=0)

    print(f"Task {task}-{disease} - 10-Fold Results ({classifier_type.upper()}):")
    print(f"  Average Accuracy: {mean_accuracy:.4f}")
    print(f"  Average AUC:      {mean_auc:.4f}")
    print(f"  Recall Class 0:   {mean_per_class_recall[0]:.4f}")
    print(f"  Recall Class 1:   {mean_per_class_recall[1]:.4f}")
    return mean_auc, mean_per_class_recall[0], mean_per_class_recall[1]


task_funcs = [(3, load_3), (4, load_4), (7, load_7), (8, load_8)]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
exclude = True

for task, load_func in task_funcs:
    ret = load_func(exclude_hiv=exclude, other_diseases=True)
    disease_order = ['Healthy Control'] + [d for d in ret if d != 'Healthy Control']

    for disease in disease_order:
        X, y = ret[disease]
        if len(y) < 10 or len(np.unique(y)) < 2:
            print(f"Skipping Task {task}, Disease {disease}: insufficient data or class diversity")
            continue
        plot_cv_transfered_split(
        X, 
        y, 
        disease=disease,
        n_epochs=150, 
        num_folds=10, task=task, 
        classifier_type="svm")

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KernelDensity
from sklearn.svm import OneClassSVM


from sklearn.metrics import accuracy_score, roc_auc_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture

from scipy.spatial.distance import cdist

def plot_cv_transfered_split(X, y, task, disease, n_epochs=100, num_folds=10, classifier_type="svm", do_clustering=True):
    X = X.astype(np.float32)
    y = y.astype(np.float32)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    class BottleneckMLP(nn.Module):
        def __init__(self, input_size):
            super(BottleneckMLP, self).__init__()
            dims = [32, 16]
            p = 0.2
            self.encoder = nn.Sequential(
                nn.BatchNorm1d(input_size),
                nn.Linear(input_size, dims[0]),
                nn.BatchNorm1d(dims[0]),
                nn.ReLU(),
                nn.Dropout(p),
                nn.Linear(dims[0], dims[1]),
                nn.BatchNorm1d(dims[1]),
                nn.ReLU(),
                nn.Dropout(p),
                nn.Linear(dims[1], 2),
                nn.Dropout(p),
            )
            self.classifier = nn.Sequential(
                nn.ReLU(),
                nn.Linear(2, 1)
            )

        def forward(self, x, return_features=False):
            z = self.encoder(x)
            if return_features:
                return z
            return torch.sigmoid(self.classifier(z))

    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
    
    fig, axes = plt.subplots(int(num_folds/5), 5, figsize=(25/(10/num_folds), 10))
    axes = axes.flatten()

    accuracies = []
    aucs = []
    per_class_recalls = []

    ################################################################## Feature Extraction

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = BottleneckMLP(input_size=X.shape[1]).to(device)
        X_train_tensor = torch.tensor(X_train).to(device)
        y_train_tensor = torch.tensor(y_train).unsqueeze(1).to(device)

        torch.manual_seed(0)
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
        criterion = nn.BCELoss()

        model.train()
        for epoch in range(n_epochs):
            optimizer.zero_grad()
            output = model(X_train_tensor)
            loss = criterion(output, y_train_tensor)
            loss.backward()
            optimizer.step()


        model.eval()
        with torch.no_grad():
            Z_train = model(torch.tensor(X_train).to(device), return_features=True).cpu().numpy()
            Z_test = model(torch.tensor(X_test).to(device), return_features=True).cpu().numpy()

        Z_all = np.vstack([Z_train])
        y_all = np.hstack([y_train])

        ################################################################## Classifier
        if classifier_type == "svm":
            one = (y_all == 1).astype(float).sum()
            zero = (y_all == 0).astype(float).sum()
            # w = (one/zero * 3.5)
            w = one/zero
            # w = zero/one
            # print(w)
            clf = SVC(kernel="rbf", class_weight={0: w, 1: 1}, probability=True)
            clf.fit(Z_all, y_all)
            
            # delta = w/50 + 0.1
            # delta = w
            delta = 0.75
            # delta = 1.5
            # print(delta)
            # delta = 1.
            decision_scores = clf.decision_function(Z_test)
            y_pred = (decision_scores >= delta).astype(np.float32)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]

        elif classifier_type == "knn":
            clf = KNeighborsClassifier(n_neighbors=5)
            clf.fit(Z_all, y_all)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]
            y_pred = (y_pred_proba >= 0.5).astype(np.float32)

        elif classifier_type == "cluster":
            # Manually compute class means (centers)
            center_0 = Z_train[y_train == 0].mean(axis=0)
            center_1 = Z_train[y_train == 1].mean(axis=0)
            centers = np.stack([center_0, center_1])

            # Predict each test point based on closest center
            dists = cdist(Z_test, centers)
            closest = np.argmin(dists, axis=1)
            y_pred = closest.astype(np.float32)  # class 0 or 1

            # Crude probability: inverse normalized distance
            y_pred_proba = 1 - np.min(dists, axis=1) / (np.max(dists) + 1e-8)

        elif classifier_type == "lda":
            clf = LinearDiscriminantAnalysis()
            clf.fit(Z_all, y_all)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]
            y_pred = (y_pred_proba >= 0.5).astype(np.float32)

        elif classifier_type == "gnb":
            clf = GaussianNB()
            clf.fit(Z_all, y_all)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]
            y_pred = (y_pred_proba >= 0.5).astype(np.float32)

        elif classifier_type == "gmm":
            # Fit one GMM per class
            gmm_0 = GaussianMixture(n_components=1).fit(Z_train[y_train == 0])
            gmm_1 = GaussianMixture(n_components=1).fit(Z_train[y_train == 1])
            
            # Compute log-likelihood for each class
            log_p0 = gmm_0.score_samples(Z_test)
            log_p1 = gmm_1.score_samples(Z_test)
            
            y_pred = (log_p1 > log_p0).astype(np.float32)
            # Normalize scores for pseudo-probabilities
            y_pred_proba = 1 / (1 + np.exp(log_p0 - log_p1))

        elif classifier_type == "kde":
            kde0 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(Z_train[y_train == 0])
            kde1 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(Z_train[y_train == 1])
        
            log_p0 = kde0.score_samples(Z_test)
            log_p1 = kde1.score_samples(Z_test)
        
            y_pred = (log_p1 > log_p0).astype(np.float32)
            y_pred_proba = 1 / (1 + np.exp(log_p0 - log_p1))

        elif classifier_type == "kde_anomaly":
            kde = KernelDensity(kernel='gaussian', bandwidth=0.2)
            kde.fit(Z_train[y_train == 1])  # train only on class 1
        
            # Get density scores for class 1 training data to set threshold
            log_density_pos_train = kde.score_samples(Z_train[y_train == 1])
            threshold = np.percentile(log_density_pos_train, 10)  # 90% inclusion
        
            # Score test data
            log_density_test = kde.score_samples(Z_test)
            y_pred = (log_density_test >= threshold).astype(np.float32)
        
            # Normalize for crude probability (optional)
            y_pred_proba = (log_density_test - log_density_test.min()) / (log_density_test.max() - log_density_test.min() + 1e-8)

        elif classifier_type == "svm_anomaly":
            clf = OneClassSVM(kernel='poly', gamma='scale', nu=0.1)
            clf.fit(Z_train[y_train == 1])  # train only on class 1
        
            y_pred_raw = clf.predict(Z_test)  # returns +1 (inlier), -1 (outlier)
            y_pred = (y_pred_raw == 1).astype(np.float32)
        
            # You can optionally assign a fake "confidence" score (not probability)
            y_pred_proba = (clf.decision_function(Z_test) - clf.decision_function(Z_test).min()) / (
            clf.decision_function(Z_test).max() - clf.decision_function(Z_test).min() + 1e-8)

        
        else:
            raise ValueError(f"Unsupported classifier_type: {classifier_type}")

        ################################################################## Metrics
        acc = accuracy_score(y_test, y_pred)
        try:
            auc = roc_auc_score(y_test, y_pred_proba)
        except:
            auc = np.nan
        recalls = recall_score(y_test, y_pred, average=None, labels=[0, 1])

        accuracies.append(acc)
        aucs.append(auc)
        per_class_recalls.append(recalls)

        ################################################################## Plotting
        ax = axes[fold]

        def plot_points(Zf, labels, name, marker, alpha):
            for label in [0, 1]:
                idx = labels == label
                ax.scatter(Zf[idx, 0], Zf[idx, 1], label=f"{name} Class {int(label)}", marker=marker, alpha=alpha)

        plot_points(Z_train, y_train, "Train", "o", 0.1)
        plot_points(Z_test, y_test, "Test", "*", 1)

        if do_clustering and classifier_type == "cluster":
            ax.scatter(centers[:, 0], centers[:, 1], c='black', marker='X', s=150, label='Class Centers')
        
        x_min, x_max = Z_all[:, 0].min() - 1, Z_all[:, 0].max() + 1
        y_min, y_max = Z_all[:, 1].min() - 1, Z_all[:, 1].max() + 1
        xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                             np.linspace(y_min, y_max, 300))
        grid = np.c_[xx.ravel(), yy.ravel()]

        if classifier_type == "cluster":
            # Plot class centers
            ax.scatter(centers[:, 0], centers[:, 1], c='black', marker='X', s=150, label='Class Centers')
        
            # Decision boundary on meshgrid
            dists = cdist(grid, centers)  # shape: (N_grid_points, 2)
            cluster_pred = np.argmin(dists, axis=1).astype(float)
            Z_grid = cluster_pred.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "svm":
            
            Z_grid = clf.decision_function(grid).reshape(xx.shape)
            contour = ax.contour(xx, yy, Z_grid, levels=sorted([0.6, delta, 0.9]), linewidths=1.5, linestyles='--')
            ax.clabel(contour, inline=True, fontsize=10)

        if classifier_type == "lda":
            Z_grid = clf.predict_proba(grid)[:, 1].reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "gnb":
            Z_grid = clf.predict_proba(grid)[:, 1].reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "gmm":
            log_p0 = gmm_0.score_samples(grid)
            log_p1 = gmm_1.score_samples(grid)
            Z_grid = 1 / (1 + np.exp(log_p0 - log_p1))  # pseudo-prob
            Z_grid = Z_grid.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')


        if classifier_type == "kde":
            log_p0_grid = kde0.score_samples(grid)
            log_p1_grid = kde1.score_samples(grid)
            Z_grid = 1 / (1 + np.exp(log_p0_grid - log_p1_grid))
            Z_grid = Z_grid.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "kde_anomaly":
            log_density_grid = kde.score_samples(grid)
            Z_grid = (log_density_grid >= threshold).astype(float)
            Z_grid = Z_grid.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "svm_anomaly":
            Z_grid = clf.decision_function(grid).reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0], colors='black', linewidths=1.5, linestyles='--')

        ax.set_title(f"Fold {fold + 1}")
        ax.set_xlabel("Comp 1")
        ax.set_ylabel("Comp 2")
        ax.grid(True)

    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right')
    fig.suptitle(f"Task {task}-{disease}: 10-Fold Bottleneck + {classifier_type.upper()}", fontsize=18)
    plt.tight_layout(rect=[0, 0, 0.98, 0.95])
    
    plt.savefig(f'plots/3478_others/{task}-{disease}.png')
    # plt.show()

    mean_accuracy = np.mean(accuracies)
    mean_auc = np.nanmean(aucs)
    mean_per_class_recall = np.mean(np.vstack(per_class_recalls), axis=0)

    print(f"Task {task}-{disease} - 10-Fold Results ({classifier_type.upper()}):")
    print(f"  Average Accuracy: {mean_accuracy:.4f}")
    print(f"  Average AUC:      {mean_auc:.4f}")
    print(f"  Recall Class 0:   {mean_per_class_recall[0]:.4f}")
    print(f"  Recall Class 1:   {mean_per_class_recall[1]:.4f}")
    return mean_auc, mean_per_class_recall[0], mean_per_class_recall[1]


task_funcs = [(3, load_3), (4, load_4), (7, load_7), (8, load_8)]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
exclude = True

for task, load_func in task_funcs:
    ret = load_func(exclude_hiv=exclude, other_diseases=True)
    disease_order = ['Healthy Control'] + [d for d in ret if d != 'Healthy Control']
    
    for disease in disease_order:
        X, y = ret[disease]
        if len(y) < 10 or len(np.unique(y)) < 2 or (y==0).astype(int).sum() < 10:
            print(f"Skipping Task {task}, Disease {disease}: insufficient data or class diversity")
            continue
        plot_cv_transfered_split(
        X, 
        y, 
        disease=disease,
        n_epochs=100, 
        num_folds=10, task=task, 
        classifier_type="cluster")

In [ ]:
# Rename and restructure the DataFrame to match the screenshot's format
mapped_df = pd.read_csv('others-3478.csv')

# Rename columns
mapped_df.rename(columns={
    "Task": "task",
    "Disease": "disease",
    "Accuracy": "mean_accuracy",
    "AUC": "mean_auc",
    "Recall_0": "class_0_acc",
    "Recall_1": "class_1_acc"
}, inplace=True)

# Add 'classifier' column
mapped_df["classifier"] = "SVM"

# Add placeholders for mean_pos_acc and mean_neg_acc
# (Assuming mean_pos_acc = class_1_acc and mean_neg_acc = class_0_acc if no other info is available)
mapped_df["mean_pos_acc"] = mapped_df["class_1_acc"]
mapped_df["mean_neg_acc"] = mapped_df["class_0_acc"]

# Reorder columns
mapped_df = mapped_df[[
    "task", "disease", "classifier",
    "mean_accuracy", "mean_auc", "mean_pos_acc", "mean_neg_acc",
    "class_0_acc", "class_1_acc"
]]

# Save to new CSV
final_csv_path = "svm_results_structured.csv"
mapped_df.to_csv(final_csv_path, index=False)

tools.display_dataframe_to_user(name="Structured SVM Results", dataframe=mapped_df)


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KernelDensity
from sklearn.svm import OneClassSVM


from sklearn.metrics import accuracy_score, roc_auc_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture

from scipy.spatial.distance import cdist

def plot_cv_supervised_split(X, y, task, n_epochs=100, num_folds=10, classifier_type="svm", do_clustering=True):
    X = X.astype(np.float32)
    y = y.astype(np.float32)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    class BottleneckMLP(nn.Module):
        def __init__(self, input_size):
            super(BottleneckMLP, self).__init__()
            dims = [32, 16]
            p = 0.5
            self.encoder = nn.Sequential(
                nn.BatchNorm1d(input_size),
                nn.Linear(input_size, dims[0]),
                nn.BatchNorm1d(dims[0]),
                nn.ReLU(),
                nn.Dropout(p),
                nn.Linear(dims[0], dims[1]),
                nn.BatchNorm1d(dims[1]),
                nn.ReLU(),
                nn.Dropout(p),
                nn.Linear(dims[1], 2),
                nn.Dropout(p),
            )
            self.classifier = nn.Sequential(
                nn.ReLU(),
                nn.Linear(2, 1)
            )

        def forward(self, x, return_features=False):
            z = self.encoder(x)
            if return_features:
                return z
            return torch.sigmoid(self.classifier(z))

    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
    
    fig, axes = plt.subplots(int(num_folds/5), 5, figsize=(25/(10/num_folds), 10))
    axes = axes.flatten()

    accuracies = []
    aucs = []
    per_class_recalls = []

    ################################################################## Feature Extraction

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = BottleneckMLP(input_size=X.shape[1]).to(device)
        X_train_tensor = torch.tensor(X_train).to(device)
        y_train_tensor = torch.tensor(y_train).unsqueeze(1).to(device)

        torch.manual_seed(0)
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
        criterion = nn.BCELoss()

        model.train()
        for epoch in range(n_epochs):
            optimizer.zero_grad()
            output = model(X_train_tensor)
            loss = criterion(output, y_train_tensor)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            Z_train = model(torch.tensor(X_train).to(device), return_features=True).cpu().numpy()
            Z_test = model(torch.tensor(X_test).to(device), return_features=True).cpu().numpy()

        Z_all = np.vstack([Z_train])
        y_all = np.hstack([y_train])

        ################################################################## Classifier
        if classifier_type == "svm":
            one = (y_all == 1).astype(float).sum()
            zero = (y_all == 0).astype(float).sum()
            w = (one/zero * 3.5)
            # print(w)
            clf = SVC(kernel="rbf", class_weight={0: w, 1: 1}, probability=True)
            clf.fit(Z_all, y_all)
            
            delta = w/50
            # print(delta)
            # delta = 1.
            decision_scores = clf.decision_function(Z_test)
            y_pred = (decision_scores >= delta).astype(np.float32)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]

        elif classifier_type == "knn":
            clf = KNeighborsClassifier(n_neighbors=5)
            clf.fit(Z_all, y_all)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]
            y_pred = (y_pred_proba >= 0.5).astype(np.float32)

        elif classifier_type == "cluster":
            # Manually compute class means (centers)
            center_0 = Z_train[y_train == 0].mean(axis=0)
            center_1 = Z_train[y_train == 1].mean(axis=0)
            centers = np.stack([center_0, center_1])

            # Predict each test point based on closest center
            dists = cdist(Z_test, centers)
            closest = np.argmin(dists, axis=1)
            y_pred = closest.astype(np.float32)  # class 0 or 1

            # Crude probability: inverse normalized distance
            y_pred_proba = 1 - np.min(dists, axis=1) / (np.max(dists) + 1e-8)

        elif classifier_type == "lda":
            clf = LinearDiscriminantAnalysis()
            clf.fit(Z_all, y_all)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]
            y_pred = (y_pred_proba >= 0.5).astype(np.float32)

        elif classifier_type == "gnb":
            clf = GaussianNB()
            clf.fit(Z_all, y_all)
            y_pred_proba = clf.predict_proba(Z_test)[:, 1]
            y_pred = (y_pred_proba >= 0.5).astype(np.float32)

        elif classifier_type == "gmm":
            # Fit one GMM per class
            gmm_0 = GaussianMixture(n_components=1).fit(Z_train[y_train == 0])
            gmm_1 = GaussianMixture(n_components=1).fit(Z_train[y_train == 1])
            
            # Compute log-likelihood for each class
            log_p0 = gmm_0.score_samples(Z_test)
            log_p1 = gmm_1.score_samples(Z_test)
            
            y_pred = (log_p1 > log_p0).astype(np.float32)
            # Normalize scores for pseudo-probabilities
            y_pred_proba = 1 / (1 + np.exp(log_p0 - log_p1))

        elif classifier_type == "kde":
            kde0 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(Z_train[y_train == 0])
            kde1 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(Z_train[y_train == 1])
        
            log_p0 = kde0.score_samples(Z_test)
            log_p1 = kde1.score_samples(Z_test)
        
            y_pred = (log_p1 > log_p0).astype(np.float32)
            y_pred_proba = 1 / (1 + np.exp(log_p0 - log_p1))

        elif classifier_type == "kde_anomaly":
            kde = KernelDensity(kernel='gaussian', bandwidth=0.2)
            kde.fit(Z_train[y_train == 1])  # train only on class 1
        
            # Get density scores for class 1 training data to set threshold
            log_density_pos_train = kde.score_samples(Z_train[y_train == 1])
            threshold = np.percentile(log_density_pos_train, 10)  # 90% inclusion
        
            # Score test data
            log_density_test = kde.score_samples(Z_test)
            y_pred = (log_density_test >= threshold).astype(np.float32)
        
            # Normalize for crude probability (optional)
            y_pred_proba = (log_density_test - log_density_test.min()) / (log_density_test.max() - log_density_test.min() + 1e-8)

        elif classifier_type == "svm_anomaly":
            clf = OneClassSVM(kernel='poly', gamma='scale', nu=0.1)
            clf.fit(Z_train[y_train == 1])  # train only on class 1
        
            y_pred_raw = clf.predict(Z_test)  # returns +1 (inlier), -1 (outlier)
            y_pred = (y_pred_raw == 1).astype(np.float32)
        
            # You can optionally assign a fake "confidence" score (not probability)
            y_pred_proba = (clf.decision_function(Z_test) - clf.decision_function(Z_test).min()) / (
            clf.decision_function(Z_test).max() - clf.decision_function(Z_test).min() + 1e-8)

        
        else:
            raise ValueError(f"Unsupported classifier_type: {classifier_type}")

        ################################################################## Metrics
        acc = accuracy_score(y_test, y_pred)
        try:
            auc = roc_auc_score(y_test, y_pred_proba)
        except:
            auc = np.nan
        recalls = recall_score(y_test, y_pred, average=None, labels=[0, 1])

        accuracies.append(acc)
        aucs.append(auc)
        per_class_recalls.append(recalls)

        ################################################################## Plotting
        ax = axes[fold]

        def plot_points(Zf, labels, name, marker, alpha):
            for label in [0, 1]:
                idx = labels == label
                ax.scatter(Zf[idx, 0], Zf[idx, 1], label=f"{name} Class {int(label)}", marker=marker, alpha=alpha)

        plot_points(Z_train, y_train, "Train", "o", 0.1)
        plot_points(Z_test, y_test, "Test", "*", 1)

        if do_clustering and classifier_type == "cluster":
            ax.scatter(centers[:, 0], centers[:, 1], c='black', marker='X', s=150, label='Class Centers')
        
        x_min, x_max = Z_all[:, 0].min() - 1, Z_all[:, 0].max() + 1
        y_min, y_max = Z_all[:, 1].min() - 1, Z_all[:, 1].max() + 1
        xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                             np.linspace(y_min, y_max, 300))
        grid = np.c_[xx.ravel(), yy.ravel()]

        if classifier_type == "cluster":
            # Plot class centers
            ax.scatter(centers[:, 0], centers[:, 1], c='black', marker='X', s=150, label='Class Centers')
        
            # Decision boundary on meshgrid
            dists = cdist(grid, centers)  # shape: (N_grid_points, 2)
            cluster_pred = np.argmin(dists, axis=1).astype(float)
            Z_grid = cluster_pred.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "svm":
            
            Z_grid = clf.decision_function(grid).reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[delta], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "lda":
            Z_grid = clf.predict_proba(grid)[:, 1].reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "gnb":
            Z_grid = clf.predict_proba(grid)[:, 1].reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "gmm":
            log_p0 = gmm_0.score_samples(grid)
            log_p1 = gmm_1.score_samples(grid)
            Z_grid = 1 / (1 + np.exp(log_p0 - log_p1))  # pseudo-prob
            Z_grid = Z_grid.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')


        if classifier_type == "kde":
            log_p0_grid = kde0.score_samples(grid)
            log_p1_grid = kde1.score_samples(grid)
            Z_grid = 1 / (1 + np.exp(log_p0_grid - log_p1_grid))
            Z_grid = Z_grid.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "kde_anomaly":
            log_density_grid = kde.score_samples(grid)
            Z_grid = (log_density_grid >= threshold).astype(float)
            Z_grid = Z_grid.reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0.5], colors='black', linewidths=1.5, linestyles='--')

        if classifier_type == "svm_anomaly":
            Z_grid = clf.decision_function(grid).reshape(xx.shape)
            ax.contour(xx, yy, Z_grid, levels=[0], colors='black', linewidths=1.5, linestyles='--')

        ax.set_title(f"Fold {fold + 1}")
        ax.set_xlabel("Comp 1")
        ax.set_ylabel("Comp 2")
        ax.grid(True)

    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right')
    fig.suptitle(f"Task {task}: 10-Fold Bottleneck + {classifier_type.upper()}", fontsize=18)
    plt.tight_layout(rect=[0, 0, 0.98, 0.95])
    # plt.show()

    mean_accuracy = np.mean(accuracies)
    mean_auc = np.nanmean(aucs)
    mean_per_class_recall = np.mean(np.vstack(per_class_recalls), axis=0)

    print(f"Task {task} - 10-Fold Results ({classifier_type.upper()}):")
    print(f"  Average Accuracy: {mean_accuracy:.4f}")
    print(f"  Average AUC:      {mean_auc:.4f}")
    print(f"  Recall Class 0:   {mean_per_class_recall[0]:.4f}")
    print(f"  Recall Class 1:   {mean_per_class_recall[1]:.4f}")
    return mean_auc, mean_per_class_recall[0], mean_per_class_recall[1]

# for i, f in enumerate((load_1, load_2, load_3, load_4, load_5, load_6, load_7, load_8)):
for i, f in zip([4, 7, 8], (load_4, load_7, load_8)):
    X, y = f(exclude_hiv=False)
    plot_cv_supervised_split(
        X, 
        y, 
        n_epochs=150, 
        num_folds=10, task=i, 
        classifier_type="svm")



In [ ]:
lst_no_hiv = [
    [0.6036, 0.4389, 0.5449],
    [0.5159, 0.7304, 0.6791],
    [0.1000, 0.8179, 0.3679],
    # [0.4000, 0.9094, 0.7911],
    [0.1000,0.9133,0.6132],
    [0.7014, 0.4827, 0.6430],
    [0.6000, 0.7099, 0.7056],
    [0.1000, 0.8554, 0.2339],
    [0.4500, 0.9136, 0.7784]
]

In [ ]:
lst = [[0.4300,0.7278,0.6697],
      [0.3242,0.8451,0.6775],
       [0.0000, 0.8946, 0.3232],
       [0.0000,0.8933, 0.6065],
       [0.5005,0.7221,0.6625],
       [0.4273,0.8498,0.7193],
       [0.0000,0.9482,0.2982],
       [0.3500,0.9569,0.8002]
      ] 

# Above is with hiv

In [ ]:
a = pd.read_csv('machine_learning_summary.csv').drop('Unnamed: 0', axis=1)
a['CV AUC-SVM-no-HIV'] = np.array(l)[:, 1]
a['CV AUC-NC-no-HIV'] = np.array(l)[:, 0]

a['CV NEG_ACC-SVM-no-HIV'] = np.array(l)[:, 3]
a['CV POS_ACC-SVM-no-HIV'] = np.array(l)[:, 5]

a['CV NEG_ACC-NC-no-HIV'] = np.array(l)[:, 2]
a['CV POS_ACC-NC-no-HIV'] = np.array(l)[:, 4]
a['CV POS_ACC-MLP-no-HIV'] = np.array(lst_no_hiv)[:, 1]
a['CV NEG_ACC-MLP-no-HIV'] = np.array(lst_no_hiv)[:, 0]
a['CV AUC-MLP-no-HIV'] = np.array(lst_no_hiv)[:, 2]

a['CV POS_ACC'] = np.array(lst)[:, 1]
a['CV NEG_ACC'] = np.array(lst)[:, 0]
a['CV AUC-MLP'] = np.array(lst)[:, 2]

for c in df_metrics:
    if 'Ada' in c:
        continue
    a[c] = df_metrics[c]
# a
# a = a.drop(['CV AUC'], axis=1)
a.to_csv('machine_learning_model_summary.csv', index=False)

In [ ]:
def plot_vae(X, y, task, ax):
    X = X.astype(np.float32)
    y = y.astype(np.float32)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    class VAE(nn.Module):
        def __init__(self, input_size):
            super(VAE, self).__init__()
            # Encoder
            self.fc1 = nn.Linear(input_size, 32)
            self.bn1 = nn.BatchNorm1d(32)
    
            self.fc2 = nn.Linear(32, 16)
            self.bn2 = nn.BatchNorm1d(16)
    
            self.fc31 = nn.Linear(16, 2)  # Mean
            self.fc32 = nn.Linear(16, 2)  # Log variance
    
            # Decoder
            self.fc4 = nn.Linear(2, 16)
            self.bn4 = nn.BatchNorm1d(16)
    
            self.fc5 = nn.Linear(16, 32)
            self.bn5 = nn.BatchNorm1d(32)
    
            self.fc6 = nn.Linear(32, input_size)
    
        def encode(self, x):
            h1 = torch.relu(self.bn1(self.fc1(x)))
            h2 = torch.relu(self.bn2(self.fc2(h1)))
            return self.fc31(h2), self.fc32(h2)
    
        def reparameterize(self, mu, logvar):
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return mu + eps * std
    
        def decode(self, z):
            h4 = torch.relu(self.bn4(self.fc4(z)))
            h5 = torch.relu(self.bn5(self.fc5(h4)))
            return self.fc6(h5)
    
        def forward(self, x):
            mu, logvar = self.encode(x)
            z = self.reparameterize(mu, logvar)
            return self.decode(z), mu, logvar


    def loss_function(recon_x, x, mu, logvar):
        recon_loss = nn.functional.mse_loss(recon_x, x, reduction='sum')
        # KL divergence
        kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return recon_loss + kl

    X_tensor = torch.tensor(X).to(device)
    torch.manual_seed(0)
    model = VAE(input_size=X.shape[1]).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-2)

    model.train()
    
    for epoch in range(400):
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(X_tensor)
        loss = loss_function(recon_batch, X_tensor, mu, logvar)
        loss.backward()
        optimizer.step()

    # Use mean of latent distribution as embedding
    model.eval()
    with torch.no_grad():
        mu, _ = model.encode(X_tensor)
        features_2d = mu.cpu().numpy()

    for label in np.unique(y):
        idx = y == label
        ax.scatter(features_2d[idx, 0], features_2d[idx, 1], label=f"Class {int(label)}", alpha=0.7)

    ax.set_xlabel("Latent Dim 1")
    ax.set_ylabel("Latent Dim 2")
    ax.set_title(f"VAE Latent Space Task {task}")
    ax.legend()
    ax.grid(True)


fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = axes.flatten()
for f, t, ax in zip(
    (load_3, load_4, load_7, load_8), (3, 4, 7, 8), axes):
    X, y = f()
    plot_vae(X, y, t, ax)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class Autoencoder(nn.Module):
    def __init__(self, input_dim=111, hidden_dim=64, latent_dim=16):
        super().__init__()
        # Encode
        # p = 0.2
        self.encoder = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim),
            nn.BatchNorm1d(latent_dim)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon




X, y = load_7()

set_seed(42)
dataset = TensorDataset(torch.from_numpy(X[y.astype(bool)]).float())
loader = DataLoader(dataset, batch_size=32, shuffle=True)
set_seed(42)
model = Autoencoder(hidden_dim=32, latent_dim=16)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=9e-2)
smallest = 1e5
neg_X = X[~y.astype(bool)]
pos_X = X[y.astype(bool)]

print(neg_X.shape[0], pos_X.shape[0])
for epoch in range(1000):
    total_loss = 0
    for (batch,) in loader:
        model.train()
        optimizer.zero_grad()
        recon = model(batch)
        loss = criterion(recon, batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    # if total_loss < 0.03:
    if total_loss < smallest:
        smallest = total_loss
        criterion_eval = nn.MSELoss(reduction='none')

        with torch.no_grad():
            model.eval()
            y_orig = torch.from_numpy(neg_X).float()
            recon = model(y_orig)
            neg_loss = criterion_eval(recon, y_orig).mean(1)
            
            y_orig = torch.from_numpy(pos_X).float()
            recon = model(y_orig)
            pos_loss = criterion_eval(recon, y_orig).mean(1)

T = 0.4
plt.xlim(0, 2.1)

y_jitter = np.random.normal(0, 0.01, size=len(neg_loss))
plt.scatter(neg_loss.numpy() * 1, y=np.zeros(len(neg_loss)), label='Low Responders')

y_jitter = np.random.normal(0, 0.01, size=len(pos_loss))
plt.scatter(pos_loss.numpy() * 1, np.zeros(len(pos_loss)), alpha=0.2, label='High Responders')
plt.axvline(x=T, color='red', linestyle='--', linewidth=2, label=f'Threshold at T={T}')
plt.xlabel('Reconstruction loss with AE')

a, b = (neg_loss > T).float().mean(), (pos_loss < T).float().mean()
plt.title(f'Clf accuracy: (High Resp: {b.cpu().numpy():.3f}, Low Resp: {a.cpu().numpy():.2f})')
plt.legend()

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

# === Load your dataset ===
X, y = load_7()  # Replace with your actual loader

def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# === Autoencoder definition ===
class Autoencoder(nn.Module):
    def __init__(self, input_dim=111, hidden_dim=64, latent_dim=16):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim),
            nn.BatchNorm1d(latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
        )

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

# === Plot reconstruction loss per fold ===
def plot_fold_reconstruction_losses(recon_loss, y_true, threshold, fold_num):
    pos_loss = recon_loss[y_true == 0]
    neg_loss = recon_loss[y_true == 1]

    a = (neg_loss > threshold).mean()
    b = (pos_loss < threshold).mean()

    plt.figure(figsize=(8, 3))
    plt.xlim(0, 15)
    plt.ylim(-0.05, 0.05)

    y_jitter = np.random.normal(0, 0.0, size=len(neg_loss))
    plt.scatter(neg_loss, y_jitter, label='Anomalies (Neg)', alpha=0.7)

    y_jitter = np.random.normal(0, 0.0, size=len(pos_loss))
    plt.scatter(pos_loss, y_jitter, label='Normals (Pos)', alpha=0.2)

    plt.axvline(x=threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold = {threshold:.2f}')
    plt.xlabel('Reconstruction Loss')
    plt.title(f'Fold {fold_num+1} | Acc: Normals={b:.2f}, Anomalies={a:.2f}')
    plt.legend()
    plt.tight_layout()
    plt.show()

# === Run one fold ===
def run_fold(X_train, X_test, y_test, fold_num):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Autoencoder(input_dim=X_train.shape[1], hidden_dim=32, latent_dim=16).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    train_loader = DataLoader(
        TensorDataset(torch.from_numpy(X_train).float()),
        batch_size=32,
        shuffle=True
    )

    smallest_loss = float("inf")
    best_model = None

    for epoch in range(1000):
        model.train()
        epoch_loss = 0.0
        for (batch,) in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            recon = model(batch)
            loss = criterion(recon, batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        if epoch_loss < smallest_loss:
            smallest_loss = epoch_loss
            # Save best model
            best_model = model.state_dict()

    # Reload best model
    model.load_state_dict(best_model)
    model.eval()

    with torch.no_grad():
        # === Compute threshold on train set ===
        train_tensor = torch.from_numpy(X_train).float().to(device)
        train_recon = model(train_tensor)
        train_loss = nn.MSELoss(reduction='none')(train_recon, train_tensor).mean(dim=1).cpu().numpy()
        threshold = np.percentile(train_loss, 95)

        # === Evaluate on test set ===
        test_tensor = torch.from_numpy(X_test).float().to(device)
        test_recon = model(test_tensor)
        recon_loss = nn.MSELoss(reduction='none')(test_recon, test_tensor).mean(dim=1).cpu().numpy()
        y_pred = (recon_loss > threshold).astype(int)
        y_true = (y_test == 0).astype(int)

    tp = np.sum((y_pred == 1) & (y_true == 1))
    tn = np.sum((y_pred == 0) & (y_true == 0))
    fp = np.sum((y_pred == 1) & (y_true == 0))
    fn = np.sum((y_pred == 0) & (y_true == 1))

    pos_acc = tn / (tn + fp + 1e-8)
    neg_acc = tp / (tp + fn + 1e-8)
    overall_acc = (tp + tn) / len(y_true)

    plot_fold_reconstruction_losses(recon_loss, y_true, threshold, fold_num)

    return {
        'accuracy': overall_acc,
        'pos_accuracy': pos_acc,
        'neg_accuracy': neg_acc,
        'threshold': threshold,
        'recon_loss': recon_loss,
        'y_true': y_true,
        'y_pred': y_pred
    }

# === Cross-validation ===
X_pos = X[y == 1]
y_pos = y[y == 1]
X_neg = X[y == 0]
y_neg = y[y == 0]

skf_pos = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
neg_chunks = np.array_split(np.arange(len(X_neg)), 10)

results = []

for fold, ((pos_train_idx, pos_test_idx), neg_idx) in enumerate(zip(skf_pos.split(X_pos, y_pos), neg_chunks)):
    print(f"\nFold {fold + 1}")

    X_train = X_pos[pos_train_idx]
    X_test_pos = X_pos[pos_test_idx]
    y_test_pos = y_pos[pos_test_idx]

    X_test_neg = X_neg[neg_idx]
    y_test_neg = y_neg[neg_idx]

    X_test = np.vstack([X_test_pos, X_test_neg])
    y_test = np.concatenate([y_test_pos, y_test_neg])

    result = run_fold(X_train, X_test, y_test, fold_num=fold)
    print(f"  Accuracy: {result['accuracy']:.3f}")
    print(f"  Normal Accuracy (positive): {result['pos_accuracy']:.3f}")
    print(f"  Anomaly Accuracy (negative): {result['neg_accuracy']:.3f}")
    results.append(result)

# === Summary ===
avg_acc = np.mean([r['accuracy'] for r in results])
avg_pos_acc = np.mean([r['pos_accuracy'] for r in results])
avg_neg_acc = np.mean([r['neg_accuracy'] for r in results])

print("\n====== Cross-Validation Summary ======")
print(f"Average Accuracy: {avg_acc:.3f}")
print(f"Average Normal Accuracy (positive class): {avg_pos_acc:.3f}")
print(f"Average Anomaly Accuracy (negative class): {avg_neg_acc:.3f}")


In [ ]:
T = 0.5

y_jitter = np.random.normal(0, 0.01, size=len(neg_loss))
plt.scatter(neg_loss.numpy() * 1000, y=np.zeros(len(neg_loss)), label='Low Responders')

y_jitter = np.random.normal(0, 0.01, size=len(pos_loss))
plt.scatter(pos_loss.numpy() * 1000, np.zeros(len(pos_loss)), alpha=0.2, label='High Responders')
plt.axvline(x=T*1000, color='red', linestyle='--', linewidth=2, label=f'Threshold at T={T*1000}')


a, b = (neg_loss > T).float().mean(), (pos_loss < T).float().mean()
plt.title(f'Clf accuracy: (High Resp: {a.cpu().numpy():.2f}, Low Resp: {b.cpu().numpy():.2f})')
plt.legend()

In [ ]:
from sklearn.manifold import TSNE

from sklearn.decomposition import PCA

with torch.no_grad():
        model.eval()
        encoded = model.encoder(torch.from_numpy(X).float())

tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(encoded)

import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.scatter(X_tsne[:,0], X_tsne[:,1], c=y)
plt.title("t-SNE on Hidden Representations")
plt.colorbar(label="Class")
plt.savefig('leidos_tsne_ae.png')
plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


class VAE(nn.Module):
    def __init__(self, input_dim=121, hidden_dim=64, latent_dim=16):
        super().__init__()
        # Encoder: input -> mean and log-variance
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
        )
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

        # Decoder: latent -> output
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)  # sample from N(0,1)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        recon = self.decoder(z)
        return recon, mu, logvar

def vae_loss(recon_x, x, mu, logvar):
    recon_loss = nn.functional.mse_loss(recon_x, x, reduction='sum')
    kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kl_div




In [ ]:
# Load your data
def vae(config):
    X, y = load_xy_B0_healthy_control()
    
    # Convert to tensor if needed
    if not isinstance(X, torch.Tensor):
        X = torch.tensor(X, dtype=torch.float32)
    
    # Dataset
    dataset = TensorDataset((X[y.astype(bool)]))
    set_seed(42)
    loader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    # 🚀 Train VAE
    set_seed(42)
    model = VAE(hidden_dim=config['hidden_dim'], latent_dim=config['latent_dim'])
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=config['lr'])

    best = 1e8
    for epoch in range(config['epochs']):
        model.train()
        total_loss = 0
        for (batch,) in loader:
            optimizer.zero_grad()
            recon, mu, logvar = model(batch)
            loss = vae_loss(recon, batch, mu, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            best = min(best, total_loss)
        # print(f"Epoch {epoch+1:02d}, Loss: {total_loss:.4f}")
    
    # 🔍 Latent vectors (mean)
    with torch.no_grad():
        model.eval()
        _, mu, _ = model(X)

    return best

In [ ]:
hidden = [84, 64, 32]
latent = [84, 64, 32]
lr = [1e-3]

list_of = []
for h, la, l in product(hidden, latent, lr):
    config = {
        'hidden_dim': h,
        'latent_dim': la,
        'lr': l,
        'epochs': 1000
    }
    loss = vae(config)
    list_of.append((loss, config))
    print(loss, config)

In [ ]:
X, y = load_xy_B0_healthy_control()

# Convert to tensor if needed
if not isinstance(X, torch.Tensor):
    X = torch.tensor(X, dtype=torch.float32)

# Dataset
dataset = TensorDataset((X[y.astype(bool)]))

loader = DataLoader(dataset, batch_size=32, shuffle=True)

# 🚀 Train VAE
model = VAE(hidden_dim=84, latent_dim=84)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

best = 1e8
for epoch in range(1000):
    model.train()
    total_loss = 0
    for (batch,) in loader:
        optimizer.zero_grad()
        recon, mu, logvar = model(batch)
        loss = vae_loss(recon, batch, mu, logvar)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        # best = min(best, total_loss)
    if total_loss < 1520:
        break

In [ ]:
X, y = load_xy_B0_healthy_control()

neg_X = X[~y.astype(bool)]
pos_X = X[y.astype(bool)]

criterion = nn.MSELoss(reduction='none')

with torch.no_grad():
    model.eval()
    y_orig = torch.from_numpy(neg_X).float()
    recon = model(y_orig)[0]
    neg_loss = criterion(recon, y_orig).mean(1)
    
    y_orig = torch.from_numpy(pos_X).float()
    recon = model(y_orig)[0]
    pos_loss = criterion(recon, y_orig).mean(1)

neg_loss

y_jitter = np.random.normal(0, 0.01, size=len(neg_loss))
y = np.zeros_like(neg_loss.numpy())
plt.scatter(neg_loss.numpy(), y, alpha=1, label='negative')

y_jitter = np.random.normal(0, 0.01, size=len(pos_loss))
y = np.zeros_like(pos_loss.numpy())
plt.scatter(pos_loss.numpy(), y, label='positive', alpha=0.1)
plt.xlabel('Reconstruction Loss')
plt.axvline(x=0.25, color='red', linestyle='--', label='x = 0.25')


plt.legend()


In [ ]:


all_ = pos_loss.shape[0] + neg_loss.shape[0]
(neg_loss > 0.25).float().mean(), (pos_loss < 0.25).float().mean(), ((neg_loss > 0.25).float().mean() * neg_loss.shape[0]/all_ + (pos_loss < 0.25).float().mean() * pos_loss.shape[0]/all_)


In [ ]:
from sklearn.manifold import TSNE

from sklearn.decomposition import PCA

# pca = PCA(n_components=50, random_state=42)
# X_reduced = pca.fit_transform(X_reduced)  # shape becomes (n_samples, 50)

X, y = load_xy_B0_healthy_control()
with torch.no_grad():
        model.eval()
        _, mu, _ = model(torch.from_numpy(X).float())
tsne = TSNE(n_components=2, perplexity=30, learning_rate='auto', random_state=42)
X_tsne = tsne.fit_transform(mu)  # or X_896 if skipping PCA

import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.scatter(X_tsne[:,0], X_tsne[:,1], c=y)
plt.title("t-SNE on Hidden Representations")
plt.colorbar(label="Class")
plt.savefig('leidos_tsne_vae.png')
plt.show()


## Anomaly

In [ ]:
import torch
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.linalg import inv

# Assume:
# mu: torch.Tensor of shape (n_samples, latent_dim)
# y: labels as torch.Tensor or NumPy array

# Convert if needed
if isinstance(y, torch.Tensor):
    y = y.cpu().numpy()

mu_np = mu.cpu().numpy()
mu_pos = mu_np[y == 1]   # Positive class
mu_neg = mu_np[y == 0]   # Negative class

# Compute mean and covariance from positives
mu_mean = mu_pos.mean(axis=0)
mu_cov = np.cov(mu_pos.T)
mu_cov_inv = inv(mu_cov)  # For Mahalanobis distance


In [ ]:
# Function to compute Mahalanobis distance from the positive class distribution
def mahalanobis_batch(X, mean, cov_inv):
    return np.array([mahalanobis(x, mean, cov_inv) for x in X])

dists_pos = mahalanobis_batch(mu_pos, mu_mean, mu_cov_inv)
dists_neg = mahalanobis_batch(mu_neg, mu_mean, mu_cov_inv)


In [ ]:
import matplotlib.pyplot as plt

plt.hist(dists_pos, bins=30, alpha=0.6, label="Positive", color="green")
plt.hist(dists_neg, bins=30, alpha=0.6, label="Negative", color="red")
plt.axvline(np.percentile(dists_pos, 95), color='black', linestyle='--', label="95% cutoff")
plt.title("Mahalanobis Distances from Positive Distribution")
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.legend()
plt.show()


In [ ]:
threshold = np.percentile(dists_pos, 95)
anomaly_flags_neg = dists_neg > threshold
print(f"Out of {len(mu_neg)} negative samples, {anomaly_flags_neg.sum()} were flagged as anomalies.")
